In [16]:
################## DEFINIZIONI FUNZIONI E VARIABILI UTILI ###################
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from astropy.io import fits
from scipy.optimize import curve_fit
import os
import re
from os.path import exists, isfile, join
import subprocess
import time
import urllib.request
import urllib.parse
import json
import logging
import multiprocessing
from numpy import linspace, round as around
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sympy as sp
import sys
import random

####
# path_to_pythonEnv = f"/home/starcat/Scrivania/ANALISIATM/.venv/bin/python"
# path_to_linemake = f"/home/starcat/LINEE"
# path_to_ARES = f"/home/starcat/Modelli/ARES-master"
# path_to_abundance = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c/"
# path_to_abundance_0 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_0/"
# path_to_abundance_1 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_1/"
# path_to_abundance_2 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_2/"
# path_to_mainfolder = f"/home/starcat/Scrivania/ANALISIATM/"
# path_to_atmosphere = f"/home/starcat/Scrivania/ANALISIATM/ATMOSFERE/"
# path_to_rnd = f"/home/starcat/Scrivania/ANALISIATM/rnd"
# path_to_output = f"/home/starcat/Scrivania/ANALISIATM/ABBONDANZE_OUT/"
# path_to_interpol = f"/home/starcat/Scrivania/ANALISIATM/interpol/PyKMOD-main/"
# path_to_correctionGrids = f"/home/starcat/Scrivania/ANALISIATM/correctionGrids/"
# path_to_correctionGrids1 = f"/home/starcat/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/"
# path_to_nlteInterpol = f"/home/starcat/Scrivania/ANALISIATM/1L-3NErrors-main/"
# path_to_calibrationFolder = f"/home/starcat/Scrivania/ANALISIATM/Calibrazione"
# path_to_fitsFiles = f"/home/starcat/Documenti/Reduced_AOT41/Reduced_AOT41" #path_to_ARES

path_to_linemake = f"/home/starcat8/Scrivania/ANALISIATM/LINEE"
path_to_ARES = f"/home/starcat8/Scrivania/ARES-master"
path_to_abundance = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c/"
path_to_abundance_0 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_0/"
path_to_abundance_1 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_1/"
path_to_abundance_2 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_2/"
path_to_mainfolder = f"/home/starcat8/Scrivania/ANALISIATM/"
path_to_atmosphere = f"/home/starcat8/Scrivania/ANALISIATM/ATMOSFERE/"
path_to_rnd = f"/home/starcat8/Scrivania8/ANALISIATM/rnd"
path_to_output = f"/home/starcat8/Scrivania/ANALISIATM/ABBONDANZE_OUT/"
path_to_interpol = f"/home/starcat8/Scrivania/ANALISIATM/interpol/PyKMOD-main/"
path_to_correctionGrids = f"/home/starcat8/Scrivania/ANALISIATM/correctionGrids/"
path_to_correctionGrids1 = f"/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/"
path_to_nlteInterpol = f"/home/starcat8/Scrivania/ANALISIATM/1L-3NErrors-main/"
path_to_calibrationFolder = f"/home/starcat8/Scrivania/ANALISIATM/Calibrazione"
path_to_pythonEnv = f"/home/starcat8/Scrivania/ANALISIATM/.venv/bin/python"
path_to_fitsFiles = f"/home/starcat8/Documenti/Reduced_AOT41"
####


NEW_MATRIX = np.array([[-2.73752381e-04,  2.22800000e-03,  1.25531429e-03,  3.30684003e-03],
            [-3.53622447e-02, -4.34489796e-01, -1.34193878e-01, -6.97848163e+00],
            [-1.50857143e-02, -4.02285714e-01, -1.15394286e+00, -2.93710716e+00],
            [-4.89264069e-03, -2.08303030e-01, -4.36747186e-01, -2.23545660e+01]])
MATRIX = NEW_MATRIX.T
#crea_MatriceRicerca(T, G, M, v): #il fatto che alcune grandezze richiedano un fit quadratico implica una dipendenza dalle grandezze stesse.

N_stelle = 14
trials = 30
temp_log = np.zeros(trials)
logg_log = np.zeros(trials)
T_slope = np.zeros((trials, 2))
Logg_slope = np.zeros((trials, 2))
temp_best_guess1 = 0
temp_best_guess2 = 0
temp_best_guess3 = 0
temperatura = [3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000, 6250, 6500, 6750, 7000, 7250, 7500, 7750, 8000, 8250, 8500, 8750, 9000, 9250, 9500, 9750, 10000, 10250, 10500, 10750, 11000, 11250, 11500, 11750, 12000, 12250, 12500, 12750, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000]
logg_arr = [0, 0.50000, 1.00000, 1.50000, 2.00000, 2.50000, 3.00000, 3.50000, 4.00000, 4.50000, 5.00000]
LOGG_arr = [1.50000, 2.00000, 2.50000, 3.00000]
MH = ["05", "02", "00", "-05", "-10", "-15", "-20"]
metallicity = ["0.5", "0.2", "0.0", "-0.5", "-1.0", "-1.5", "-2.0", "-2.5"]
metallic = [0.5, 0.2, 0.0, -0.5, -1.0, -1.5, -2.0, -2.5]
METAL = [4.25, 4.50, 4.75, 5.00, 5.25, 5.50, 5.75, 6.00, 6.25, 6.50, 6.75, 7.00, 7.25, 7.50, 7.75]
metallic_header = [-2.5, 2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5]
T_EFF_int = np.arange(3500, 5500 + 1, 25)
LOGG_int = np.arange(0.5, 5 + 0.06, 0.05)
MH_int = np.arange(0.0, -2.5 - 0.06, -0.05)
T_EFF_Nint = np.arange(3500, 6000 , 250)
LOGG_Nint = np.arange(0.5, 5 + 0.06, 0.5)
MH_Nint = np.arange(0.0, -2.5 - 0.06, -0.5)
MICROTURB = [0.75, 1.5, 3.0]
ABUNDANCE_SCALE = [3.16228, 1.58489, 1.00000, 0.31620, 0.10000, 0.03162, 0.01000, 0.00316]
ABUNDANCE_SCALE_header = [0.00316, 0.01000, 0.03162, 0.10000, 0.31620, 1.00000, 1.58489, 3.16228]
lineList = "Iron_lines"
minceData = [["HARPN1", 4746, 1.89, -0.56],
             ["HARPN2", 4610, 1.73, -0.58],
             ["HARPN3", 4458, 1.54, -0.56],
             ["HARPN4", 4045, 1.21, -0.64],
             ["HARPN5", 4444, 4,-0.5],
             ["HARPN6", 4722, 1.89, 0.02],
             ["HARPN7", 4725, 2.41, -0.19],
             ["HARPN8", 4444, 4, -0.5],
             ["HARPN9", 4458, 1.54, -0.56],
             ["HARPN10", 4393, 1.08, -1.99],
             ["HARPN11", 4193, 0.71, -1.95],
             ["HARPN12", 4167, 0.82, -1.45],
             ["HARPN13", 4154, 0.52, -2.09],
             ["HARPN14", 4287, 0.70, -1.71],
             ["HARPN15", 4316, 0.87, -1.46],
             ["HARPN16", 4276, 0.70, -2.27],
             ["HARPN17", 4249, 0.79, -1.61]]
NLTE_wavelenght = [3440.610, 
                   436.886,
            3440.990,
            3465.860,
            3475.440,
            3490.570,
            3565.380,
            3570.100,
            3581.190,
            3608.860,
            3618.770,
            3719.920,
            3737.120,
            3727.620,
            3745.560,
            3748.260,
            3758.230,
            3763.790,
            3815.840,
            3820.430,
            3824.440,
            3825.870,
            3840.440,
            3856.370,
            3859.910,
            3922.910,
            4045.810,
            4063.590,
            4071.740,
            4235.940,
            4250.120,
            4250.790,
            4415.120,
            4445.470,
            4494.560,
            4920.500,
            4994.130,
            5044.210,
            5198.700,
            5216.270,
            5225.520,
            5232.940,
            5236.200,
            5242.490,
            5247.040,
            5250.210,
            5269.540,
            5281.790,
            5379.570,
            5383.370,
            5434.521,
            5491.830,
            5586.750,
            5600.220,
            5661.350,
            5662.520,
            5696.090,
            5701.540,
            5705.460,
            5778.450,
            5784.660,
            5855.080,
            5916.250,
            5956.690,
            6065.479,
            6082.710,
            6151.620,
            6173.330,
            6200.310,
            6219.280,
            6240.650,
            6252.560,
            6265.130,
            6297.790,
            6311.500,
            6430.850,
            6498.940,
            6518.370,
            6574.229,
            6593.870,
            6609.110,
            6699.140,
            6726.670,
            6739.521,
            6750.150,
            6793.260,
            6810.260,
            6837.010,
            6854.820,
            6945.200,
            6978.850,
            7401.680,
            7912.870,
            8293.500,
            4233.170,
            4491.390,
            4508.290,
            4576.330,
            4582.840,
            4583.830,
            4620.520,
            4923.930,
            5018.440,
            5169.020,
            5197.580,
            5234.620,
            5264.810,
            5284.110,
            5325.550,
            5414.070,
            5425.260,
            6239.950,
            6247.560,
            6369.460,
            6432.680,
            6456.380,
            6516.080,
            7222.390,
            7224.490,
            7515.830,
            7711.720]
nomi_fileFits = ["HARPN.2020-05-10T21-18-51.039_s1d_A.fits",
"HARPN.2020-05-10T22-12-22.186_s1d_A.fits",
"HARPN.2020-05-10T23-06-53.246_s1d_A.fits",
"HARPN.2020-05-11T00-59-19.289_s1d_A.fits",
"HARPN.2020-05-11T02-03-44.593_s1d_A.fits",
"HARPN.2020-05-11T03-06-48.862_s1d_A.fits",
"HARPN.2020-05-11T03-59-13.184_s1d_A.fits",
"HARPN.2020-05-11T04-36-55.934_s1d_A.fits",
"HARPN.2020-05-11T20-54-51.023_s1d_A.fits",
"HARPN.2020-06-09T21-12-39.990_s1d_A.fits",
"HARPN.2020-06-09T22-15-01.629_s1d_A.fits",
"HARPN.2020-06-09T23-18-01.289_s1d_A.fits",
"HARPN.2020-06-10T00-12-36.574_s1d_A.fits",
"HARPN.2020-06-10T01-16-38.066_s1d_A.fits",
"HARPN.2020-06-10T01-39-10.823_s1d_A.fits",
"HARPN.2020-06-10T02-23-08.791_s1d_A.fits",
"HARPN.2020-06-10T03-26-11.908_s1d_A.fits"]
class Star:
    def __init__(self, name, T, G, M, v):
        self.name = name
        self.T = int(T)
        self.G = float(G)
        self.M = float(M)
        self.v = float(v)
def def_s_Matrix():
    vv=sp.symbols('v')
    dsdv1=-0.015639
    dFdv1=0.1194805
    dYdv1=0.2560606
    dSdv1=1.7464315
    dsdv2=0.0242265
    dFdv2=-0.445342
    dYdv2=-0.939681
    dSdv2=-26.17745
    dsdt=-0.000273752
    dFdt=0.002228
    dYdt=0.001255
    dSdt=0.003307
    dsdg=-0.035362
    dFdg=-0.434498
    dYdg=-0.134194
    dSdg=-6.978472
    dsdm=-0.015086
    dFdm=-0.402286
    dYdm=-1.153942
    dSdm=-2.937107
    s_Matrix=sp.Matrix([[dsdt, dsdg, dsdm, dsdv1*vv+dsdv2],
                        [dFdt, dFdg, dFdm, dFdv1*vv+dFdv2],
                        [dYdt, dYdg, dYdm, dYdv1*vv+dYdv2],
                        [dSdt, dSdg, dSdm, dSdv1*vv+dSdv2]])
    return s_Matrix
s_Matrix=def_s_Matrix()
# Funzione per bloccare le stampe a schermo
def blocca_stampe():
    sys.stdout = open(os.devnull, 'w')
    sys.stderr = open(os.devnull, 'w')
# Funzione per riattivare le stampe a schermo
def crea_dynMatrix(x1, y1, x2, y2): #
    '''Matrice delle derivate, ma creata step by step.\n
      In x ci va (T, G, M, V), in y (s, F, Y, S).\n
      Restituisce la matrice che deve essere invertita per il calcolo della variazione dei parametri.'''
    # print(len(y1))
    # print(len(x1))
    # print(len(y2))
    # print(len(x2))
    # print(y1)
    # print(x1)
    # print(y2)
    # print(x2)
    if len(y1)== 4 and len(x1) == 4 and len(y2)== 4 and len(x2) == 4:
        print("AIEEEEEEEIROA!")
        dsdt=(y2[0]-y1[0])/(x2[0]-x1[0])
        dsdg=(y2[0]-y1[0])/(x2[1]-x1[1])
        dsdm=(y2[0]-y1[0])/(x2[2]-x1[2])
        dsdv=(y2[0]-y1[0])/(x2[3]-x1[3])
        dFdt=(y2[1]-y1[1])/(x2[0]-x1[0])
        dFdg=(y2[1]-y1[1])/(x2[1]-x1[1])
        dFdm=(y2[1]-y1[1])/(x2[2]-x1[2])
        dFdv=(y2[1]-y1[1])/(x2[3]-x1[3])
        dYdt=(y2[2]-y1[2])/(x2[0]-x1[0])
        dYdg=(y2[2]-y1[2])/(x2[1]-x1[1])
        dYdm=(y2[2]-y1[2])/(x2[2]-x1[2])
        dYdv=(y2[2]-y1[2])/(x2[3]-x1[3])
        dSdt=(y2[3]-y1[3])/(x2[0]-x1[0])
        dSdg=(y2[3]-y1[3])/(x2[1]-x1[1])
        dSdm=(y2[3]-y1[3])/(x2[2]-x1[2])
        dSdv=(y2[3]-y1[3])/(x2[3]-x1[3])
    else:
        print("Errore nel fornire y!")
    matrice = np.array([
    [dsdt, dsdg, dsdm, dsdv],
    [dFdt, dFdg, dFdm, dFdv],
    [dYdt, dYdg, dYdm, dYdv],
    [dSdt, dSdg, dSdm, dSdv]
    ])
    return matrice
def crea_dynMatrix1(x1, y1, x2, y2):
    """
    Constructs the derivative matrix step by step.
    Inputs:
        x1, x2: lists of independent variable values [T, G, M, V].
        y1, y2: lists of dependent variable values [s, F, Y, S].
    Output:
        A 4x4 matrix of derivatives.
    """
    if len(y1) == 4 and len(x1) == 4 and len(y2) == 4 and len(x2) == 4:
        print("AIEEEEEEEIROA!")
        try:
            dsdt = (y2[0] - y1[0]) / (x2[0] - x1[0])
            dsdg = (y2[0] - y1[0]) / (x2[1] - x1[1])
            dsdm = (y2[0] - y1[0]) / (x2[2] - x1[2])
            dsdv = (y2[0] - y1[0]) / (x2[3] - x1[3])
            
            dFdt = (y2[1] - y1[1]) / (x2[0] - x1[0])
            dFdg = (y2[1] - y1[1]) / (x2[1] - x1[1])
            dFdm = (y2[1] - y1[1]) / (x2[2] - x1[2])
            dFdv = (y2[1] - y1[1]) / (x2[3] - x1[3])
            
            dYdt = (y2[2] - y1[2]) / (x2[0] - x1[0])
            dYdg = (y2[2] - y1[2]) / (x2[1] - x1[1])
            dYdm = (y2[2] - y1[2]) / (x2[2] - x1[2])
            dYdv = (y2[2] - y1[2]) / (x2[3] - x1[3])
            
            dSdt = (y2[3] - y1[3]) / (x2[0] - x1[0])
            dSdg = (y2[3] - y1[3]) / (x2[1] - x1[1])
            dSdm = (y2[3] - y1[3]) / (x2[2] - x1[2])
            dSdv = (y2[3] - y1[3]) / (x2[3] - x1[3])
        except ZeroDivisionError as e:
            print(f"Error: Division by zero encountered! Details: {e}")
            return None
    else:
        print("Errore nel fornire y!")
        return None

    matrice = np.array([
        [dsdt, dsdg, dsdm, dsdv],
        [dFdt, dFdg, dFdm, dFdv],
        [dYdt, dYdg, dYdm, dYdv],
        [dSdt, dSdg, dSdm, dSdv]
    ])
    return matrice
def nome_da_Fits(path_to_fits=path_to_fitsFiles, nome='HARPN1.fits'):
    # Apri il file FITS
    hdul = fits.open(path_to_fits+"/"+nome)

    # Stampa informazioni sul file
    #hdul.info()

    # Accedi all'header della prima estensione
    header = hdul[0].header

    # Stampa l'header completo (opzionale)
    #print(repr(header))

    # Se sai il nome della keyword che contiene il nome della stella, puoi accedere direttamente
    # Ad esempio, supponiamo che il nome della stella sia memorizzato sotto la keyword 'OBJECT'
    nome_stella = header.get('HIERARCH TNG OBS TARG NAME', 'Nome della stella non trovato')

    print(f'Nome della stella: {nome_stella}')

    # Chiudi il file FITS
    hdul.close()
    return nome_stella
def riattiva_stampe():
    sys.stdout = sys.__stdout__
    sys.stderr = sys.__stderr__
def crea_cartella(percorso):
    try:
        # Controlla se la cartella esiste già
        if not os.path.exists(percorso):
            os.makedirs(percorso)  # Crea la cartella
            print(f"Cartella '{percorso}' creata con successo!")
        else:
            print(f"La cartella '{percorso}' esiste già.")
    except Exception as e:
        print(f"Errore durante la creazione della cartella: {e}")
def crea_lista_primo_elemento(file_path):
    # Lista per memorizzare il primo elemento di ogni riga
    lista_primi_elementi = []

    with open(file_path, 'r') as file:
        for line in file:
            # Dividi la riga in segmenti separati da spazi o altri caratteri di spaziatura
            valori = line.strip().split()  # Usa split(',') se i valori sono separati da virgole
            if valori:
                # Aggiungi il primo elemento della riga alla lista
                lista_primi_elementi.append(float(valori[0]))

    return lista_primi_elementi
def trova_dodicesimo_elemento(file_path, parametro1, parametro2, parametro3, parametro4, tolleranza1=125, tolleranza2=0.25, tolleranza3=0.125, tolleranza4 = 0.01, tolleranza5 = 0.0001):
    '''file_path è il path al file contenente le correzioni;
     parametri: temperatura, logg, metallicità, wavelenght, microturbolenza\n Restituisce una lista con i valori più vicini alla griglia: \n 0: correzione \n 1: T \n 2: G \n 2: M \n 3: v \n 4: WL'''
    U = find_closest(vturbo(parametro1, parametro2, parametro3-7.5), MICROTURB)
    # Compiliamo una regex per identificare ogni segmento non vuoto separato da spazi variabili
    pattern = re.compile(r'\S+')
    stampa = False
    with open(file_path, 'r') as file:
        for line in file:
            # Trova tutti i segmenti della riga che corrispondono al pattern
            valori = pattern.findall(line)
            if len(valori) < 12:
                continue  # Salta righe con meno di 12 elementi
            try:
                # Estrai e converti i primi cinque valori
                primo_valore = float(valori[0])
                secondo_valore = float(valori[1])
                terzo_valore = float(valori[2])
                quarto_valore = float(valori[3])
                # print(vturbo(parametro1, parametro2, parametro3-7.5))
                # print(abs(quarto_valore - U) <= tolleranza5)
                quinto_valore = float(valori[5])*10
                # print(f"{primo_valore}   {secondo_valore}   {terzo_valore}   {quarto_valore}   {quinto_valore} \n {parametro1}  {parametro2}  {parametro3}  {parametro4} ")
            except ValueError:
                continue  # Salta la riga se i primi tre valori non sono numerici
            # Controlla se i valori soddisfano le tolleranze specificate
            #print(f"{primo_valore} {secondo_valore} {terzo_valore} {quarto_valore} {quinto_valore} {line}")
            if parametro2 <=1.5:
                parametro2 = 1.5
            if stampa == True:
                print(f"{abs(primo_valore - parametro1) <= tolleranza1}  {primo_valore} - {parametro1}")
                print(f"{abs(secondo_valore - parametro2) <= tolleranza2} {secondo_valore}-{parametro2}")
                print(f"{abs(terzo_valore - parametro3) <= tolleranza3}   {terzo_valore} - {parametro3}")
                print(f"{abs(quarto_valore - U) <= tolleranza5}    {quarto_valore} - {U}")
                print(f"{abs(quinto_valore - parametro4) <= tolleranza4}    {quinto_valore} - {parametro4}")
            if (abs(primo_valore - parametro1) <= tolleranza1 and   #temperatura
                abs(secondo_valore - parametro2) <= tolleranza2 and #logg
                abs(terzo_valore - parametro3) <= tolleranza3 and   #metal
                abs(quarto_valore - U) <= tolleranza5 and           #vturb
                abs(quinto_valore - parametro4) <= tolleranza4):    #wl
                #print("AAAAAAIAAAAAAAAAAA")
                # Restituisci il dodicesimo elemento se la riga soddisfa le condizioni
                returner = [float(valori[11]), valori[10], primo_valore, secondo_valore, terzo_valore, quarto_valore, parametro4]
                #print(returner)
                return returner  # l'indice 11 è il dodicesimo elemento (indice 0-based)
    # Se nessuna riga soddisfa i criteri, restituisce None
    return None
def findLine(file_path, parametro1, parametro2, parametro3, parametro4, parametro5, tolleranza1=125, tolleranza2=0.25, tolleranza3=0.125, tolleranza4 = 0.005, tolleranza5 = 0.0001):
    '''file_path è il path al file contenente le correzioni;
     parametri: temperatura, logg, metallicità, wavelenght\n Restituisce una lista con i valori più vicini alla griglia: \n 0: correzione \n 1: T \n 2: G \n 2: M \n 3: v \n 4: WL'''
    #U = find_closest(vturbo(parametro1, parametro2, parametro3-7.5), MICROTURB)
    # Compiliamo una regex per identificare ogni segmento non vuoto separato da spazi variabili
    pattern = re.compile(r'\S+')
    stampa = False
    print(f"Ciao!  {parametro1}  -    {parametro2}  -  {parametro3}  -  {parametro4}  -  {parametro5}  -")
    if parametro5 > 3:
        parametro5 = 3
        print("Non ci sono punti sulla griglia con microturbolenza maggiore di 3! Microturbolenza impostata a 3")
    if parametro5 < 0.75:
        parametro5 = 0.75
        print("Non ci sono punti sulla griglia con microturbolenza inferiore a 0.75! Microturbolenza impostata a 0.75")
    if parametro2 < 1.5:
        parametro2 = 1.5
        print("Non ci sono punti sulla griglia con logg inferiore a 1.5! logg impostato a 1.5")
    if parametro1 < 4000:
        parametro1 = 4000
        print("Non ci sono punti sulla griglia con temperatura inferiore a 4000! Teff impostato a 4000")
    if parametro1 > 5500:
        parametro1 = 5500
        print("Non ci sono punti sulla griglia con temperatura superiore a 5500! Teff impostato a 5500")
    if parametro2 <= 1.75 and parametro1 > 4500:
        parametro1 = 4500
    elif parametro2 <= 1.75 and parametro1 < 4000: 
        parametro1 = 4000
    if parametro2 >= 2.75 and parametro1 < 4500:
        parametro1 = 4500 
    with open(file_path, 'r') as file:
        for line in file:
            # Trova tutti i segmenti della riga che corrispondono al pattern
            valori = pattern.findall(line)
            if len(valori) < 12:
                continue  # Salta righe con meno di 12 elementi
            try:
                # Estrai e converti i primi cinque valori
                primo_valore = float(valori[0])
                secondo_valore = float(valori[1])
                terzo_valore = float(valori[2])
                quarto_valore = float(valori[3])
                # print(vturbo(parametro1, parametro2, parametro3-7.5))
                # print(abs(quarto_valore - U) <= tolleranza5)
                quinto_valore = float(valori[5])*10
                # print(f"{primo_valore}   {secondo_valore}   {terzo_valore}   {quarto_valore}   {quinto_valore} \n {parametro1}  {parametro2}  {parametro3}  {parametro4} ")
            except ValueError:
                continue  # Salta la riga se i primi tre valori non sono numerici
            # Controlla se i valori soddisfano le tolleranze specificate
            #print(f"{primo_valore} {secondo_valore} {terzo_valore} {quarto_valore} {quinto_valore} {line}")
            if parametro5 > 1.5 and parametro5 < 3.0:
                tolleranza5 = 0.75
            elif parametro5 > 0.75 and parametro5 < 1.5:
                tolleranza5 = 0.375
            elif parametro5 == 0.75:
                tolleranza5 = 0.0001
            elif parametro5 == 1.5:
                tolleranza5 = 0.0001
            elif parametro5 == 3:
                tolleranza5 = 0.0001
            if stampa == True:
                print(f"{abs(primo_valore - parametro1) <= tolleranza1}  {primo_valore} - {parametro1}")
                print(f"{abs(secondo_valore - parametro2) <= tolleranza2} {secondo_valore}-{parametro2}")
                print(f"{abs(terzo_valore - parametro3) <= tolleranza3}   {terzo_valore} - {parametro3}")
                print(f"{abs(quarto_valore - parametro5) <= tolleranza5}    {quarto_valore} - {parametro5}")
                print(f"{abs(quinto_valore - parametro4) <= tolleranza4}    {quinto_valore} - {parametro4}")
            if (abs(primo_valore - parametro1) <= tolleranza1 and   #temperatura
                abs(secondo_valore - parametro2) <= tolleranza2 and #logg
                abs(terzo_valore - parametro3) <= tolleranza3 and   #metal
                abs(quinto_valore - parametro4) <= tolleranza4 and    #wl
                abs(quarto_valore-parametro5) <= tolleranza5):
                #print("AAAAAAIAAAAAAAAAAA")
                #print("ASSHASSHA")
                # Restituisci il dodicesimo elemento se la riga soddisfa le condizioni
                returner = [float(valori[11]), valori[10], primo_valore, secondo_valore, terzo_valore, quarto_valore, parametro4]
                return returner
    # Se nessuna riga soddisfa i criteri, restituisce None
    return None
def stampaOra():
    current_time = time.localtime()
    formatted_time = time.strftime("%H:%M:%S", current_time)
    print("L'ora corrente è:", formatted_time)
def correzione(M):
    #return -0.15-0.075*M
    return 0
def findBestAtmosphereinFolder(path, mode, attiva_tolleranza=True, tolleranzaG=0.2, tolleranzaT=0.2, elemento1=26.0, elemento2=26.1, requiredT=0, requiredG=0, excludeT=0, excludeG=0): #tolleranza permette di tener di conto di un'eventuale discrepanza tra l'abbondanza di FeI e FeII (una temperatura può essere più accurata anche se la differenza tra abbondanza di FeI e FeII è maggiore di quella ottenuta per un altro logg e temperatura). Non attivare se si ha solo 1 elemento
    """ output:
     mode T: elem1, slope, std, temp, logg, elem2
     mode G: FeI, FeII, G, M
     mode A: FeI FeII T G M s(FeI) stdFeI stdFeII"""
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    os.chdir(old_dir)
    if mode=="T":
        if requiredG==0:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if (abs(H[i][2]-H[i][1]) < tolleranzaG) and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
                else:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        else:
            returner = np.zeros(6)
            for i in range(len(a)):
                if a[i][0]=='T':
                    b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
            min = 10
            j=0
            for i in range(len(b)):
                tempS = simpleAnalysisSingle(path, b[i], 26.0)
                tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
                H[i][0] = tempS[1]
                H[i][1] = float(tempD[0])
                H[i][2] = float(tempD[1])
                H[i][3] = (parseFileName(path, b[i]))[1]
                H[i][4] = (parseFileName(path, b[i]))[2]
                H[i][5] = (parseFileName(path, b[i]))[0]
                H[i][6] = tempS[2]
                if attiva_tolleranza:
                    if abs(H[i][2]-H[i][1]) < tolleranzaG and (abs(H[i][0]) < abs(min)) and H[i][4]==requiredG:
                        min = H[i][0]
                        j=i
                else:
                    if (abs(H[i][0]) < abs(min)):
                        min = H[i][0]
                        j=i
            returner[0]=H[j][1] #el1
            returner[1]=H[j][0] #slope
            returner[2]=H[j][6] #std
            returner[3]=H[j][3] #temp
            returner[4]=H[j][4] #logg
            returner[5]=H[j][2] #el2
            

            return returner
        
    elif mode=="G":
        returner = np.zeros(4)
        for i in range(len(a)):
            if a[i][0]=='G':
                b.append(a[i])
            H = np.zeros((len(b), 7)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            tempD = simpleAnalysisDouble(path, b[i], elemento1, elemento2)
            H[i][0] = tempS[1]
            H[i][1] = float(tempD[0])
            H[i][2] = float(tempD[1])
            H[i][3] = (parseFileName(path, b[i]))[1]
            H[i][4] = (parseFileName(path, b[i]))[2]
            H[i][5] = (parseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaT and (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
            else:
                if (abs(H[i][0]) < abs(min)):
                    min = H[i][0]
                    j=i
        returner[0]=H[j][1]
        returner[1]=H[j][2]
        returner[2]=H[j][4]
        returner[3]=H[j][5]
        return returner
    elif mode=="A":
        returner = np.zeros(8)
        for i in range(len(a)): #and (a[i][0]!='l' and a[i][0] != 'e'):
            b.append(a[i])
        H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
        min = 10
        j=0
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            H[i][3] = (sparseFileName(path, b[i]))[1]
            H[i][4] = (sparseFileName(path, b[i]))[2]
            H[i][5] = (sparseFileName(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            if attiva_tolleranza:
                if abs(H[i][2]-H[i][1]) < tolleranzaG and abs(H[i][0]) < tolleranzaT and abs(H[i][5]-H[i][1]) < 0.6 and H[i][6] < 0.2 and H[i][0] < min:
                    min = H[i][0]
                    j=i
            # else:
            #     if (abs(H[i][0]) < abs(min)):
            #         min = H[i][0]
            #         j=i
        returner[0]=H[j][1] 
        returner[1]=H[j][2] 
        returner[2]=H[j][3]
        returner[3]=H[j][4]
        returner[4]=H[j][5]
        returner[5]=H[j][0]
        returner[6]=H[j][6]
        returner[7]=H[j][7]
        return returner
    else:
        print("mode deve essere 'T' oppure 'G', per valutare le rispettivamente le abbondanze dedotte muovendosi su temperatura o logg")
        return None
def parseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = f"{nome_file[0]}"+r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName2(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'AH_STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName3(path, nome_file, name_suffix='BD+062880'):
    olddir=os.getcwd()
    os.chdir(path)
    #pattern = name_suffix + r'_INT(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(\d+\.\d+)'
    pattern = name_suffix + r'_(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    print("nome_file:")
    print(nome_file)
    print(pattern)
    print(os.getcwd())
    if match:
        returner.append(float(match.group(1)))
        print(returner[0])
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        returner.append(float(match.group(4)))
        #returner.append(1)
        return returner
    else:
        return None
def sparseFileName4(path, nome_file, name_suffix='BD+062880_'):
        os.chdir(path)
        #pattern = name_suffix + r'_INT(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(\d+\.\d+)'
        pattern = name_suffix + r'(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(\d+\.\d+)'
        match = re.match(pattern, nome_file)
        returner = []
        if match:
            returner.append(float(match.group(1)))
            returner.append(int(match.group(2)))
            returner.append(float(match.group(3)))
            returner.append(float(match.group(4)))
            return returner
        else:
            return None
def make_set(T, G, M, DT=500, DG=1, DM=1):
    set1 = [[T-DT, G+DG, M-DM], [T, G+DG, M-DM], [T+DT, G+DG, M-DM], [T-DT, G, M-DM]]
    set2 = [[T, G, M-DM], [T+DT, G, M-DM], [T-DT, G-DG, M-DM], [T, G-DG, M-DM]]
    set3 = [[T+DT, G-DG, M-DM],[T-DT, G+DG, M], [T, G+DG, M], [T+DT, G+DG, M]]
    set4 = [[T-DT, G, M], [T, G, M], [T+DT, G, M], [T-DT, G-DG, M]]
    set5 = [[T, G-DG, M], [T+DT, G-DG, M], [T-DT, G+DG, M+DM], [T, G+DG, M+DM]]
    set6 = [[T+DT, G+DG, M+DM], [T-DT, G, M+DM], [T, G, M+DM], [T+DT, G, M+DM]]
    set7 = [[T-DT, G-DG, M+DM], [T, G-DG, M+DM], [T+DT, G-DG, M+DM]]
    SET_ALL = [set1, set2, set3, set4, set5, set6, set7]
    return SET_ALL
def searchPatch(nome_stella, T=4750, G=1.50, M=-1.00, DT=500, DG=1, DM=1):
    SET_ALL = make_set(T-250, G, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T-250, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
    SET_ALL = make_set(T, G+0.5, M-0.5)
    for i in range(7):
        if i < 6:
            findAbundancePatch(SET_ALL[i], nome_stella)
        if i == 6:
            findAbundancePatch(SET_ALL[i], nome_stella, 3)
def findAbundancePatch(atmoParameters, nome_stella, n_istanze=4, path_to_output=path_to_output, label='', multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    preparaAbundance(nome_stella)
    bashcmd = f"mkdir {path_to_output}{nome_stella}"
    os.system(bashcmd)
    copia("stellaFes", path_to_abundance, path_to_abundance_0, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_1, "stellaFes")
    copia("stellaFes", path_to_abundance, path_to_abundance_2, "stellaFes")
    if n_istanze == 1:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]))  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 2:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 3:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    if n_istanze == 4:
        copia(f"atmoT{int(atmoParameters[0][0])}{atmoParameters[0][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[0][2]}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[0][0], atmoParameters[0][1], atmoParameters[0][2]), 0)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[1][0])}{atmoParameters[1][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[1][2]}", path_to_abundance_0, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[1][0], atmoParameters[1][1], atmoParameters[1][2]), 1)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[2][0])}{atmoParameters[2][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[2][2]}", path_to_abundance_1, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[2][0], atmoParameters[2][1], atmoParameters[2][2]), 2)  #fai paramfile di abundance
        copia(f"atmoT{int(atmoParameters[3][0])}{atmoParameters[3][1]}.txt", path_to_atmosphere + f"MH{atmoParameters[3][2]}", path_to_abundance_2, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
        creaParamfileAbnd(vturbo(atmoParameters[3][0], atmoParameters[3][1], atmoParameters[3][2]), 3)  #fai paramfile di abundance
        SlanciaAbundance(n_istanze)                                  #lancia abundance
        temp = path_to_output
        path_to_output = path_to_output + f"{nome_stella}"
        sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{atmoParameters[0][2]}_{int(atmoParameters[0][0])}_{atmoParameters[0][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS1", path_to_abundance_0, path_to_output, f"{label}STELLA{atmoParameters[1][2]}_{int(atmoParameters[1][0])}_{atmoParameters[1][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS2", path_to_abundance_1, path_to_output, f"{label}STELLA{atmoParameters[2][2]}_{int(atmoParameters[2][0])}_{atmoParameters[2][1]}") #copia l'output di abundance nella cartella degli output
        sposta("PINOLOS3", path_to_abundance_2, path_to_output, f"{label}STELLA{atmoParameters[3][2]}_{int(atmoParameters[3][0])}_{atmoParameters[3][1]}") #copia l'output di abundance nella cartella degli output
        path_to_output=temp
    # if multiple_elements == 0:
    #     results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    # else:
    #     results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    # return results
def iterative_Search(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def linear_func(x, a, b):
    return b +a*x
def quad_func(x, a, b, c): #ax^2+bx+c
    return c +b*x + a*x*x
def expo_func(x, a, b):
    return a * np.exp(b*x)
def crea_ATMinterpol(T, logg, MH_g, nome_Stella, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if n_istanze == 1:
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        #print(f"{T}   {logg}   {vturb}   {MH_g}")
        commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance()
        gnome_stella = nome_da_Fits(nome=nome_Stella)
        nome_output = f"{gnome_stella}_{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return nome_output
def crea_ATMinterpol2(T, logg, MH_g, microTurb, nome_Stella, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if processore==0:
        if n_istanze == 1:
            #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
            vturb=microTurb
            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            #preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturb)
            SlanciaAbundance()
            nome_output = f"{nome_Stella}_{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
            #comando_bsh2 = f"mv PINOLOS {path_to_calibrationFolder}/{nome_output}"# calibrazione
            comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}" # ricerca
            os.system(comando_bsh2)
            return nome_output
        if n_istanze == 2:
            ##### prima atmosfera
            #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
            print(vturb)
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturbo(T, logg, MH_g))
            ##### seconda atmosfera
            vturb0 = 0.14 - 0.08 * (MH_g0) + 4.9 * T / 10000 - 0.47 * logg0

            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T0} {logg0} {vturb0} {MH_g0}"
            print(vturb)
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T0}.  GRAVITY {logg0} LTE\nTITLE  [{MH_g0}] VTURB={vturb0}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g0):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturbo(T, logg, MH_g))
            SlanciaAbundance(2)
            nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
            comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
            os.system(comando_bsh2)
            return lista_nome_output
    else:
        #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        for i in range(processore):
            vturb=microTurb[i]
            #print(f"{type(T)}   {type(logg)}   {type(MH_g)}   {type(vturb)}")
            os.chdir(path_to_abundance)
            password = "Birdos"
            commandInterpol = f"sudo {path_to_pythonEnv} {path_to_interpol}pykmod.py {T} {logg} {vturb} {MH_g}"
            # Usa subprocess per passare la password a sudo
            process = subprocess.run(
                f"echo {password} | sudo -S {commandInterpol}",
                shell=True,
                text=True,  # Necessario per trattare input/output come stringhe
                capture_output=True  # Per catturare stdout e stderr
            )
            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            MH_g=float(MH_g)
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()
                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]
                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)
            #print(f"aa {os.getcwd()}  ehiehi")
            os.chdir(path_to_interpol)
            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/Model{i+1}"
            os.system(comandoATMOS)
            creaParamfileAbnd(vturb, i+1)
        os.chdir(path_to_abundance)
        if __name__ == "__main__":
            main_integration()
        crea_cartella(path_to_calibrationFolder + "/" + nome_Stella)
        for i in range(processore):
            vturb=microTurb[i]
            nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
            copia("stellaFes", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, "stellaFes")
            sposta(f"PINOLOS{i+1}", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, nome_output)
        return nome_output
def sposta(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"mv {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def copia(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"cp {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def getTail(T, L, M):
    OLD_DIR = os.getcwd()
    file_path = path_to_atmosphere + f"MH{M:.1f}/atmoT{int(T)}{L:.1f}.txt"
    # Leggi l'ultima riga del file
    with open(file_path, 'r') as file:
        last_line = file.readlines()[-2]
    # Estrai i numeri dalla riga usando una regex
    numbers = re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', last_line)
    # Converti i numeri estratti in float
    numbers = [float(num) for num in numbers]
    numbers[0]

    if numbers[1] == 0.0:
        grandezza = "E+00"
    if numbers[1] < 0:
        grandezza = f"E-0{int(abs(numbers[1]))}"
    if numbers[1] > 0:
        grandezza = f"E+0{int(numbers[1])}"

    returner = f"{numbers[0]}" + grandezza
    os.chdir(OLD_DIR)
    return returner
def vturbo(T, LG, Met):
    return 0.14 - 0.08 * (Met) + 4.9 * T / 10000 - 0.47 * LG
def creaParamfileAbnd(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    if numero_abundance==0:
        with open(f'param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance, 'param.txt')
    else:
        with open(f'param{numero_abundance}.txt', 'w') as paramfile:                #{numero_abundance}
            sposta(f'param{numero_abundance}.txt', os.getcwd(), path_to_abundance, f'param{numero_abundance}.txt')#f'param{numero_abundance}.txt')
def creaParamfileAbnd1(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    with open(f'param{numero_abundance}.txt', 'w') as paramfile:                #{numero_abundance}
        paramfile.write(f'ATMname Model{numero_abundance}\nLINESname {file_linee}\nOUTname PINOLOS{numero_abundance}\nvturb {str(microturb)}')
        sposta(f'param{numero_abundance}.txt', os.getcwd(), path_to_abundance, f'param{numero_abundance}.txt')#f'param{numero_abundance}.txt')
def SlanciaAbundance(n_istanze=1):
    if (n_istanze >= 1) and (n_istanze <= 4): 
        if n_istanze==1:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            process1.wait()
            os.chdir(DIR_INI)
        if n_istanze==2:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            os.chdir(DIR_INI)
        if n_istanze==3:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            os.chdir(DIR_INI)
        if n_istanze==4:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_2)
            process4 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            process3.wait()
            os.chdir(DIR_INI)
    else:
        print(f"Numero di istanze assegnate DEVE essere compreso tra 1 e 4 inclusi. Inserito {n_istanze}")
        return None
def indici_elementiuguali(lista1, lista2):
    indici = [i for i, (x, y) in enumerate(zip(lista1, lista2)) if x == y]
    return indici
def folderLog(path, mode=1):
    '''Crea una file di testo log.txt contenente i principali risultati dell'analisi atmosferica.
    folderLog2 stampa anche le quantità necessarie alla determinazione della microturbolenza. 
    Restituisce una lista con i parametri delle atmosfere vincenti.'''
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    returner = np.zeros(8)
    for i in range(len(a)):
        if a[i] != "log.txt" and a[i] != "exit":
            b.append(a[i]) #in b i nomi delle stelle
    H = np.zeros((len(b), 8)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std
    min = 10
    j=0
    with open("log.txt", 'w') as f:
        f.write("    T      G      M      FeI            FeII           FeI-M/H        FeI-FeII       s(FeI)    \n")
        for i in range(len(b)):
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            if mode == 1:
                H[i][3] = (sparseFileName(path, b[i]))[1]
                H[i][4] = (sparseFileName(path, b[i]))[2]
                H[i][5] = (sparseFileName(path, b[i]))[0]
            if mode == 2:
                H[i][3] = (sparseFileName2(path, b[i]))[1]
                H[i][4] = (sparseFileName2(path, b[i]))[2]
                H[i][5] = (sparseFileName2(path, b[i]))[0]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}\n")
    winners = []
    for i in range(len(b)): #erano 0.6                            0.21                    0.11
        if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
            winners.append([H[i][3], H[i][4], H[i][6], 100])
    ds=0.002
    dg=0.04
    dm=0.08
    q=1
    for u in range(100):
        for i in range(len(b)):
            if len(winners) < 4:
                if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                    if 100-5*q <= -700:
                        winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                    else:
                        winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
            q=q+1
            ds=ds+0.001
            dg=dg+0.005
            dm=dm+0.01
    os.chdir(old_dir)
    return winners
def folderLog2(path, mode=1, baseNome=''):
    '''Crea una file di testo log.txt contenente i principali risultati dell'analisi atmosferica.
    folderLog2 stampa anche le quantità necessarie alla determinazione della microturbolenza. 
    Restituisce una lista con i parametri delle atmosfere vincenti.\n
    mode (1, 2 o 3) varia a seconda del tipo di parsing del nome dei file che vogliamo analizzare.
    1 per i file che iniziano con 'STELLA', 2 per quelli che iniziano con 'AH_STELLA'. \n mode=3 è per trattare un nome qualsiasi, in questo caso si DEVE specificare anche il nome tramite la variabile 'baseNome', eg. "HARPN1.fits_INT <--- per ora non implementato, solo per uso manuale (cioè modifica la funzione sparseFile3 se vuoi cambiare)'''
    old_dir = os.getcwd()
    os.chdir(path)
    a = os.listdir()
    b = []
    returner = np.zeros(9)
    for i in range(len(a)):
        if a[i] != "log.txt" and a[i] != "exit" and a[i] != 'stellaFes':
            b.append(a[i]) #in b i nomi delle stelle
    H = np.zeros((len(b), 9)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std(FeI), H[7]=std(FeII), H[8]=slope(EWR-A(Fe))
    min = 10
    j=0
    for i in range(len(b)):
        print(b[i])
    with open("log.txt", 'w') as f:
        f.write("    T      G      M    vturb    FeI        FeII           FeI-M/H        FeI-FeII       s(FeI)    EWR\n")
        for i in range(len(b)):
            print(b[i])
            tempD = simpleAnalysisSingle(path, b[i], 26.1)
            tempS = simpleAnalysisSingle(path, b[i], 26.0)
            H[i][0] = tempS[1]
            H[i][1] = float(tempS[0])
            H[i][2] = float(tempD[0])
            if mode == 1:
                H[i][3] = (sparseFileName(path, b[i]))[1]
                H[i][4] = (sparseFileName(path, b[i]))[2]
                H[i][5] = (sparseFileName(path, b[i]))[0]
            if mode == 2:
                H[i][3] = (sparseFileName2(path, b[i]))[1]
                H[i][4] = (sparseFileName2(path, b[i]))[2]
                H[i][5] = (sparseFileName2(path, b[i]))[0]
            if mode == 3:
                H[i][3] = (sparseFileName3(path, b[i]))[1]
                H[i][4] = (sparseFileName3(path, b[i]))[2]
                H[i][5] = (sparseFileName3(path, b[i]))[0]
                vturbos = (sparseFileName3(path, b[i]))[3]
            H[i][6] = tempS[2]
            H[i][7] = tempD[2]
            f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {vturbos:.2f}    {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}        {tempS[3]:.5f}\n")
    winners = []
    for i in range(len(b)): #erano 0.6                            0.21                    0.11
        if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
            winners.append([H[i][3], H[i][4], H[i][6], 100])
    ds=0.002
    dg=0.04
    dm=0.08
    q=1
    for u in range(100):
        for i in range(len(b)):
            if len(winners) < 4:
                if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                    if 100-5*q <= -700:
                        winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                    else:
                        winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
            q=q+1
            ds=ds+0.001
            dg=dg+0.005
            dm=dm+0.01
    os.chdir(old_dir)
    return winners
def tra_le_atmosfere(path_to_output=path_to_output, nome_stella='HARPN'):
    '''la miglior atmosfera, si restituisce una lista contenente le '''
    a=[]
    # findBestAtmosphereinFolder(path_to_output+f"HARPN{I}.fits", "A")
    for I in range(1, N_stelle+1):
        a.append(folderLog(path_to_output+f"{nome_stella}{I-1}.fits"))
    return a
def intermediateATM(atmospheric_param):
    RET = []
    return_param = np.zeros(3)
    if len(atmospheric_param) == 0:
        print("nessuna atmosfera passata!")
        return None
    elif len(atmospheric_param) == 1:
        return atmospheric_param[0]
    elif len(atmospheric_param) == 2:
        for i in range(3):
            return_param[i] = (1.0*atmospheric_param[0][i]+1.0*atmospheric_param[1][i])/2
        RET.append(return_param[0])
        RET.append(return_param[1])
        RET.append(return_param[2])
        return RET
    elif len(atmospheric_param) == 3:
        A = []
        A.append((atmospheric_param[0][0]+atmospheric_param[1][0]+atmospheric_param[2][0])/3)
        A.append((atmospheric_param[0][1]+atmospheric_param[1][1]+atmospheric_param[2][1])/3)
        A.append((atmospheric_param[0][2]+atmospheric_param[1][2]+atmospheric_param[2][2])/3)
        return A
    elif len(atmospheric_param) == 4:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3])/normaliz)
        return A
    elif len(atmospheric_param) == 5:
        A = []
        normaliz = (atmospheric_param[0][3]+atmospheric_param[1][3]+atmospheric_param[2][3]+atmospheric_param[3][3]+atmospheric_param[4][3]) #media ponderata sui pesi, a seconda di quanto è buona l'atmosfera
        A.append((atmospheric_param[0][0]*atmospheric_param[0][3]+atmospheric_param[1][0]*atmospheric_param[1][3]+atmospheric_param[2][0]*atmospheric_param[2][3]+atmospheric_param[3][0]*atmospheric_param[3][3]+atmospheric_param[4][0]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][1]*atmospheric_param[0][3]+atmospheric_param[1][1]*atmospheric_param[1][3]+atmospheric_param[2][1]*atmospheric_param[2][3]+atmospheric_param[3][1]*atmospheric_param[3][3]+atmospheric_param[4][1]*atmospheric_param[4][3])/normaliz)
        A.append((atmospheric_param[0][2]*atmospheric_param[0][3]+atmospheric_param[1][2]*atmospheric_param[1][3]+atmospheric_param[2][2]*atmospheric_param[2][3]+atmospheric_param[3][2]*atmospheric_param[3][3]+atmospheric_param[4][2]*atmospheric_param[4][3])/normaliz)
        return A
    else:
        A = [4750, 1.5, -1.0]
def stampaAnalisi(path_file, nome):
    A = simpleAnalysisSingle(path_file, nome, 26.0)
    B = simpleAnalysisSingle(path_file, nome, 26.1)
    with open("exit", 'w') as f:
        f.write(f"FeI = {A[0]}+{A[2]}\nFeII = {B[0]}+{B[2]}\n s(FeI) = {A[1]}\nDF = {A[0]-B[0]}")
    sposta("exit", os.getcwd(), path_file, "exit")
def simpleAnalysisSingle(path_file, nome_file, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std e la slope EWR"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(4)
    t1 = Table.read('stellaFes', format='ascii')
    EW = t1['col8']
    WL = t1['col1']
    EWR = EW/WL
    # for i in range(len(EW)):
    #     EWR.append(EW[i] / WL[i])
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3'] #potential energy
    y=t2['col7'] #abbundanzia
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
    std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    returner[3] = popt2[0]   #slope EWR\A
    return returner
def simpleAnalysisSingle2(path_file, nome_file, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(4)
    t1 = Table.read('stellaFes', format='ascii')
    #print(os.getcwd())
    EW = t1['col8']
    WL = t1['col1']
    EWR = EW/WL
    #print(EWR)
    # for i in range(len(EW)):
    #     EWR.append(EW[i] / WL[i])
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3'] #potential energy
    y=t2['col7'] #abbundanzia
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
    std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    returner[3] = popt2[0]   #slope EWR\A
    #print(nome_file)
    return returner
def NLTE_AnalysisSingle(path_file, nome_file, T, G, M, V, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std\n Ogni riga viene corretta per gli effetti NLTE"""
    DIR_INI = os.getcwd()
    MM=M
    M=M+7.5
    os.chdir(path_to_output)
    file_path = 'stellaFes'
    lista_primi_elementi = crea_lista_primo_elemento(file_path)
    #print(lista_primi_elementi)
    correctionList = []
    PATH= path_to_correctionGrids1
    #AA=trova_dodicesimo_elemento(f"{PATH}4500-5000/correctionGrid", T, G, M, lista_primi_elementi[0])
    # print("BUB")
    # print(AA)
    # print("UBU")
    WL_disponibili=NLTE_caricaWLdisponibili()
    for i in range(len(lista_primi_elementi)):
        # PATH=path_to_correctionGrids1
        # if float(lista_primi_elementi[i]) < 5000:
        #     PATH=PATH+"4500-5000/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 5000 and lista_primi_elementi[i] < 5250:
        #     PATH=PATH+"5000-5250/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 5250 and lista_primi_elementi[i] < 5500:
        #     PATH=PATH+"5250-5500/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 5500 and lista_primi_elementi[i] < 5750:
        #     PATH=PATH+"5500-5750/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 5750 and lista_primi_elementi[i] < 6000:
        #     PATH=PATH+"5750-6000/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 6000 and lista_primi_elementi[i] < 6250:
        #     PATH=PATH+"6000-6250/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 6250 and lista_primi_elementi[i] < 6500:
        #     PATH=PATH+"6250-6500/correctionGrid"
        # elif float(lista_primi_elementi[i]) > 6500 and lista_primi_elementi[i] < 6750:
        #     PATH=PATH+"6500-6750/correctionGrid"
        # else:
        #     print("Nooo!")
        if lista_primi_elementi[i] in WL_disponibili:
            correctionList.append(NLTE_intCorrection(lista_primi_elementi[i], elemento, T, G, MM, V))
        #correctionList.append(trova_dodicesimo_elemento(f"{PATH}", T, G, M, lista_primi_elementi[i]))

        # print(f"{lista_primi_elementi[i]}   {i}    {T}   {G}     {M}")
        # print(correctionList[i])
    os.chdir(path_to_linemake)
    t=Table.read("Iron_lines3", format='ascii')
    W=t['col1']
    weight=t['col8']
    os.chdir(path_file)
    returner = np.zeros(4)
    t1 = Table.read('stellaFes', format='ascii')
    EW = t1['col8']
    WL = t1['col1']
    EWR = EW/WL
    #t1 = Table.read('stellaFes', format='ascii')
    #print(os.getcwd())
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3']
    y=t2['col7']
    X=t2['col1']
    #print(f"{len(x)}  {len(lista_primi_elementi)}")
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    X=X[limit]
    EWR=EWR[limit]
    correctionList=[el for el in correctionList if el is not None]
    for i in range(len(correctionList)):
        for j in range(len(X)):
            if X[j]==correctionList[i][6]:
                #print(y[j])
                y[j]=y[j]+correctionList[i][0]
                #print(f"{y[j]} + {correctionList[i][0]}")
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    X=X[limit]
    xx = x.tolist()
    yy=y.tolist()
    XX=X.tolist()
    weighting = 0
    if weighting == 1:
        for i in range(len(W)):
            for j in range(len(X)):
                if W[i]==X[j]:
                    if weight[i] == 'aa':
                        for k in range(20):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'am' or 'ma':
                        for k in range(14):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'ax' or 'xa':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'aA' or 'Aa':
                        for k in range(25):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'aX' or 'Xa':
                        for k in range(2):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'xx':
                        for k in range(4):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'xm' or 'mx':
                        for k in range(6):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Xx' or 'xX':
                        for k in range(1):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Ax' or 'xA':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'mm':
                        for k in range(8):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'mX' or 'Xm':
                        for k in range(5):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Am' or 'mA':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'AA':
                        for k in range(35):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'AX' or 'XA':
                        for k in range(10):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'X':
                        for k in range(1):
                            yy.append(y[j])
                            xx.append(x[j])
    xx=np.asarray(xx)
    yy=np.asarray(yy)
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
    std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    returner[3] = popt2[0]   #slope EWR\A
    #print(nome_file)
    return returner
def simpleAnalysisDouble(path_file, nome_file, elemento1=26.0, elemento2=26.1): #passa path del file, il nome, e restituisce le abbondanze di due elementi (o diverse ionizzazioni)
    """Restituisce: abbondanza el1, abbondanza el2"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(2)
    #t1 = Table.read('stellaFes', format='ascii')
    t2 = Table.read(nome_file, format='ascii')
    t3 = Table.read(nome_file, format='ascii')
    x2=t2['col3']
    y2=t2['col7']
    fe2=t2['col2']
    limit=((fe2==elemento1))
    x2=x2[limit]
    y2=y2[limit]
    limit=((y2<np.mean(y2)+1*np.std(y2))&(y2>np.mean(y2)-1*np.std(y2)))
    x2=x2[limit]
    y2=y2[limit]
    x3=t3['col3']
    y3=t3['col7']
    fe3=t3['col2']
    limit=((fe3==elemento2))
    x3=x3[limit]
    y3=y3[limit]
    limit=((y3<np.mean(y3)+1*np.std(y3))&(y3>np.mean(y3)-1*np.std(y3)))
    x3=x3[limit]
    y3=y3[limit]
    os.chdir(DIR_INI)
    returner[0] = np.mean(y2) #abbondanza elemento1
    returner[1] = np.mean(y3)+correzione(returner[0]) #abbondanza elemento2
    return returner
def findAbundance(tempera, logG, metal, label, multiple_elements=0,  elemento1=26.0, elemento2=26.1):
    """Restituisce: multiple element==0 media, slope, std, multiple element==1 media el1 e media el2"""
    copia(f"atmoT{int(tempera)}{logG}.txt", path_to_atmosphere + f"MH{metal}", path_to_abundance, "ModelE") #metti il file dell'atmosfera dentro la cartella di abundance
    creaParamfileAbnd(vturbo(tempera, logG, metal))  #fai paramfile di abundance
    SlanciaAbundance()                                    #lancia abundance
    sposta("PINOLOS", path_to_abundance, path_to_output, f"{label}STELLA{metal}_{int(tempera)}_{logG}") #copia l'output di abundance nella cartella degli output
    if multiple_elements == 0:
        results = simpleAnalysisSingle(path_to_output, f"TSTELLA{metal}_{int(tempera)}_{logG}", elemento1)
    else:
        results = simpleAnalysisDouble(path_to_output, f"GSTELLA{metal}_{int(tempera)}_{logG}", elemento1, elemento2)
    return results
def iterative_temperatureSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioTemp = np.zeros((trials, 4))
    i=0
    min = 1000
    control = 1
    while controller==1 and i < trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkG = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkG)) and control==1:
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] > 0:
        #         tempera=tempera+250
        #     if simpleAnalysisSingle(os.getcwd(), file_checkG)[1] < 0:
        #         tempera=tempera-250
        #     control = 0
        A = findAbundance(tempera, logG, metal, "T", 0, elemento)
        diarioTemp[i][0] = A[0]
        diarioTemp[i][1] = A[1]
        diarioTemp[i][2] = A[2]
        diarioTemp[i][3] = tempera
        if (i>=1) and (abs(diarioTemp[i][1])>abs(diarioTemp[i-1][1])):
            break
        metal = find_closest(A[0], metallic)
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioTemp[k][3]==diarioTemp[i][3]):
                controller = 0
        if ((tempera > 3499) and (tempera < 12750)): 
            if (diarioTemp[i][1]>0):
                tempera = tempera + 250
                tempera = int(tempera)
            else:
                if tempera > 3749:
                    tempera = tempera - 250
                    tempera = int(tempera)
        i=i+1
    if controller == 1:
        print("il numero di trials non è stato sufficiente ad ottenere un risultato, prova con un numero di trials maggiore")
        if i != 0:
            for j in range(trials):
                if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                    min = diarioTemp[j][1]
                    required_index = j
            #print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
            returner = np.zeros(4)
            returner[0] = diarioTemp[required_index][0] #media
            returner[1] = diarioTemp[required_index][1] #slope
            returner[2] = diarioTemp[required_index][2] #std
            returner[3] = diarioTemp[required_index][3] #temperatura
            return returner
    if controller == 0:
        for j in range(trials):
            if (diarioTemp[j][3] != 0) and (abs(diarioTemp[j][1]) < abs(min)): #!= 0 perché viene inizializzato con alcuni 0 che potrebbero non essere utilizzati, in tal caso ci si ferma prima. Si cerca la miglior slope, infatti si ricerca min e via via si sostituisce con la slope più bassa. Alla slope più bassa, la j-esima, corrisponde la j-esima atmosfera
                min = diarioTemp[j][1]
                required_index = j
        print(f"la temperatura {diarioTemp[required_index][3]} è il valore più vicino a quello cercato \ncorrispondente a una slope di {diarioTemp[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioTemp[required_index][0]} +- {diarioTemp[required_index][2]}")
        returner = np.zeros(4)
        returner[0] = diarioTemp[required_index][0] #media
        returner[1] = diarioTemp[required_index][1] #slope
        returner[2] = diarioTemp[required_index][2] #std
        returner[3] = diarioTemp[required_index][3] #temperatura
        return returner
def iterative_logGSearch(tempera, logG, metal, trials=8, elemento=26.0, controller=1):
    diarioLogG = np.zeros((trials, 3))
    diarioLogG = diarioLogG + 555.9
    i=0
    min = 1000
    while controller==1 and i<trials:
        print(f"trial = {i}")
        file_check = f"{path_to_output}/GSTELLA{metal}_{int(tempera)}_{logG}"
        file_checkT = f"{path_to_output}/TSTELLA{metal}_{int(tempera)}_{logG}"
        if (exists(file_check)):
            break
        # if (exists(file_checkT)):
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] > 0:
        #         logG=logG+0.5
        #     if simpleAnalysisDouble(os.getcwd(), file_checkT)[0]-simpleAnalysisDouble(os.getcwd(), file_checkT)[1] < 0:
        #         logG=logG-0.5
        outPut = findAbundance(tempera, logG, metal, "G", 1, 26.0, 26.1)
        diarioLogG[i][0] = outPut[0]
        diarioLogG[i][1] = outPut[1]
        diarioLogG[i][2] = logG
        required_index = 0
        for k in range(trials):
            if (k!=i) and (diarioLogG[k][2]==diarioLogG[i][2]):
                controller = 0
        if ((logG > -0.1) and (logG < 5.1)): 
            if (diarioLogG[i][0] - diarioLogG[i][1] > 0):
                logG = logG + 0.5
            else:
                if logG > 0.4:
                    logG = logG - 0.5
        print(f"{diarioLogG[i][0]-diarioLogG[i][1]} con {diarioLogG[i][2]}")
        i=i+1
    if controller == 1:
        if i != 0:
            for j in range(i):
                if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                    min = (diarioLogG[j][0]-diarioLogG[j][1])
                    required_index = j
            print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
            returner = np.zeros(3)
            returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
            returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
            returner[2] = diarioLogG[required_index][2] #logG cercata
            return returner
    if controller == 0:
        for j in range(i):
            if abs(diarioLogG[j][1]-diarioLogG[j][0]) < abs(min):
                min = (diarioLogG[j][0]-diarioLogG[j][1])
                required_index = j
        print(f"il logG {diarioLogG[required_index][2]} è il valore più vicino a quello cercato \ncorrispondente a elemento 26.1 {diarioLogG[required_index][1]} \nl'abbondanza dell'elemento {elemento} è di {diarioLogG[required_index][0]}")
        returner = np.zeros(3)
        returner[0] = diarioLogG[required_index][0] #abbondanza elemento 1
        returner[1] = diarioLogG[required_index][1] #abbondanza elemento 2
        returner[2] = diarioLogG[required_index][2] #logG cercata
        return returner                                 #Iron_lines#al posto di molto_linee per il ferro
def creaOptFileARES(specfits='Stello.fits', readlinedat='Iron_lines', fileout='stella.ares', lambdai="4600.", lambdaf="6700.", smoothder="4", space="3.0", rejt="1;5764,5766,6047,6052,6068,6076", lineresol="0.1", miniline="2", plots_flag="0", rvmask="3,6021.8,6024.06,6027.06,6024.06,20"):
    os.chdir(path_to_ARES)
    copia(specfits, path_to_fitsFiles, path_to_ARES, specfits)
    with open('mine.opt', 'w') as paramfile:
        paramfile.write(f'specfits=\'{specfits}\'\nreadlinedat=\'{readlinedat}\'\nfileout=\'{fileout}\'\nlambdai={lambdai}\nlambdaf={lambdaf}\nsmoothder={smoothder}\nspace={space}\nrejt={rejt}\nlineresol={lineresol}\nminiline={miniline}\nplots_flag={plots_flag}\nrvmask=\'{rvmask}\'')
def parameterAnalysis(param_space_directions, T_center, log_center, M_center, T_steps=50, log_steps=0.1, M_steps=0.1, T_tot=8, log_tot=8, M_tot=10):
    if (param_space_directions < 1) or (param_space_directions>4):
        print("Il primo argomento in parameterAnalysis deve essere un int \n1 - muoviti solo in temperatura\n2 - muoviti in temperatura e logg\n3 - muoviti in temperatura logg e metallicità\n4 muoviti solo in logg")
    elif param_space_directions == 1:
        T = T_center - (T_steps*T_tot)/2
        T_temp = T
        ### muovi temperatura
        for i in range(T_tot):
            crea_ATMinterpol(round(T_temp), round(log_center, 1), round(M_center, 2))                
            T_temp = T_temp + T_steps
    elif param_space_directions == 2:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        T_temp = T
        log_temp = log
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_center, 2))
                log_temp = log_temp + log_steps
            T_temp = T_temp + T_steps
            log_temp = log
    elif param_space_directions == 3:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        M = M_center - (M_steps*T_tot)/2
        T_temp = T
        log_temp = log
        M_temp = M
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                for k in range(M_tot):
                    crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_temp, 2))
                    M_temp = M_temp + M_steps
                log_temp = log_temp + log_steps
                M_temp = M
            T_temp = T_temp + T_steps
            log_temp = log
    print("FINITO")
def find_closest(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    return arr[closest_index]
def write_file(output_file, header1, header2, lines, tail):
        with open(output_file, 'w') as f_out:
            f_out.write(header1)
            f_out.write(header2)
            f_out.writelines(lines)
            f_out.write(tail)
def rimuoviHeader_atm(input_file, input_path, output_file, output_path):
    A=(os.getcwd())
    os.chdir(input_path)
    with open(input_file, 'r') as filein:
        lines = filein.readlines()
    lines = lines[23:-2]
    os.chdir(output_path)
    with open(output_file, 'w') as fileout:
        fileout.writelines(lines)
    os.chdir(A)
def SinterpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    for i in range(72):
        for j in range(10):
            a=[]
            for k in range(len(T_EFF_Nint)):
                a.append((P[k])[i][j])
            z = np.polyfit(T_EFF_Nint, np.asarray(a), 8)
            f = np.poly1d(z)
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_Nint)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaT(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_int) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}")
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(T_EFF_Nint)):
        nomeATMint = f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int'
        copia(f'atmoT{T_EFF_Nint[i]}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(f'T{T_EFF_Nint[i]:.0f}{logg:.1f}.int', format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(t_pradk, T_EFF_Nint) == 3500):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([T_EFF_Nint[0],T_EFF_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(t_pradk, T_EFF_Nint) == 6000):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([T_EFF_Nint[-1],T_EFF_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(t_pradk, T_EFF_Nint) > 3500) and (find_closest(t_pradk, T_EFF_Nint) < 6000):
                for k in range(len(T_EFF_Nint)): #scorro su atmosfere
                    if (T > T_EFF_Nint[k-1]) and (T <= T_EFF_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([T_EFF_Nint[k-1],T_EFF_Nint[k],T_EFF_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(T_EFF_Nint[k]-T_EFF_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(T_EFF_int[0], T_EFF_int[-1], len(T_EFF_int)*1000)
            y_new = f(x_new)
            T = find_closest(T, x_new)
            indice = np.where(x_new == T)[0]
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def interpolaM(T, logg, M, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    Mi=M
    t_pradk = find_closest(T, T_EFF_Nint)
    T=find_closest(T, T_EFF_Nint) #temperatura più fine (25K) rispetto a logg e M qua sotto (che ancora sono su griglia vanilla)
    logg=find_closest(logg, LOGG_Nint)
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)
    vturbs = vturbo(T, logg, M)
    header1 = f"TEFF   {T}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs}  L/H=1.25 NOVER NEW ODF\n"
    header2 = " OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   0.10000 ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(t_pradk, logg, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    for i in range(len(MH_Nint)):
        os.chdir(path_to_atmosphere + f"MH{MH_Nint[i]:.1f}")
        nomeATMint = f'{MH_Nint[i]}T{T:.0f}{logg:.1f}.int'
        copia(f'atmoT{T}{logg:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd())
        P.append(Table.read(nomeATMint, format='ascii'))
    y = np.zeros((72, 10))
    o=-1
    for i in range(72):
        for j in range(10):
            A=[]
            if (find_closest(M, MH_Nint) == 0.5):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([MH_Nint[0],MH_Nint[1]], np.asarray(A), 2)         
            elif (find_closest(M, MH_Nint) == -2.5):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([MH_Nint[-1],MH_Nint[-2]], np.asarray(A), 2)  
            elif (find_closest(M, MH_Nint) > -2.5) and (find_closest(M, MH_Nint) < 0.5):
                for k in range(len(MH_Nint)): #scorro su atmosfere
                    if (Mint <= MH_Nint[k-1]) and (Mint > MH_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([MH_Nint[k-1],MH_Nint[k],MH_Nint[k+1]], np.asarray(A), 2)
                        m=(A[1]-A[0])/(MH_Nint[k]-MH_Nint[k-1])
                        o=k-1
            f = np.poly1d(z)
            
            x_new = np.linspace(MH_int[0], MH_int[-1], len(MH_int)*1000)
            y_new = f(x_new)
            Mint = find_closest(Mint, x_new)
            indice = np.where(x_new == Mint)[0]
            #y_new = A[0]+(T-T_EFF_Nint[o])*m
            y[i][j] = y_new[indice]
            if y[i][j]<0:
                y[i][j] = 0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'{Mi:.2f}atm{T:.0f}_{logg}.int'
    write_file( nome_out, header1, header2, linea, tail)
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return P
def Sinterpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(M)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{M:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            a=[]
            for k in range(len(LOGG_Nint)): #scorro su atmosfere
                a.append((P[k])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
            z = np.polyfit(LOGG_Nint, np.asarray(a), 6) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def interpolalog(T, logg, M, variazioneTemperatura, path_ATM=path_to_atmosphere):
    P=[]
    linea = []
    T_in=T
    T=find_closest(T, T_EFF_Nint)              #temperatura su griglia originale
    logg_pradk = find_closest(logg, LOGG_Nint) #per formattazione
    logg=find_closest(logg, LOGG_int)          #logg su griglia interpolata
    Mint=find_closest(M, MH_int)
    M=find_closest(M, MH_Nint)                 #M su griglia originale
    vturbs = vturbo(T, logg, M)                #calcola vturb
    Fe_H = creaHeader_metal(Mint)
    header1 = f"TEFF   {T_in:.0f}.  GRAVITY {logg:.1f} LTE\nTITLE  [{Mint:.1f}] VTURB={vturbs:.2f}  L/H=1.25 NOVER NEW ODF\n"
    header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {Fe_H:.3f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
    tail = f"PRADK {getTail(T, logg_pradk, M)}" + "\nBEGIN                    ITERATION  15 COMPLETED"
    os.chdir(path_to_atmosphere + f"MH{M:.1f}") #vai nella cartella con le atmosfere della giusta metallicità
    for i in range(len(LOGG_Nint)):             #scorri tutte le atmosfere che hanno temperatura fissata sulla griglia cambiando il logg
        nomeATMint = f'T{T:.0f}{LOGG_Nint[i]:.1f}.int'
        copia(f'atmoT{T}{LOGG_Nint[i]:.1f}.txt', os.getcwd(), os.getcwd(), nomeATMint)
        rimuoviHeader_atm(nomeATMint, os.getcwd(), nomeATMint, os.getcwd()) #prende solo la tabella 72x10
        P.append(Table.read(f'T{T:.0f}{LOGG_Nint[i]:.1f}.int', format='ascii')) #la metto in una lista, la i-esima atmosfera corrisponde a quella avente T=T_fissato_su_griglia con log(g_i)
    y = np.zeros((72, 10)) 
    o=-1
    for i in range(72): #scorro su righe
        for j in range(10): #scorro su colonne
            A=[]
            if (find_closest(logg_pradk, LOGG_Nint) == 0.0):
                A.append((P[0])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[1])[i][j])
                A.append((P[2])[i][j])
                z = np.polyfit([LOGG_Nint[0],LOGG_Nint[1],LOGG_Nint[2]], np.asarray(A), 2)         
            elif (find_closest(logg_pradk, LOGG_Nint) == 5.0):
                A.append((P[-1])[i][j]) #a contiene l'elemente i,j-esimo delle matrici 72x10 per ogni atmosfera con i vari log(g_i)
                A.append((P[-2])[i][j])
                A.append((P[-3])[i][j])
                z = np.polyfit([LOGG_Nint[-1],LOGG_Nint[-2],LOGG_Nint[-3]], np.asarray(A), 2)  
            elif (find_closest(logg_pradk, LOGG_Nint) > 0.0) and (find_closest(logg_pradk, LOGG_Nint) < 5.0):
                for k in range(len(LOGG_Nint)): #scorro su atmosfere
                    if (logg >= LOGG_Nint[k-1]) and (logg <= LOGG_Nint[k]):
                        A.append((P[k-1])[i][j])
                        A.append((P[k])[i][j])
                        A.append((P[k+1])[i][j])
                        z = np.polyfit([LOGG_Nint[k-1],LOGG_Nint[k],LOGG_Nint[k+1]], np.asarray(A), 2) #fit polinomiale di 6° grado (molto buono per metallicità circa solare, meno per bassa metallicità)
            f = np.poly1d(z) #f è un polinomio di 6° grado con coefficienti dati da z
            # Y = np.asarray(A)
            # if find_closest(, LOGG_Nint)
            x_new = np.linspace(LOGG_Nint[0], LOGG_Nint[-1], len(LOGG_Nint)*1000) #spanno lo spazio del dominio (1000 volte più grande dell'originale)
            y_new = f(x_new) #creo l'immagine
            # params, cov = curve_fit(expo_func, X, Y, maxfev=1000)
            # a, b, c = params
            # y_new = expo_func(x_new, a, b, c)
            LoG = find_closest(logg, x_new) #prendo il logg come quello più vicino a uno dei punti x_new
            indice = np.where(x_new == LoG)[0] #l'indice di x_new_i corretto corrisponde a quello di y_i corretto
            if y[i][j]<0: #non deve mai succedere
                y[i][j]=0
            y[i][j] = y_new[indice] + variazioneTemperatura[i][j]
            if i>0:
                if y[i][j]<y[i-1][j]: #scendendo con le righe deve sempre crescere
                    y[i][j] = y[i-1][j] #se non cresce blocco. In realtà da provare con fit esponenziale oppure facendo media del j-esimo elemento di differenti i-esime righe
                    if (y[i-1][j]==0) and (y[i+1][j]) ==0:
                        y[i][j]=0
        linea.append(f" {y[i][0]:.8E}   {y[i][1]:.1f} {y[i][2]:.3E} {y[i][3]:.3E} {y[i][4]:.3E} {y[i][5]:.3E} {y[i][6]:.3E} {y[i][7]:.3E} {y[i][8]:.3E} {y[i][9]:.3E}\n")
    nome_out = f'atmL{T_in:.0f}_{logg:.2f}.int'
    write_file( nome_out, header1, header2, linea, tail)
    copia( nome_out, os.getcwd(), path_to_output, nome_out + "aiai")
    sposta( nome_out, os.getcwd(), path_to_output, nome_out)
    return nome_out
def differenza_INT_grid(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'atm{find_closest(temperats, T_EFF_int)}_{logg_close:.1f}.int', path_to_output, f'atm{temperats:.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'atm{temperats:.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def differenza_INT_grid_M(temperats, loggheg, metallus, linea=[]):
    T_close = find_closest(temperats, T_EFF_Nint)
    logg_close=find_closest(loggheg, LOGG_Nint)
    M_close=find_closest(metallus, MH_Nint)
    rimuoviHeader_atm(f'atmoT{T_close}{logg_close:.1f}.txt', path_to_atmosphere+f"MH{M_close}", f'atmoT{T_close}{logg_close:.1f}.txt', path_to_output)
    os.chdir(path_to_output)
    tavola = Table.read(f'atmoT{int(T_close)}{logg_close:.1f}.txt', format='ascii')
    rimuoviHeader_atm(f'{metallus}atm{find_closest(temperats, T_EFF_Nint)}_{logg_close:.1f}.int', path_to_output, f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', path_to_output)
    tavola1 = Table.read(f'{metallus}atm{find_closest(temperats, T_EFF_Nint):.0f}_{logg_close:.1f}.int', format='ascii')
    matrice_differenze = np.zeros((72, 10))
    for i in range(72):
        m0=tavola1[i][0]-tavola[i][0]
        m1=tavola1[i][1]-tavola[i][1]
        m2=tavola1[i][2]-tavola[i][2]
        m3=tavola1[i][3]-tavola[i][3]
        m4=tavola1[i][4]-tavola[i][4]
        m5=tavola1[i][5]-tavola[i][5]
        m6=tavola1[i][6]-tavola[i][6]
        m7=tavola1[i][7]-tavola[i][7]
        m8=tavola1[i][8]-tavola[i][8]
        m9=tavola1[i][9]-tavola[i][9]
        matrice_differenze[i][0]=m0
        matrice_differenze[i][1]=m1
        matrice_differenze[i][2]=m2
        matrice_differenze[i][3]=m3
        matrice_differenze[i][4]=m4
        matrice_differenze[i][5]=m5
        matrice_differenze[i][6]=m6
        matrice_differenze[i][7]=m7
        matrice_differenze[i][8]=m8
        matrice_differenze[i][9]=m9
        linea.append(f" {m0:.8E}   {m1:.1f} {m2:.3E} {m3:.3E} {m4:.3E} {m5:.3E} {m6:.3E} {m7:.3E} {m8:.3E} {m9:.3E}\n")
    write_file( "differenza", '', '', linea, '')
    return matrice_differenze
def creaHeader_metal(Fe_H, metallic=metallic_header, ABUNDANCE_SCALE=ABUNDANCE_SCALE_header):
    return 10**Fe_H
def preparaAbundance(stellaFITS, path_to_abundance=path_to_abundance, eqw_low=10, eqw_high=150, lineList='Iron_lines'):
    # Apre il file in modalità lettura
    """Produce il file "StellaFes" contenente le EQUIVALENT WIDTH della stella.\nNome del file FITS (doppler corretto) contenente la stella da analizzare
    eqw low and high sono gli estremi di equivalent width accettata nel processo di analisi, per trattare la curva di crescita delle eqw e rimanere fuori del rumore e dalla saturazione della riga"""
    i=0
    os.chdir(path_to_linemake)
              #Iron_lines#
    with open('Iron_lines', 'r') as file:
        LINEM = repr(file.read())
    os.chdir(path_to_ARES)
    creaOptFileARES(stellaFITS, lineList)
    ! ./ARES
    with open('stella.ares', 'r') as file:
        ARES = repr(file.read())

    vect1 = LINEM.split("\\n")
    vect2 = ARES.split("\\n")
    VECT1 = np.zeros((len(vect1),9))
    VECT2 = np.zeros((len(vect2),9))

    for i in range(1,len(vect2)-1):
        temp2 = vect2[i].split()
        VECT2[i-1][0]= temp2[0]
        VECT2[i-1][1]= temp2[1]
        VECT2[i-1][2]= temp2[2]
        VECT2[i-1][3]= temp2[3]
        VECT2[i-1][4]= temp2[4]
        VECT2[i-1][5]= temp2[5]
        VECT2[i-1][6]= temp2[6]
        VECT2[i-1][7]= temp2[7]
        VECT2[i-1][8]= temp2[8]
    t4 = Table.read('stella.ares', format='ascii')

    for i in range(1, len(vect1)-1):
        temp1 = vect1[i].split()
        VECT1[i-1][0]= float(temp1[0])
        VECT1[i-1][1]= temp1[1]
        VECT1[i-1][2]= float(temp1[2])
        VECT1[i-1][3]= temp1[3]
        VECT1[i-1][4]= temp1[4]
        VECT1[i-1][5]= 1.0
        VECT1[i-1][6]= 99
        VECT1[i-1][7]= 0

    for i in range(len(VECT1)):
        for j in range(len(VECT2)):
            if (VECT1[i][0] == VECT2[j][0]) and (VECT2[j][4]>eqw_low) and (VECT2[j][4]<eqw_high) and (float(VECT1[i][1])==26.0):
                VECT1[i][7] = VECT2[j][4]
            elif (VECT1[i][0] == VECT2[j][0]) and (float(VECT1[i][1])==26.1):
                VECT1[i][7] = VECT2[j][4]

    # for i in range(1, len(vect1)-1): # per ARES linemake
    #     temp1 = vect1[i].split()
    #     VECT1[i-1][0]= float(temp1[0])
    #     VECT1[i-1][1]= temp1[1]
    #     VECT1[i-1][2]= float(temp1[2]) * 8066.01
    #     VECT1[i-1][3]= 0
    #     VECT1[i-1][4]= temp1[3]
    #     VECT1[i-1][5]= 1.0
    #     VECT1[i-1][6]= 99
    #     VECT1[i-1][7]= 0

    ############################## PREPARA FILE PER ABUNDANCE ################################

    os.chdir(path_to_abundance)
    bug = []
    o=0
    T_atmosold=0
    for i in range(len(VECT1)):
        if VECT1[i][7]>eqw_low and VECT1[i][7]<eqw_high: #if VECT1[i][1] == 26.0 and VECT1[i][7]>30:
            bug.append(VECT1[i][7])
            o=o+1
    # indici_righe_da_rimuovere = np.where((VECT1[:, 7] < eqw_low) | (VECT1[:, 7] > eqw_high))[0]
    # VECT1 = np.delete(VECT1, indici_righe_da_rimuovere, axis=0)
    linesout = 'stellantess'
    iron_linesout = 'stellaFes'
    with open(linesout, 'w') as file:
        for i in range(len(VECT1)):
            #print(VECT1[i][1])
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')

    # with open(iron_linesout, 'w') as file:
    #     for i in range(len(VECT1)):
    #         if VECT1[i][7]!=0 and (VECT1[i][1]==26 or VECT1[i][1]==26.1):
    #             file.write(str(VECT1[i][0]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][1]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][2]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][3]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][4]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][5]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][6]))
    #             file.write('  ')
    #             file.write(str(VECT1[i][7]))
    #             file.write('  ')
    #             file.write('a')
    #             file.write('\n')
    with open(iron_linesout, 'w') as file:
        for i in range(len(VECT1)):
            if VECT1[i][7]!=0:
                file.write(str(VECT1[i][0]))
                file.write('  ')
                file.write(str(VECT1[i][1]))
                file.write('  ')
                file.write(str(VECT1[i][2]))
                file.write('  ')
                file.write(str(VECT1[i][3]))
                file.write('  ')
                file.write(str(VECT1[i][4]))
                file.write('  ')
                file.write(str(VECT1[i][5]))
                file.write('  ')
                file.write(str(VECT1[i][6]))
                file.write('  ')
                file.write(str(VECT1[i][7]))
                file.write('  ')
                file.write('a')
                file.write('\n')
    o=0
    vturb = 1.8
    comando_bash = f"cp {iron_linesout} {path_to_output}{iron_linesout}"
    os.system(comando_bash)
    os.chdir(path_to_abundance)
def creaInterpolazione(temperat, loggh, metallu):
    interpolaT(temperat, loggh, metallu)
    T_close=find_closest(temperat, T_EFF_Nint)
    logg_close=find_closest(loggh, LOGG_Nint)
    M_close=find_closest(metallu, MH_Nint)
    differenze = differenza_INT_grid(temperat, loggh, metallu)
    interpolaM(temperat, loggh, metallu)
    differenze = differenze + differenza_INT_grid_M(temperat, loggh, metallu)
    b = interpolalog(temperat, loggh, metallu, differenze)
    return b
def abundance_interpolato(temperat, loggh, metallu, nome_stella, path_to_stella=path_to_output):
    olddir=os.getcwd()
    os.chdir(path_to_stella)
    preparaAbundance(nome_stella)
    B=creaInterpolazione(temperat, loggh, metallu)
    sposta(B, path_to_stella, path_to_abundance, "ModelE")
    os.chdir(path_to_abundance)
    creaParamfileAbnd(vturbo(temperat, loggh, metallu))
    SlanciaAbundance()
    os.chdir(path_to_stella)
    bashComm= "mkdir " + f"{nome_stella}"
    os.system(bashComm)
    path_to_output1=path_to_stella+f"{nome_stella}"
    nome_out = f"INT_{metallu}_{temperat}_{loggh}"
    sposta("PINOLOS", path_to_abundance, path_to_output1, nome_out)
    os.chdir(olddir)
    return nome_out
def avvicina_interpol(path_to_stella, nome_file, TEFF, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1
    teff=-1
    met=1
    returner = []
    if abs(temp0[1]) > 0.001:
        if (abs(temp0[1]) < 0.06) and (abs(temp0[1]) >= 0.035):
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 70*sgn
        elif abs(temp0[1] > 0.01) and abs(temp0[1]) < 0.035:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 30*sgn
        elif abs(temp0[1]) < 0.01:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 10*sgn
        elif abs(temp0[1]) < 0.13 and abs(temp0[1]) > 0.06:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 150*sgn
        else:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 300*sgn
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
    else:
        nome_file1 = nome_file
        teff = TEFF
    temp0 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.0)
    temp1 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.1)
    sgn1=1
    if (temp0[0]-temp1[0])>0:
        sgn1=-1
    if abs(temp0[0]-temp1[0]) > 0.01:
        if abs(temp0[0]-temp1[0]) < 0.4:
            g = G+(temp0[0]-temp1[0])
        else:
            g = G+((temp0[0]-temp1[0]))
        crea_ATMinterpol(teff, g, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{g:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        #copia(f"{nome_file1}", path_to_output, path_to_output, f"{nome_file1}{nome_stella}")
        met=temp2
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol2(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    if M-temp0[0] >= 0:
        dM = -(M - temp0[0])/3
    else:
        dM = (M - temp0[0])/3
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dfdg = -0.5227
    dfdt = 0.001455
    f=-1*((temp0[0])-(temp1[0]))
    s = -1*temp0[1]
    if abs(temp0[1]) > 0.0002 or abs((temp0[0])-(temp1[0])) > 0.0005 or (M - temp0[0])>0.01:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = ((dsdt * f - dfdt * s)/(dfdg * dsdt - dsdg * dfdt))
        DT = ((dfdg * s - dsdg * f)/(dfdg * dsdt - dsdg * dfdt))
        if DG > 0.5:
            DG=0.5*(DG/abs(DG))
        if DT > 250:
            DT = 250*(DT/abs(DT))
        temperatu=temperatu+DT
        teff= temperatu/0.87
        G=G+DG
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp0[0]:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol3(path_to_stella, nome_file, temperatu, G, M, nome_stella, nome_cartella=path_to_output, NLTE=0):
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dsdm = 0.0104
    dfdg = -0.5227
    dfdt = 0.001455
    dfdm = -0.3574 
    dydg = -0.13522
    dydt = 0.00041
    dydm = -0.8345
    A = np.array([[dsdg, dsdt, dsdm],
         [dfdg, dfdt, dfdm],
         [dydg, dydt, dydm]])
    A_inv = np.linalg.inv(A)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    B = np.array([s, f, m])
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DG = (np.dot(A_inv, B))[0]
        DT = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        temperatu=temperatu+DT
        G=G+DG
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol(teff, G, temp0[0]+nlte_corrM, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol4(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    blocca_stampe()
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        if abs(Dv) > 0.3:
            Dv = 0.3*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp2[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def avvicina_interpol5(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    #blocca_stampe()
    old_vturb = vturbo(temperatu, G, M)
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    #riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    #blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.10:
            DG=0.10*(DG/abs(DG))
        if abs(DT) > 150:
            DT = 150*(DT/abs(DT))
        if abs(DM) > 0.10:
            DM = 0.10*(DM/abs(DM))
        if abs(Dv) > 0.1:
            Dv = 0.1*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        nome_file1 = f"{nome_stella}_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}"
        sposta(f"{nome_stella}_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def avvicina_interpol6(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    #blocca_stampe()
    old_vturb = vturbo(temperatu, G, M)
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    #riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    #blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.10:
            DG=0.10*(DG/abs(DG))
        if abs(DT) > 150:
            DT = 150*(DT/abs(DT))
        if abs(DM) > 0.10:
            DM = 0.10*(DM/abs(DM))
        if abs(Dv) > 0.1:
            Dv = 0.1*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        nome_file1 = f"{nome_stella}2_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}"
        sposta(f"{nome_stella}_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def nextParam(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    blocca_stampe()
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dsdm = 0.0104
    dfdg = -0.5227
    dfdt = 0.001455
    dfdm = -0.3574 
    dydg = -0.13522
    dydt = 0.00041
    dydm = -0.8345
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        if abs(Dv) > 0.3:
            Dv = 0.3*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def leggi_log(path_cartella_log):
    '''Di seguito i nomi delle colonne di log.txt. Restituisce la tavola letta con Table.read(). T      G      M   vturb   FeI            FeII           FeI-M/H        FeI-FeII       s(FeI)  EWR'''
    olddir = os.getcwd()
    os.chdir(path_cartella_log)
    log_table = Table.read("log.txt", format='ascii')
    os.chdir(olddir)
    return log_table
def minimo_tavola(tab, nome_colonna, toll=2.0, mode='abs'):
    '''Passata una tabella restituisce l'elemento minimo della colonna specificata e il relativo indice di riga. mode=abs trova il minimo in valore assoluto, mode=n trova il minimo considerando anche il segno'''
    x=tab[nome_colonna]
    if mode=='abs':
        X = abs(x)
    if mode=='n':
        X = x
    returner = []
    min = 0
    i=0
    absmin = X[X.argmin()]
    while min < absmin*toll or min-absmin<=0.05:
        M = X.argmin()
        min = X[M]
        min_index = M
        returner.append([min, int(min_index)])
        X[np.where(X == min)] = 1
        i=i+1
    return returner
def final_atmo(path_cartella_log): #
    '''Passa log file, restituisce la best atmosfera. Nelle cartelle contenenti le atmosfere analizzate è possibile creare un log file attraverso la funzione folderLog. Passa qua la directory del log di quella stella. Si cerca prima l'atmosfera che più minimizza FeI-FeII. Da qua si verifica che l'atmosfera che minimizza la slope sia sufficientemente vicina al minimo (i.e. entro 1.2*minimo_slope).'''
    tab=leggi_log(path_cartella_log)
    dF=minimo_tavola(tab, 'FeI-FeII')
    slope=minimo_tavola(tab, 's(FeI)')
    if (tab[slope[0][1]] == tab[dF[0][1]]) and (tab[dF[0][1]]['FeI-M/H'] <= 0.01):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(dF[0][1], slope):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(slope[0][1], dF):
        print(tab[slope[0][1]])
        print()
        return tab[slope[0][1]]
    elif esiste_in_lista(dF[1][1], slope): 
        print(tab[dF[1][1]])
        print()
        return tab[dF[1][1]]
    elif esiste_in_lista(slope[1][1], dF): 
        print(tab[slope[1][1]])
        print()
        return tab[slope[1][1]]
    elif len(dF)>= 3 and esiste_in_lista(dF[2][1], slope): 
        print(tab[dF[2][1]])
        print()
        return tab[dF[2][1]]
    elif len(slope)>= 3:
        if esiste_in_lista(slope[2][1], dF): 
            print(tab[slope[2][1]])
            print()
            return tab[slope[2][1]]
    else:
        print("nessuna atmosfera trovata.. :C")
def esiste_in_lista(x, lista2):
    esiste = any(x == sub2[1] for sub2 in lista2)
    return esiste
def NLTE_Fe_correction(t, e, g, x, wi):
    url = 'http://www.inspect-stars.com/cp/application.py/A_from_e?element_name=Fe'

    # Cambia il valore del campo 't'
    form_data = {
        't': f'{t}', 
        'e': f'{e}',
        'g': f'{g}',
        'x': f'{x}',
        'wi': f'{wi}'
    }

    # Codifica i dati come query string per l'invio con POST
    encoded_data = urllib.parse.urlencode(form_data).encode('utf-8')

    # Fai la richiesta POST con i dati del form
    req = urllib.request.Request(url, data=encoded_data)

    # Aggiungi un User-Agent per simulare un browser reale
    req.add_header('User-Agent', 'chrome')

    with urllib.request.urlopen(req) as response:
        result = response.read().decode('utf-8')

    # 1. Estrazione del blocco <pre>
    pre_block = re.search(r'<pre>(.*?)</pre>', result, re.DOTALL)
    if pre_block:
        pre_content = pre_block.group(1)  # Ottieni il contenuto tra <pre> e </pre>
        # 2. Estrazione dei numeri (inclusi decimali)
        numbers = re.findall(r'\d+\.\d+|\d+', pre_content)
        # Stampa o utilizza i numeri come preferisci
        print("Numeri estratti:", numbers)
        if len(numbers) == 5:
            return numbers
        else:
            returner = [e, 0, 0, 0, 0]
            return returner
    else:
        print("Blocco <pre> non trovato.")
def find_closest2(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    if (arr[closest_index]-num)>0.011:
        print("No line found!")
        return 0
    else:
        return arr[closest_index]
def createCorrectionGrid():
    o=0
    OLDDIR = os.getcwd()
    os.chdir(path_to_correctionGrids)
    with open('correction_grid.txt', 'w') as gridder:
        gridder.write("EW               T               G                V                deltaNLTE\n")
        for E in range(20, 200, 10):
            print(f"EW_riga: {E}")
            for i in range(4100, 5500, 50):
                for j in range(10, 26, 1):
                    for k in range(100, 200, 5):
                        #for l in range(120):
                        U=NLTE_Fe_correction(i, E, j/10, k/100, 34)
                        a0 = int(E-20)
                        a1 = int((i-4000)/20)
                        a2 = int((j-10))
                        a3 = int((k-100)/5)
                        gridder.write(f"{U[0]}               {i}               {j/10}                {k/100}                {U[3]}")
                        gridder.write("\n")
                        o=o+1
                        if o == 200:
                            time.sleep(np.random.exponential(10))
    os.chdir(OLDDIR)
def extractSlope(log_path, x, y, p0=0, p1=0, p2=0, p3=0):
    '''Questa funzione restituisce l'andamento di una grandezza y in funzione di x. I dati sono letti dai file log.txt prodotti.\n
    Ad esempio y=s(FeI), x=T. Le quantità devono essere passate come stringhe:\n
    x):\n
    'T'= Temperatura\n
    'G'= log(g)\n
    'M'= [M/H]\n
    'v'= vturb\n
    y):\n
    's'= s(FeI)\n
    'F'= A(FeI)-A(FeII)\n
    'Y'= [Fe/H]_dedotta - [Fe/H]_modello\n
    'S'= S(EWR)\n
    p_i):\n
    p0 = T0\n
    p1 = G0\n
    p2 = M0\n
    p3 = v0\n
    Restituisce dy/dx da fit lineare centrato su tre dei punti (p0, p1 e p2, p3), uno rimane libero per ottenere la variazione.'''
    tab = leggi_log(log_path)
    # Crea i vettori dati
    if True:
        if y == 's':
            Y = tab['s(FeI)']
        elif y == 'F':
            Y = tab['FeI-FeII']
        elif y == 'Y':
            Y = tab['FeI-M/H']
        elif y == 'S':
            Y = tab['EWR']
        else:
            print("'y' inserita non valida\n Opzioni valide: s, F, Y, S")
        
        if x == 'T':
            X = tab['T']
            x1 = tab['G']
            x2 = tab['M']
            x3 = tab['vturb']            
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            #print(Y)
            x2=x2[lim]
            x3=x3[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]         
            lim=(abs(x3-p3)<0.05)
            X=X[lim]
            Y=Y[lim]
        elif x == 'G':
            X = tab['G']
            x0 = tab['T']
            x2 = tab['M']
            x3 = tab['vturb']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x2=x2[lim]
            x3=x3[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]
            lim=((x3==p3))
            X=X[lim]
            Y=Y[lim]
        elif x == 'M':
            X = tab['M']
            x0 = tab['T']
            x1 = tab['G']
            x3 = tab['vturb']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x1=x1[lim]
            x3=x3[lim]
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]
            lim=((x3==p3))
            X=X[lim]
            Y=Y[lim]
        elif x == 'v':
            X = tab['vturb']
            x0 = tab['T']
            x1 = tab['G']
            x2 = tab['M']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x1=x1[lim]
            x2=x2[lim]
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            x2=x2[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
        else:
            print("'x' inserita non valida\n Opzioni valide: T, G, M, v")
    if x == 'T' or x== 'G' or x=='M':
        popt, pcov = curve_fit(linear_func, X, Y,method='lm')
        X_fit = np.linspace(min(X), max(X), 100)
        Y_fit = linear_func(X_fit, *popt)
        m, c = popt
        # Plot dei dati originali e della retta di best fit
        plt.figure(figsize=(8, 6))
        plt.scatter(X, Y, color='red', label='Dati originali')
        plt.plot(X_fit, Y_fit, color='blue', label=f'Retta di best fit: ${y}({x}) = {m:.7f}{x} + {c:.7f}$')
        plt.xlabel(f'{x}')
        plt.ylabel(f'{y}({x})')
        plt.title('Fit Lineare')
        plt.legend()
        plt.grid()
        plt.savefig(path_to_calibrationFolder +f'/pt_{y}-{x}.pdf')
        plt.show()
        return popt
    elif x == 'v':
        popt, pcov = curve_fit(quad_func, X, Y,method='lm')
        X_fit = np.linspace(min(X), max(X), 100)
        Y_fit = quad_func(X_fit, *popt)
        m, c , d = popt
        # Plot dei dati originali e della retta di best fit
        plt.figure(figsize=(8, 6))
        plt.scatter(X, Y, color='red', label='Dati originali')
        plt.plot(X_fit, Y_fit, color='blue', label=f'Retta di best fit: ${y}({x}) = {m:.7f}{x}^2 + {c:.7f}{x} + {d:.2f}$')
        plt.xlabel(f'{x}')
        plt.ylabel(f'{y}({x})')
        plt.title('Fit Lineare')
        plt.legend()
        plt.grid()
        plt.savefig(path_to_calibrationFolder +f'/pt_{y}-{x}.pdf')
        plt.show()
        return popt
    #return c
def corrDoppler(x, v):
    '''Dando una lunghezza d'onda osservata restituisce il valore della lunghezza d'onda senza effetto doppler\nv in km/s'''
    c = 299792.458 #luce
    return x/((v/c)+1)
def NLTE_wlCorrection(wl, el, T, G, M, V):
    DIR_INI = os.getcwd()
    M=M+7.5
    os.chdir(path_to_output)
    file_path = 'stellaFes'
    #lista_primi_elementi = crea_lista_primo_elemento(file_path)
    #print(lista_primi_elementi)
    correctionList = []
    PATH= path_to_correctionGrids1
    PATH=path_to_correctionGrids1
    if wl < 5000:
        PATH=PATH+"4500-5000/correctionGrid"
    elif wl > 5000 and wl < 5250:
        PATH=PATH+"5000-5250/correctionGrid"
    elif wl > 5250 and wl < 5500:
        PATH=PATH+"5250-5500/correctionGrid"
    elif wl > 5500 and wl < 5750:
        PATH=PATH+"5500-5750/correctionGrid"
    elif wl > 5750 and wl < 6000:
        PATH=PATH+"5750-6000/correctionGrid"
    elif wl > 6000 and wl < 6250:
        PATH=PATH+"6000-6250/correctionGrid"
    elif wl > 6250 and wl < 6500:
        PATH=PATH+"6250-6500/correctionGrid"
    elif wl > 6500 and wl < 6750:
        PATH=PATH+"6500-6750/correctionGrid"
    else:
        print("Nooo!")
    correctionList.append(findLine(f"{PATH}", T, G, M, wl, V))
        #print(correctionList[i])
    correctionList=[el for el in correctionList if el is not None]
    # print("AIUO")
    #print(correctionList)

    for i in range(len(correctionList)):
        if abs(wl - correctionList[i][6])<0.001:
            return correctionList[i][0]
def checkExist_newParam(nome, T, G, M, V, path):
    file_path = path + f"/{nome}_{M:.2f}_{T:.0f}_{G:.2f}_{V:.2f}"
    if os.path.isfile(file_path):
        T=T+1
        G=G+0.01
        M=M-0.01
        return checkExist_newParam(nome, T, G, M, V, path)

    else:
        return [T, G, M, V]
#def NLTE_interpola()

def NLTE_intCorrection(wl, el, T, G, M, V):
    #maiuscole inserite, minuscole sulla griglia
    Tcheck = False
    Gcheck = False
    Mcheck = False
    Vcheck = False
    overT = []
    overG = []
    overM = []
    overV = []
    overT.append(NLTE_wlCorrection(wl, el, T, G, M, V)) #dà il valore della correzione nel punto più vicino sulla griglia
    overG.append(overT[0])
    overM.append(overT[0])
    overV.append(overT[0])
    t=find_closest(T, temperatura)
    g=find_closest(G, LOGG_arr)
    m=find_closest(M+7.5, METAL)-7.5
    v=find_closest(V, MICROTURB)
    if True:
        if (g==2.0) or (g==2.5):
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M
                m1=m


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print('ayyy lmao')
            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v


            if G>g:
                g1=g+0.5
            elif G<g:
                g1=g-0.5
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==1.5:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t1=t


            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:

                m=M+0.01
                m1=find_closest(M-0.25+7.5, METAL)-7.5


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v

            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print("oyyy vey!")

            if G>g:
                g1=g+0.5
            elif G<g:
                print("Non c'è logg sotto 1.5dex e sopra 3dex!")
                g1=g
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==3.0:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M
                m1=m
            if m1 == 4.25 or m1 == 7.75:
                print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
                return 0

            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v

            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati SOTTO i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v


            if G<g:
                g1=g-0.5
            elif G>=g:
                print(f"Non c'è logg sotto 1.5dex e SOPRA 3dex! {G}, {g}")
                g1=g


        if t1 == 3750 or t1 == 5750:
            print('Non ci sono dati sotto i 4000K o sopra i 5500K!')
            #return 0
        if m1 == 4.25 or m1 == 7.75:
            print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
            return 0
        if g1==1.5 or g == 1.5:
            if t1 == 3750 or t1 == 4750:
                print('Non ci sono dati SOTTO i 4000K o sopra i 4500K! (per questo logg)')
        if g1 == 3 or g == 3:
            if t1 == 4250 or t1 == 5750:
                print('Non ci sono dati sotto i 4500K o SOPRA i 5500K! (per questo logg)')
                #return 0
            if g==3:
                if g1 > g:
                    print(f'Non ci sono dati sopra ai 3dex di logg {G}, {g}, {g1}')
        
    #print(v1)
    print(f"true  close next {wl}\n{T}, {t}, {t1}\n{G},  {g},  {g1}\n{M},  {m},  {m1}\n{V},  {v},  {v1}")
    if g==1.5 and t1>4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    if g==3.0 and t1<4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    # if G < 1.5:
    #     if t1>4500:    
    #         overG.append(NLTE_wlCorrection(wl, el, 4500, 2, M, V)) #
    overG.append(NLTE_wlCorrection(wl, el, T, g1, M, V))
    overM.append(NLTE_wlCorrection(wl, el, T, G, m1, V))
    overV.append(NLTE_wlCorrection(wl, el, T, G, M, v1))

    #print(f"{overT} --- {overG} --- {overM} --- {overV}")
    if t != t1:
        corrt=(overT[1]-overT[0])/(max(t1, t) - min(t1, t))*(T-t)
    else:
        corrt=0
    if g != g1:
        corrg=(overG[1]-overG[0])/(max(g1, g) - min(g1, g))*(G-g)
    else:
        corrg=0
    if m != m1:
        corrm=(overM[1]-overM[0])/(max(m1, m) - min(m1, m))*(M-m)
    else:
        corrm=0
    if v != v1:
        corrv=(overV[1]-overV[0])/(max(v1, v) - min(v1, v))*(V-v)
    else:
        corrv=0
    corrT=overT[0]+corrt
    corrG=overG[0]+corrg
    corrM=overM[0]+corrm
    corrV=overV[0]+corrv
    totalCorr = corrt + corrg + corrm + corrv
    #print(f"{t}, {t1}, eheh {overT}, {corrT}\n{g}, {g1}, eheh {overG}, {corrG}\n{m}, {m1}, eheh {overM}, {corrM}\n{v}, {v1}, eheh {overV}, {corrV}\n")
    #print(f"originale: {overT[0]}, \ninterpol: {overT[0]+totalCorr}\nDelta:{totalCorr}")
    return [overT[0], overT[0]+totalCorr, totalCorr]
def NLTE_caricaWLdisponibili():
    filename = "/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/4500-5000/correctionGrid"

    # Lista per i valori univoci
    unique_values = set()
    with open(filename, "r") as file:
        for line in file:
            # Ignora righe vuote o non formattate correttamente
            if line.strip():
                # Divide la riga in colonne
                columns = line.split()
                try:
                    # Estrae il valore della sesta colonna e lo aggiunge al set
                    unique_values.add(round(float(columns[5])*10, 3))
                except IndexError:
                    # Gestisce righe che non hanno abbastanza colonne
                    print(f"Riga non valida (meno di 6 colonne): {line.strip()}")
    # Lettura del file
    for i in range(7):
        filename = f"/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/{5000+250*i}-{5250+250*i}/correctionGrid"
        with open(filename, "r") as file:
            for line in file:
                # Ignora righe vuote o non formattate correttamente
                if line.strip():
                    # Divide la riga in colonne
                    columns = line.split()
                    try:
                        # Estrae il valore della sesta colonna e lo aggiunge al set
                        unique_values.add(round(float(columns[5])*10, 3))
                    except IndexError:
                        # Gestisce righe che non hanno abbastanza colonne
                        print(f"Riga non valida (meno di 6 colonne): {line.strip()}")

    # Converte il set in una lista ordinata
    unique_values_list = sorted(unique_values)

    # Stampa dei risultati
    print("Valori univoci nella sesta colonna:")
    return unique_values_list
# from main import *
def delete_files(directory):
    """
    Cancella tutti i file che seguono il formato "PINOLOSx" (dove x è un numero) in una cartella specificata.

    Args:
        directory (str): Percorso della cartella in cui cercare i file.
    """
    try:
        # Controlla se la cartella esiste
        if not os.path.isdir(directory):
            print(f"Directory non trovata: {directory}")
            return

        # Itera sui file nella cartella
        for filename in os.listdir(directory):
            # Verifica se il nome del file corrisponde al pattern "PINOLOS" seguito da un numero
            if filename.startswith("PINOLOS") and filename[7:].isdigit():
                file_path = os.path.join(directory, filename)
                # Elimina il file
                os.remove(file_path)
                print(f"File cancellato: {file_path}")
                
    except Exception as e:
        print(f"Errore durante l'eliminazione dei file: {e}")

class ParallelRunner:
    """
    Questa classe esegue diversi programmi numerati in parallelo.
    """

    def __init__(self, base_program_path, num_programs):
        """
        Inizializza la classe.

        :param base_program_path: Directory base o prefisso del programma (es: /path/to/abundance).
        :param num_programs: Numero totale di programmi da eseguire (es: abundance1, abundance2, ..., abundanceN).
        """
        self.base_program_path = base_program_path
        self.num_programs = num_programs
        self.processes = []

    def run_parallel(self, additional_args=None):
        """
        Esegue i programmi numerati in parallelo.

        :param additional_args: Argomenti extra da passare ai programmi.
        """
        multiprocessing.freeze_support()
        additional_args = additional_args or []

        for idx in range(1, self.num_programs + 1):
            program_path = f"{self.base_program_path}{idx}"  # Costruisce il nome del programma (es: abundance1)
            command = [program_path, "-t"] + additional_args
            process = multiprocessing.Process(target=self.__run_command, args=(command, idx))
            self.processes.append(process)
            process.start()

        logging.info("Tutti i processi sono stati avviati.")

        # Aspetta che tutti i processi finiscano
        for process in self.processes:
            process.join()

        logging.info("Tutti i processi sono stati completati.")

    @staticmethod
    def __run_command(command, idx):
        """
        Esegue un singolo comando tramite subprocess.

        :param command: Il comando da eseguire (lista).
        :param idx: Identificatore del processo.
        """
        try:
            logging.info("Avvio del processo %d con comando: %s", idx, " ".join(command))
            subprocess.run(command, check=True)
            logging.info("Processo %d completato con successo.", idx)
        except subprocess.CalledProcessError as e:
            logging.error("Errore nel processo %d: %s", idx, e)

BASE_PROGRAM_PATH=f"{path_to_abundance}abundance"
NUM_PROGRAMS=12
# Integrazione nel codice principale
def main_integration():
    """
    Integrazione del ParallelRunner nel codice principale.
    """
    # Configurazione
    BASE_PROGRAM_PATH = f"{path_to_abundance}abundance"  # Sostituisci con il percorso base dei programmi
    NUM_PROGRAMS = 12  # Numero totale di programmi da eseguire

    # Eventuali argomenti aggiuntivi da passare ai programmi
    ADDITIONAL_ARGS = []

    # Logging
    logging.basicConfig(level=logging.INFO)

    # Esegui i programmi in parallelo
    runner = ParallelRunner(BASE_PROGRAM_PATH, NUM_PROGRAMS)
    runner.run_parallel(additional_args=ADDITIONAL_ARGS)

# Integra nel tuo codice principale, richiamando questa funzione
# if __name__ == "__main__":
#     main_integration()


In [ ]:
NLTE_intCorrection()
4840.172594886473, 4750, 5000
0.9606846996481666,  1.5,  1.5
-1.3314922098224213,  -1.25,  -1.5
2.3176700385594797

In [ ]:
#### CONTINUOUS SEARCH BLOCK ####
# for i in range(0, len(b)):    
#     os.chdir(path_to_output)
#     Y = [b[i][0], b[i][1], simpleAnalysisSingle(path_to_output, f"HARPN{i}.fits_INT{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}")[0], f"HARPN{i}.fits_INT{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}"]
#     for j in range(18):
#         os.system(f"mkdir harpn{i}")        
#         Y=avvicina_interpol(path_to_output, Y2[3], Y[0], Y[1], Y[2], f"H{i}.fits")
#         print(Y)
############################################

for i in range(1, 18):
    t_start1 = 4315
    t_start2= 4610
    g_start1=0.82
    g_start2=1.73
    m_start1=-1.14
    m_start2=-0.58
    crea_ATMinterpol(t_start1, g_start1, m_start1, f"HARPN{i}.fits")
    crea_ATMinterpol(t_start2, g_start2, m_start2, f"HARPN{i}.fits")
    Y = [4200, 1.0, -0.5, f"HARPN{i}.fits_INT{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}", vturbo(t_start1, g_start1, m_start1)]   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
    Y2 = [5000, 1.80, -1.50, f"HARPN{i}.fits_INT{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}", vturbo(t_start2, g_start2, m_start2)]# Un secondo set di condizioni iniziali viene creato, la ricerca giunge a termine se i due convergono ad un unico risultato, in questo modo non solo esploriamo più spazio con meno rischi di rimane nei minimi locali, ma è anche un modo per valutare la bontà della ricerca
    os.chdir(path_to_output)
    os.system(f"mkdir haru{i}")  ### crea cartella dove inserire i risultati
    o = 0
    boundary_check = False
    for j in range(25): ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
        if boundary_check:
            if Y[1] <= 0.0:
                Y[1] = 2.5
                Y[0] = 4500

            if Y[0] <= 3700:
                Y[1] = 1.2
                Y[0] = 5100

            if Y[1] > 3.2:
                Y[1] = 0.5
                Y[0] = 4100

            if Y2[1] <= 0.0:
                Y2[1] = 2.5
                Y2[0] = 4500

            if Y2[0] <= 3700:
                Y2[1] = 1.2
                Y2[0] = 5100

            if Y2[1] > 3.2:
                Y2[1] = 0.5
                Y2[0] = 4100
        Y_old = Y
        Y2_old = Y2
        Y=avvicina_interpol4(path_to_output, Y[3], Y[0], Y[1], Y[2], Y[4], f"HARPN{i}.fits")
        Y2=avvicina_interpol4(path_to_output, Y2[3], Y2[0], Y2[1], Y2[2], Y[4], f"HARPN{i}.fits")
        copia(f"AH_STELLA{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}")
        copia(f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}")
        #copia(f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA2{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}")
        if ((abs(Y_old[0] - Y[0])<2) and (abs(Y_old[1] - Y[1])<0.02) and (abs(Y_old[2] - Y[2])<0.05)) and ((abs(Y2_old[0] - Y2[0])<2) and (abs(Y2_old[1] - Y2[1])<0.02) and (abs(Y2_old[2] - Y2[2])<0.05)):
            break
    folderLog(path_to_output+f"haru{i}", 2)

for i in range(1, 18): #questi cicli for servono unicamente a ciclare su diverse stelle
    print(i)
    folderLog(path_to_output+f"haru{i}", 2)

os.chdir(path_to_output)
with open("final_results.txt", "w") as out:
    for i in range(2, 18):
        y=final_atmo(path_to_output + f"haru{i}")
        out.write(str(np.asarray(y)))
        out.write('\n')

# #f"HARPN{i}.fits{b[i][2]:.2f}_{b[i][0]:.0f}_{b[i][1]:.2f}"

In [9]:
class Stella:
    def __init__(self, fits_name='', rad_vel=-20, t_eff=0, LOGg=0, metal=0, v_turbulence=0):
        """
        Inizializza un oggetto Stella.
        
        :param nome: Nome della stella
        :param temperatura: Temperatura della stella (in Kelvin)
        :param spettro: Lista o array con i dati dello spettro
        """
        self.fits_name = fits_name
        self.nome = nome_da_Fits(nome=self.fits_name)
        self.t_eff = t_eff
        self.v_r = rad_vel
        self.logg = LOGg
        self.metal = metal
        self.v_m = v_turbulence
        self.folder = path_to_output + self.nome
        self.fits_folder = path_to_fitsFiles #path_to_ARES
        self.param_error = -1
        self.ALL_STARS = []
        crea_cartella(self.folder)
        os.chdir(self.fits_folder)
        self.fits = (fits.open(self.fits_name))
        hdul = fits.open(self.fits_name)
        self.fits_data = hdul[0].data
        if self.v_r == -20:
            self.v_r = hdul[0].header.get('TNG TEL TARG RADVEL')
        self.xi = self.fits[0].header.get('CRVAL1')
        self.dxi= self.fits[0].header.get('CDELT1')
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)

    def saveFits(self, new_filename="spettro_corretto.fits"):
        new_header = self.fits[0].header.copy()
        new_header['CRVAL1'] = self.xi
        new_header['CDELT1'] = self.dxi # Nuovo incremento
        new_header['TNG TEL TARG RADVEL'] = self.v_r
        new_hdu = fits.PrimaryHDU(data=self.fits_data, header=new_header)
        new_hdu.writeto(new_filename, overwrite=True)
        self.fits.close()
        print(f"File FITS aggiornato salvato come {new_filename}")
        self.fits_name = new_filename
        self.fits = (fits.open(self.fits_name))
        hdul = fits.open(self.fits_name)
        self.fits_data = hdul[0].data
        self.v_r = hdul[0].header.get('TNG TEL TARG RADVEL')
        self.xi = self.fits[0].header.get('CRVAL1')
        self.dxi= self.fits[0].header.get('CDELT1')
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)

    def EW(self, limits=[10, 150]):
        preparaAbundance(self.fits_name, eqw_low=limits[0], eqw_high=limits[1])

    def creaModel(self, T=-1, logg=10, MH_g=10, microTurb=10, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
        '''T: temperatura (kelvin)
        logg: gravità superficiale (dex)
        MH_g: [M/H] (dex).\n
        Processore il "numero del processore" che dovrà usare il modello per il calcolo. \n
        Crea il file ModelEx, dove x è il numero del processore.'''
        nome_Stella = self.fits_name
        if T==-1:
            T=self.t_eff
        if logg==10:
            logg=self.logg
        if MH_g ==10:
            MH_g = self.metal
        if microTurb == 10:
            microTurb = self.v_m        
        os.chdir(path_to_interpol)
        #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        vturb=microTurb
        commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
        os.system(commandInterpol)
        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()
            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]
            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)
        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/Model{processore}"
        os.system(comandoATMOS)
        #preparaAbundance(nome_Stella)
        os.chdir(path_to_abundance)
        creaParamfileAbnd1(vturb, processore)
        return processore

    def x_axis(self):
        returner = np.zeros(self.fits[0].header.get("NAXIS1"))
        for i in range(self.fits[0].header.get("NAXIS1")):
            returner[i] = self.xi + self.dxi*i
        self.xaxis = returner
        return self.xaxis

    def next_param(self, NLTE=0, searchMatrix=s_Matrix):
        #blocca_stampe()
        temperatu = self.t_eff
        G = self.logg
        M = self.metal
        V = self.v_m
        nome_file = f"HD115575_-1.99_4342_0.63_1.94"
        nome_stella = self.fits_name
        path_to_stella = self.folder
        if NLTE==0:
            temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
            temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
        elif NLTE == 1:
            temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
            temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
        sgn=1
        g=-1.0
        teff=-1.0
        met=1.0
        DG = 0
        DT = 0
        returner = []
        v=sp.symbols('v')
        # A = np.array([[dsdg, dsdt, dsdm],
        #      [dfdg, dfdt, dfdm],
        #      [dydg, dydt, dydm]])
        #riattiva_stampe()
        #sp.pprint(s_Matrix)
        A = searchMatrix
        AA=A.subs(v, V)
        A_numpy = np.array(AA.tolist(), dtype=float)
        A_inv = np.linalg.inv(A_numpy)
        f=-1*((temp0[0])-(temp1[0]))
        s = -1*(temp0[1])
        m = -1*(temp0[0]-(M))
        ewr = -1*(temp0[3])
        B = np.array([s, f, m, ewr])
        #blocca_stampe()
        if abs(temp0[1]) > 0.0001 or abs((temp0[0])-(temp1[0])) > 0.0003 or abs(M - (temp0[0]))>0.001:
            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
            DT = (np.dot(A_inv, B))[0]
            DG = (np.dot(A_inv, B))[1]
            DM = (np.dot(A_inv, B))[2]
            Dv = (np.dot(A_inv, B))[3]
            print(DT)
            print(DG)
            print(DM)
            print(Dv)
            print(f)
            print(s)
            print(m)
            print(ewr)
            if abs(DG) > 0.5:
                DG=0.5*(DG/abs(DG))
            if abs(DT) > 250:
                DT = 250*(DT/abs(DT))
            if abs(DM) > 0.5:
                DM = 0.5*(DM/abs(DM))
            if abs(Dv) > 0.3:
                Dv = 0.3*(Dv/abs(Dv))
            temperatu=temperatu+DT
            g=G+DG
            met = M+DM
            velturb=V+Dv
            teff= temperatu
        returner.append(teff)
        returner.append(g)
        returner.append(met)
        returner.append(velturb)
        return returner

    def calcola_abundance(processore):
        if __name__ == "__main__":
            main_integration()
    
    def searchParam(self):
        t_start1 = int(random.uniform(4000, 5000))
        t_start2= int(random.uniform(4000, 5000))
        g_start1=random.uniform(0, 2)
        g_start2=random.uniform(0, 2)
        m_start1=random.uniform(-2, 0)
        m_start2=random.uniform(-2, 0)
        self.dopcor()
        self.saveFits()
        crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name)
        crea_ATMinterpol(t_start2, g_start2, m_start2, self.fits_name)
        Y = [t_start1, g_start1, m_start1, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}", vturbo(t_start1, g_start1, m_start1)]   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
        Y2 = [t_start2, g_start2, m_start2, f"{self.nome}_{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}", vturbo(t_start2, g_start2, m_start2)]# Un secondo set di condizioni iniziali viene creato, la ricerca giunge a termine se i due convergono ad un unico risultato, in questo modo non solo esploriamo più spazio con meno rischi di rimane nei minimi locali, ma è anche un modo per valutare la bontà della ricerca
        os.chdir(path_to_output)
        o = 0
        boundary_check = True
        for j in range(15): ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
            if boundary_check:
                if Y[1] <= 0.0:
                    Y[1] = 0.5


                if Y[0] <= 3700:
                    Y[0] = 3750

                if Y[1] > 2.8:
                    Y[1] = 2.5


                if abs(Y[4] - vturbo(Y[0], Y[1], Y[2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    Y[4] = vturbo(Y[0], Y[1], Y[2])-0.081
                if (Y[4] - vturbo(Y[0], Y[1], Y[2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    Y[4] = vturbo(Y[0], Y[1], Y[2])+0.081
                
                if Y2[1] <= 0.0:
                    Y2[1] = 0.5

                if Y2[0] <= 3700:
                    Y2[0] = 3750

                if Y2[1] > 2.8:
                    Y2[1] = 2.5

                
                if (Y2[4] - vturbo(Y2[0], Y2[1], Y2[2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    Y2[4] = vturbo(Y2[0], Y2[1], Y2[2])-0.1
                if (Y2[4] - vturbo(Y2[0], Y2[1], Y2[2])) < -0.081:
                    print("NOOOOOOOOOOOOOOOOO")
                    Y2[4] = vturbo(Y2[0], Y2[1], Y2[2])+0.081
            Y_old = Y
            Y2_old = Y2
            
            Y=avvicina_interpol5(path_to_output, Y[3], Y[0], Y[1], Y[2], Y[4], self.nome, NLTE=1)
            Y2=avvicina_interpol6(path_to_output, Y2[3], Y2[0], Y2[1], Y2[2], Y[4], self.nome, NLTE=1)
            copia(f"{self.nome}_{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}_{Y[4]:.2f}", path_to_output, self.folder, f"{self.nome}_{Y[2]:.2f}_{Y[0]:.0f}_{Y[1]:.2f}_{Y[4]:.2f}")
            copia(f"{self.nome}_{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}_{Y2[4]:.2f}", path_to_output, self.folder, f"{self.nome}_{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}_{Y2[4]:.2f}")
            #copia(f"AH_STELLA{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}", path_to_output, path_to_output + f"haru{i}", f"AH_STELLA2{Y2[2]:.2f}_{Y2[0]:.0f}_{Y2[1]:.2f}")
            if ((abs(Y_old[0] - Y[0])<2) and (abs(Y_old[1] - Y[1])<0.02) and (abs(Y_old[2] - Y[2])<0.05)) and ((abs(Y2_old[0] - Y2[0])<2) and (abs(Y2_old[1] - Y2[1])<0.02) and (abs(Y2_old[2] - Y2[2])<0.05)):
                break

    def searchParam1(self, processori=1, NLTE=1, TRIALS = 50, t0=-1, g0=-2, m0=10, v0=-1):
        CORRECTION = True
        if t0==-1:
            t_start1 = int(random.uniform(4000, 4300))
        else:
            t_start1 = t0
        t_start2= int(random.uniform(4700, 5000))
        if g0==-2:
            g_start1= random.uniform(0, 1.0)
        else:
            g_start1 = t0
        g_start2=random.uniform(1.3, 1.75)
        if m0==10:
            m_start1= random.uniform(-2, 0)
        else:
            m_start1 = t0
        if v0==-1:
            v0=vturbo(t_start1, g_start1, m_start1)
        m_start2=random.uniform(-2, 0)
        self.dopcor()
        self.saveFits()
        blocca_stampe()
        crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name)
        crea_ATMinterpol(t_start2, g_start2, m_start2, self.fits_name)
        search_data1 = []
        search_data2 = []
        master_search_data1 = []
        master_search_data2 = []
        self.ALL_STARS=[]
        search_data1.append([t_start1, g_start1, m_start1, v0, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
        search_data2.append([t_start2, g_start2, m_start2, vturbo(t_start2, g_start2, m_start2), f"{self.nome}_{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
        os.chdir(path_to_output)
        o = 0
        boundary_check = True
        s=[]
        F=[]
        Y=[]
        S=[]
        crea_cartella(self.folder)
        crea_cartella(self.folder+"/TEMP")
        DDT1 = 100
        DDG1 = 0.15
        DDM1 = 0.15
        DDV1 = 0.15
        DDT2 = 100
        DDG2 = 0.15
        DDM2 = 0.15
        DDV2 = 0.15
        best1=1
        best2=1
        nome_temp=self.nome
        for j in range(TRIALS): ## Numero di iterazioni
            if boundary_check: ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
                if search_data1[0][1] <= 0.0:
                    search_data1[0][1] = 0.5
                if search_data1[0][0] <= 3700:
                    search_data1[0][0] = 3750
                if search_data1[0][1] > 2.8:
                    search_data1[0][1] = 2.5
                if abs(search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])-0.081
                if (search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])+0.081
                
                if search_data2[0][1] <= 0.0:
                    search_data2[0][1] = 0.5
                if search_data2[0][0] <= 3700:
                    search_data2[0][0] = 3750
                if search_data2[0][1] > 2.8:
                    search_data2[0][1] = 2.5
                if abs(search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])-0.081
                if (search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])+0.081

            for k in range(int(processori/2)): # Crea i modelli da analizzare
                if k == 0:
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], processore=k+1)
                    print(f"{search_data1[k][0]}, {search_data1[k][1]}, {search_data1[k][2]}, {search_data1[k][3]}")
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], processore=k+1+int(processori/2))
                    print(f"{search_data2[k][0]}, {search_data2[k][1]}, {search_data2[k][2]}, {search_data2[k][3]}")
                else:
                    if abs(DDT1) < 6:
                        DDT1 = 6
                    if abs(DDG1) < 0.025:
                        DDG1=0.025
                    if abs(DDM1) < 0.025:
                        DDM1=0.025
                    if abs(DDV1) < 0.025:
                        DDV1 = 0.025
                    if abs(DDT2) < 6:
                        DDT2 = 6
                    if abs(DDG2) < 0.025:
                        DDG2=0.025
                    if abs(DDM2) < 0.025:
                        DDM2=0.025
                    if abs(DDV2) < 0.052:
                        DDV2 = 0.025
                    if best1 > 100:
                        best1 = 100
                    search_data1.append([np.random.uniform(search_data1[0][0]-DDT1,search_data1[0][0]+DDT1), np.random.uniform(search_data1[0][1]-DDG1, search_data1[0][1]+DDG1), np.random.uniform(search_data1[0][2]-DDM1, search_data1[0][2]+DDM1), np.random.uniform(search_data1[0][3]+DDV1, search_data1[0][3]-DDV1)])
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], k+1)
                    if best2>100:
                        best2=100
                    search_data2.append([np.random.uniform(search_data2[0][0]-DDT2,search_data2[0][0]+DDT2), np.random.uniform(search_data2[0][1]-DDG2, search_data2[0][1]+DDG2), np.random.uniform(search_data2[0][2]-DDM2, search_data2[0][2]+DDM2), np.random.uniform(search_data2[0][3]+DDV2, search_data2[0][3]-DDV2)])
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], k+1+int(processori/2))
            if __name__ == "__main__": # Calcolo abbondanze
                main_integration()
            copia("stellaFes", path_to_abundance, self.folder, "stellaFes")
            copia("stellaFes", path_to_abundance, self.folder+"/TEMP", "stellaFes")
            best1=10000
            best1_index=-1
            best2=10000
            best2_index=-1
            for i in range(int(processori/2)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                self.nome = "1" + nome_temp
                nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                sposta(f"PINOLOS{i+1}", path_to_abundance, self.folder+"/TEMP", nome_output1)
                if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP"):
                    best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP")
                    best1_index=i
                self.nome = "2" + nome_temp
                nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                sposta(f"PINOLOS{i+1+int(processori/2)}", path_to_abundance, self.folder+"/TEMP", nome_output2)
                if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP"):
                    best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP")
                    best2_index=i
            self.nome=nome_temp
            tt1=search_data1[best1_index][0]
            gg1=search_data1[best1_index][1]
            mm1=search_data1[best1_index][2]
            vv1=search_data1[best1_index][3]
            tt2=search_data2[best2_index][0]
            gg2=search_data2[best2_index][1]
            mm2=search_data2[best2_index][2]
            vv2=search_data2[best2_index][3]
            (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2]])
            riattiva_stampe()
            if best1 <= 0.05:
                if best2 < best1:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    return 0
                else:
                    print(f"ECCOLA! T: {tt1}\nG: {gg1}\nM: {mm1}\nv: {vv1}\nDelta: {best1}")
                    self.t_eff = tt1
                    self.logg = gg1
                    self.metal = mm1
                    self.v_m = vv1
                    self.param_error = best2
                    return 0
            if best2 <= 0.05:
                print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                self.t_eff = tt2
                self.logg = gg2
                self.metal = mm2
                self.v_m = vv2
                self.param_error = best2
                return 0
            blocca_stampe()
            nome_out1 = f"1{self.nome}_{mm1:.2f}_{tt1:.0f}_{gg1:.2f}_{vv1:.2f}"
            nome_out2 = f"2{self.nome}_{mm2:.2f}_{tt2:.0f}_{gg2:.2f}_{vv2:.2f}"
            if CORRECTION==True: #Dalla migliore atmosfera calcola i parametri della successiva
                if NLTE==0:
                    temp01 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.0)
                    temp11 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.1)
                    temp02 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.0)
                    temp12 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.1)
                elif NLTE == 1:
                    temp01 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.0)
                    temp11 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.1)
                    temp02 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.0)
                    temp12 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.1)
                master_search_data1.append([[tt1, gg1, mm1, vv1], [(temp01[1]), (temp01[0])-(temp11[0]), temp01[0]-(mm1), (temp01[3])]])
                master_search_data2.append([[tt2, gg2, mm2, vv2], [(temp02[1]), (temp02[0])-(temp12[0]), temp02[0]-(mm2), (temp02[3])]])
                if j==0:
                    FIXD_MAT = 1
                if FIXD_MAT == 1:
                    v1=sp.symbols('v')
                    v2=sp.symbols('v')
                    A1 = s_Matrix
                    A2 = s_Matrix
                    AA1=A1.subs(v1, vv1)
                    AA2=A2.subs(v2, vv2)
                    A_numpy1 = np.array(AA1.tolist(), dtype=float)
                    A_inv1 = np.linalg.inv(A_numpy1)
                    A_numpy2 = np.array(AA2.tolist(), dtype=float)
                    A_inv2 = np.linalg.inv(A_numpy2)
                    f1=-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])
                    #blocca_stampe()
                    if abs(s1)<0.001 and abs(m1) < 0.01:
                        DT1 = 0
                        DM1 = 0
                        Dv1 = 0
                        s1 = 0
                        m1 = 0
                        ewr1 = 0
                        B1 = np.array([s1, f1, m1, ewr1])
                        DM1 = (np.dop(A_inv1, B1), [1])
                    else:
                        if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT1 = (np.dot(A_inv1, B1))[0]
                            DG1 = (np.dot(A_inv1, B1))[1]
                            DM1 = (np.dot(A_inv1, B1))[2]
                            Dv1 = (np.dot(A_inv1, B1))[3]
                            if abs(DG1) > 0.5:
                                DG1=0.5*(DG1/abs(DG1))
                            if abs(DT1) > 250:
                                DT1 = 250*(DT1/abs(DT1))
                            if abs(DM1) > 0.25:
                                DM1 = 0.25*(DM1/abs(DM1))
                            if abs(Dv1) > 0.3:
                                Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT2 = (np.dot(A_inv2, B2))[0]
                        DG2 = (np.dot(A_inv2, B2))[1]
                        DM2 = (np.dot(A_inv2, B2))[2]
                        Dv2 = (np.dot(A_inv2, B2))[3]
                        if abs(DG2) > 0.5:
                            DG2=0.5*(DG2/abs(DG2))
                        if abs(DT2) > 250:
                            DT2 = 250*(DT2/abs(DT2))
                        if abs(DM2) > 0.25:
                            DM2 = 0.25*(DM2/abs(DM2))
                        if abs(Dv2) > 0.3:
                            Dv2 = 0.3*(Dv2/abs(Dv2))
                    FIXD_MAT=0
                else:
                    A_numpy1 = crea_dynMatrix(master_search_data1[j-1][0], master_search_data1[j-1][1], master_search_data1[j][0], master_search_data1[j][1])
                    DDT1 = 2*(master_search_data1[j-1][0][0]-master_search_data1[j][0][0])
                    DDG1 = 2*(master_search_data1[j-1][0][1]-master_search_data1[j][0][1])
                    DDM1 = 2*(master_search_data1[j-1][0][2]-master_search_data1[j][0][2])
                    DDV1 = 2*(master_search_data1[j-1][0][3]-master_search_data1[j][0][3])
                    A_inv1 = np.linalg.pinv(A_numpy1)
                    f1 =-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])

                    A_numpy2 = crea_dynMatrix(master_search_data2[j-1][0], master_search_data2[j-1][1], master_search_data2[j][0], master_search_data2[j][1])
                    DDT2 = 2*(master_search_data2[j-1][0][0]-master_search_data2[j][0][0])
                    DDG2 = 2*(master_search_data2[j-1][0][1]-master_search_data2[j][0][1])
                    DDM2 = 2*(master_search_data2[j-1][0][2]-master_search_data2[j][0][2])
                    DDV2 = 2*(master_search_data2[j-1][0][3]-master_search_data2[j][0][3])
                    A2_inv1 = np.linalg.pinv(A_numpy1)
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])

                    #blocca_stampe()
                    if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT1 = (np.dot(A_inv1, B1))[0]
                        DG1 = (np.dot(A_inv1, B1))[1]
                        DM1 = (np.dot(A_inv1, B1))[2]
                        Dv1 = (np.dot(A_inv1, B1))[3]
                        if abs(DG1) > 0.5:
                            DG1=0.5*(DG1/abs(DG1))
                        if abs(DT1) > 250:
                            DT1 = 250*(DT1/abs(DT1))
                        if abs(DM1) > 0.25:
                            DM1 = 0.25*(DM1/abs(DM1))
                        if abs(Dv1) > 0.3:
                            Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT2 = (np.dot(A_inv2, B2))[0]
                        DG2 = (np.dot(A_inv2, B2))[1]
                        DM2 = (np.dot(A_inv2, B2))[2]
                        Dv2 = (np.dot(A_inv2, B2))[3]
                        if abs(DG2) > 0.5:
                            DG2=0.5*(DG2/abs(DG2))
                        if abs(DT2) > 250:
                            DT2 = 250*(DT2/abs(DT2))
                        if abs(DM2) > 0.25:
                            DM2 = 0.25*(DM2/abs(DM2))
                        if abs(Dv2) > 0.3:
                            Dv2 = 0.3*(Dv2/abs(Dv2))
                temperatu1=tt1+DT1
                G1=gg1+DG1
                velturb1=vv1+Dv1
                teff1= temperatu1
                temperatu2=tt2+DT2
                G2=gg2+DG2
                velturb2=vv2+Dv2
                teff2= temperatu2
            search_data1 = []
            search_data1.append([teff1, G1, mm1+DM1, velturb1])
            search_data2 = []
            search_data2.append([teff2, G2, mm2+DM2, velturb2])
        #riattiva_stampe()
        riattiva_stampe()
        self.nome = "1" + nome_temp
        O1 = self.evaluate_Models(tt1, gg1, mm1, vv1, self.folder+"/TEMP")
        self.nome = "2" + nome_temp
        O2 = self.evaluate_Models(tt2, gg2, mm2, vv2, self.folder+"/TEMP")
        print(O1)
        print(O2)
        self.nome = nome_temp
        print(f"{tt1}, {gg1}, {mm1}. {vv1}")
        print(f"{tt2}, {gg2}, {mm2}. {vv2}")
        vettori_selezionati = []
        for coppia in self.ALL_STARS:
            # Verifica che la coppia contenga due vettori
            if len(coppia) == 2:
                # Confronta best1 e best2
                if coppia[0][0] < coppia[1][0]:
                    vettori_selezionati.append(coppia[0])  # Se best1 < best2, prendi il primo vettore
                else:
                    vettori_selezionati.append(coppia[1])  # Altrimenti, prendi il secondo vettore
        vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])
        self.t_eff = vettore_minimo[1]
        self.logg = vettore_minimo[2]
        self.metal = vettore_minimo[3]
        self.v_m = vettore_minimo[4]
        self.param_error = vettore_minimo[0]
        #lista_ordinata = sorted(self.ALL_STARS, key=lambda x: x[0])
#        self.crea_log(self.folder + "/TEMP")


    def searchParam2(self, processori=1, NLTE=1, TRIALS = 50, t0=-1, g0=-2, m0=10, v0=-1):
        '''Usare un numero di processori divisibile per 4'''
        if processori%4 != 0:
            print("Inserisci un numero di processori divisibile per 4. (e.g. processori=4, processori=12)")
            return 200
        CORRECTION = True
        if t0==-1:
            t_start1 = int(random.uniform(4000, 4300))
        else:
            t_start1 = t0
        t_start2= int(random.uniform(4700, 5000))
        if g0==-2:
            g_start1= random.uniform(0, 1.0)
        else:
            g_start1 = t0
        g_start2=random.uniform(1.3, 1.75)
        if m0==10:
            m_start1= random.uniform(-2, 0)
        else:
            m_start1 = t0
        if v0==-1:
            v0=vturbo(t_start1, g_start1, m_start1)
        m_start2=random.uniform(-2, 0)
        t_start3 = int(random.uniform(4000, 5000))
        g_start3 = random.uniform(0, 2)
        m_start3 = random.uniform(-2, 0)
        t_start4 = int(random.uniform(4000, 5000))
        g_start4 = random.uniform(0, 2)
        m_start4 = random.uniform(-2, 0)
        self.dopcor()
        self.saveFits()
        delete_files(path_to_abundance)
        #blocca_stampe()
        crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name)
        crea_ATMinterpol(t_start2, g_start2, m_start2, self.fits_name)
        crea_ATMinterpol(t_start3, g_start3, m_start3, self.fits_name)
        crea_ATMinterpol(t_start4, g_start4, m_start4, self.fits_name)
        search_data1 = []
        search_data2 = []
        search_data3 = []
        search_data4 = []
        master_search_data1 = []
        master_search_data2 = []
        master_search_data3 = []
        master_search_data4 = []
        self.ALL_STARS=[]
        search_data1.append([t_start1, g_start1, m_start1, v0, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
        search_data2.append([t_start2, g_start2, m_start2, vturbo(t_start2, g_start2, m_start2), f"{self.nome}_{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
        search_data3.append([t_start3, g_start3, m_start3, vturbo(t_start3, g_start3, m_start3), f"{self.nome}_{m_start3:.2f}_{t_start3:.0f}_{g_start3:.2f}"])
        search_data4.append([t_start4, g_start4, m_start4, vturbo(t_start4, g_start4, m_start4), f"{self.nome}_{m_start4:.2f}_{t_start4:.0f}_{g_start4:.2f}"])
        os.chdir(path_to_output)
        o = 0
        boundary_check = True
        s=[]
        F=[]
        Y=[]
        S=[]
        crea_cartella(self.folder)
        crea_cartella(self.folder+"/TEMP")
        if True: #help block
            DDT1 = 100
            DDG1 = 0.15
            DDM1 = 0.15
            DDV1 = 0.15
            DDT2 = 100
            DDG2 = 0.15
            DDM2 = 0.15
            DDV2 = 0.15
            DDT3 = 100
            DDG3 = 0.15
            DDM3 = 0.15
            DDV3 = 0.15
            DDT4 = 100
            DDG4 = 0.15
            DDM4 = 0.15
            DDV4 = 0.15
            best1=1
            best2=1
            best3=1
            best4=1
            old_best1=200000
            old_best2=200000
            old_best3=200000
            old_best4=200000
            old_best1_index=-2
            old_best2_index=-2
            old_best3_index=-2
            old_best4_index=-2
        nome_temp=self.nome
        for j in range(TRIALS): ## Numero di iterazioni
            if boundary_check: ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
                if search_data1[0][1] <= 0.0:
                    search_data1[0][1] = 0.5
                if search_data1[0][0] <= 3700:
                    search_data1[0][0] = 3750
                if search_data1[0][1] > 2.8:
                    search_data1[0][1] = 2.5
                if abs(search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])-0.081
                if (search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])+0.081
                
                if search_data2[0][1] <= 0.0:
                    search_data2[0][1] = 0.5
                if search_data2[0][0] <= 3700:
                    search_data2[0][0] = 3750
                if search_data2[0][1] > 2.8:
                    search_data2[0][1] = 2.5
                if abs(search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])-0.081
                if (search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])+0.081
                
                if search_data3[0][1] <= 0.0:
                    search_data3[0][1] = 0.5
                if search_data3[0][0] <= 3700:
                    search_data3[0][0] = 3750
                if search_data3[0][1] > 2.8:
                    search_data3[0][1] = 2.5
                if abs(search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])-0.081
                if (search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])+0.081
                

                if search_data4[0][1] <= 0.0:
                    search_data4[0][1] = 0.5
                if search_data4[0][0] <= 3700:
                    search_data4[0][0] = 3750
                if search_data4[0][1] > 2.8:
                    search_data4[0][1] = 2.5
                if abs(search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])-0.081
                if (search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])+0.081

            for k in range(int(processori/4)): # Crea i modelli da analizzare
                if k == 0:
                    a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP")
                    search_data1[k][0] = a1[0]
                    search_data1[k][1] = a1[1]
                    search_data1[k][2] = a1[2]
                    search_data1[k][3] = a1[3]
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], processore=k+1)
                    print(f"{search_data1[k][0]}, {search_data1[k][1]}, {search_data1[k][2]}, {search_data1[k][3]}")
                    a1=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP")
                    search_data2[k][0] = a1[0]
                    search_data2[k][1] = a1[1]
                    search_data2[k][2] = a1[2]
                    search_data2[k][3] = a1[3]
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], processore=k+1+int(processori/4))
                    print(f"{search_data2[k][0]}, {search_data2[k][1]}, {search_data2[k][2]}, {search_data2[k][3]}")
                    a1=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP")
                    search_data3[k][0] = a1[0]
                    search_data3[k][1] = a1[1]
                    search_data3[k][2] = a1[2]
                    search_data3[k][3] = a1[3]
                    self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], processore=k+1+2*int(processori/4))
                    print(f"{search_data3[k][0]}, {search_data3[k][1]}, {search_data3[k][2]}, {search_data3[k][3]}")
                    a1=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP")
                    search_data4[k][0] = a1[0]
                    search_data4[k][1] = a1[1]
                    search_data4[k][2] = a1[2]
                    search_data4[k][3] = a1[3]
                    self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], processore=k+1+3*int(processori/4))
                    print(f"{search_data4[k][0]}, {search_data4[k][1]}, {search_data4[k][2]}, {search_data4[k][3]}")
                else:
                    if abs(DDT1) < 6:
                        DDT1 = 6
                    if abs(DDG1) < 0.025:
                        DDG1=0.025
                    if abs(DDM1) < 0.025:
                        DDM1=0.025
                    if abs(DDV1) < 0.025:
                        DDV1 = 0.025
                    if abs(DDT2) < 6:
                        DDT2 = 6
                    if abs(DDG2) < 0.025:
                        DDG2=0.025
                    if abs(DDM2) < 0.025:
                        DDM2=0.025
                    if abs(DDV2) < 0.052:
                        DDV2 = 0.025
                    if abs(DDT3) < 6:
                        DDT3 = 6
                    if abs(DDG3) < 0.025:
                        DDG3=0.025
                    if abs(DDM3) < 0.025:
                        DDM3=0.025
                    if abs(DDV3) < 0.052:
                        DDV3 = 0.025
                    if abs(DDT4) < 6:
                        DDT4 = 6
                    if abs(DDG4) < 0.025:
                        DDG4=0.025
                    if abs(DDM4) < 0.025:
                        DDM4=0.025
                    if abs(DDV4) < 0.052:
                        DDV4 = 0.025
                    if best1 > 100:
                        best1 = 100
                    search_data1.append([np.random.uniform(search_data1[0][0]-DDT1,search_data1[0][0]+DDT1), np.random.uniform(search_data1[0][1]-DDG1, search_data1[0][1]+DDG1), np.random.uniform(search_data1[0][2]-DDM1, search_data1[0][2]+DDM1), np.random.uniform(search_data1[0][3]+DDV1, search_data1[0][3]-DDV1)])
                    a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP")
                    search_data1[k][0] = a1[0]
                    search_data1[k][1] = a1[1]
                    search_data1[k][2] = a1[2]
                    search_data1[k][3] = a1[3]
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], k+1)
                    if best2>100:
                        best2=100
                    search_data2.append([np.random.uniform(search_data2[0][0]-DDT2,search_data2[0][0]+DDT2), np.random.uniform(search_data2[0][1]-DDG2, search_data2[0][1]+DDG2), np.random.uniform(search_data2[0][2]-DDM2, search_data2[0][2]+DDM2), np.random.uniform(search_data2[0][3]+DDV2, search_data2[0][3]-DDV2)])
                    a1=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP")
                    search_data2[k][0] = a1[0]
                    search_data2[k][1] = a1[1]
                    search_data2[k][2] = a1[2]
                    search_data2[k][3] = a1[3]
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], k+1+int(processori/4))
                    if best3>100:
                        best3=100
                    search_data3.append([np.random.uniform(search_data3[0][0]-DDT3,search_data3[0][0]+DDT3), np.random.uniform(search_data3[0][1]-DDG3, search_data3[0][1]+DDG3), np.random.uniform(search_data3[0][2]-DDM3, search_data3[0][2]+DDM3), np.random.uniform(search_data3[0][3]+DDV3, search_data3[0][3]-DDV3)])
                    a1=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP")
                    search_data3[k][0] = a1[0]
                    search_data3[k][1] = a1[1]
                    search_data3[k][2] = a1[2]
                    search_data3[k][3] = a1[3]
                    self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], k+1+2*int(processori/4))
                    if best4>100:
                        best4=100
                    search_data4.append([np.random.uniform(search_data4[0][0]-DDT2,search_data4[0][0]+DDT4), np.random.uniform(search_data4[0][1]-DDG4, search_data4[0][1]+DDG4), np.random.uniform(search_data4[0][2]-DDM4, search_data4[0][2]+DDM4), np.random.uniform(search_data4[0][3]+DDV4, search_data4[0][3]-DDV4)])
                    a1=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP")
                    search_data4[k][0] = a1[0]
                    search_data4[k][1] = a1[1]
                    search_data4[k][2] = a1[2]
                    search_data4[k][3] = a1[3]
                    self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], k+1+3*int(processori/4))
            if __name__ == "__main__": # Calcolo abbondanze
                main_integration()
            copia("stellaFes", path_to_abundance, self.folder, "stellaFes")
            copia("stellaFes", path_to_abundance, self.folder+"/TEMP", "stellaFes")
            best1=10000
            best1_index=-1
            best2=10000
            best2_index=-1
            best3=10000
            best3_index=-1
            best4=10000
            best4_index=-1
            for i in range(int(processori/4)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                self.nome = "1" + nome_temp
                nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                sposta(f"PINOLOS{i+1}", path_to_abundance, self.folder+"/TEMP", nome_output1)
                if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP"):
                    best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP")
                    best1_index=i
                self.nome = "2" + nome_temp
                nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                sposta(f"PINOLOS{i+1+int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output2)
                if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP"):
                    best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP")
                    best2_index=i
                
                self.nome = "3" + nome_temp
                nome_output3 = f"{self.nome}_{search_data3[i][2]:.2f}_{(search_data3[i][0]):.0f}_{search_data3[i][1]:.2f}_{search_data3[i][3]:.2f}"
                sposta(f"PINOLOS{i+1+2*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output3)
                if best3 > self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP"):
                    best3 = self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP")
                    best3_index=i
                self.nome = "4" + nome_temp
                nome_output4 = f"{self.nome}_{(search_data4[i][2]):.2f}_{search_data4[i][0]:.0f}_{search_data4[i][1]:.2f}_{search_data4[i][3]:.2f}"
                sposta(f"PINOLOS{i+1+3*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output4)
                if best4 > self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP"):
                    best4 = self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP")
                    best4_index=i
            if best1>old_best1 and best1 < 15:
                best1=old_best1
                best1_index=old_best1_index
            if best2>old_best2 and best2 < 15:
                best2=old_best2
                best2_index=old_best2_index
            if best3>old_best3 and best3 < 15:
                best3=old_best3
                best3_index=old_best3_index
            if best4>old_best4 and best4 < 15:
                best4=old_best4
                best4_index=old_best4_index
            self.nome=nome_temp
            tt1=search_data1[best1_index][0]
            gg1=search_data1[best1_index][1]
            mm1=search_data1[best1_index][2]
            vv1=search_data1[best1_index][3]
            tt2=search_data2[best2_index][0]
            gg2=search_data2[best2_index][1]
            mm2=search_data2[best2_index][2]
            vv2=search_data2[best2_index][3]
            tt3=search_data3[best3_index][0]
            gg3=search_data3[best3_index][1]
            mm3=search_data3[best3_index][2]
            vv3=search_data3[best3_index][3]
            tt4=search_data4[best4_index][0]
            gg4=search_data4[best4_index][1]
            mm4=search_data4[best4_index][2]
            vv4=search_data4[best4_index][3]
            (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
            #riattiva_stampe()
            if best1 <= 0.05:
                if best2 < best1 and best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    return 0
                elif best3 < best1 and best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                elif best4 < best1 and best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
                else:
                    print(f"ECCOLA! T: {tt1}\nG: {gg1}\nM: {mm1}\nv: {vv1}\nDelta: {best1}")
                    self.t_eff = tt1
                    self.logg = gg1
                    self.metal = mm1
                    self.v_m = vv1
                    self.param_error = best2
                    return 0
            if best2 <= 0.05:
                if best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    return 0
                elif best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                elif best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
            if best3 <= 0.05:
                if best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                else:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
            #blocca_stampe()
            nome_out1 = f"1{self.nome}_{mm1:.2f}_{tt1:.0f}_{gg1:.2f}_{vv1:.2f}"
            nome_out2 = f"2{self.nome}_{mm2:.2f}_{tt2:.0f}_{gg2:.2f}_{vv2:.2f}"
            nome_out3 = f"3{self.nome}_{mm3:.2f}_{tt3:.0f}_{gg3:.2f}_{vv3:.2f}"
            nome_out4 = f"4{self.nome}_{mm4:.2f}_{tt4:.0f}_{gg4:.2f}_{vv4:.2f}"
            if CORRECTION==True: #Dalla migliore atmosfera calcola i parametri della successiva
                if NLTE==0:
                    temp01 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.0)
                    temp11 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.1)
                    temp02 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.0)
                    temp12 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.1)
                    temp03 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.0)
                    temp13 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.1)
                    temp04 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.0)
                    temp14 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.1)
                elif NLTE == 1:
                    temp01 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.0)
                    temp11 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.1)
                    temp02 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.0)
                    temp12 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.1)
                    temp03 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.0)
                    temp13 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.1)
                    temp04 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.0)
                    temp14 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.1)
                master_search_data1.append([[tt1, gg1, mm1, vv1], [(temp01[1]), (temp01[0])-(temp11[0]), temp01[0]-(mm1), (temp01[3])]])
                master_search_data2.append([[tt2, gg2, mm2, vv2], [(temp02[1]), (temp02[0])-(temp12[0]), temp02[0]-(mm2), (temp02[3])]])
                master_search_data3.append([[tt3, gg3, mm3, vv3], [(temp03[1]), (temp03[0])-(temp13[0]), temp03[0]-(mm3), (temp03[3])]])
                master_search_data4.append([[tt4, gg4, mm4, vv4], [(temp04[1]), (temp04[0])-(temp14[0]), temp04[0]-(mm4), (temp04[3])]])
                if j==0:
                    FIXD_MAT = 1
                if FIXD_MAT == 1:
                    v1=sp.symbols('v')
                    v2=sp.symbols('v')
                    v3=sp.symbols('v')
                    v4=sp.symbols('v')
                    A1 = s_Matrix
                    A2 = s_Matrix
                    A3 = s_Matrix
                    A4 = s_Matrix
                    AA1=A1.subs(v1, vv1)
                    AA2=A2.subs(v2, vv2)
                    AA3=A3.subs(v3, vv3)
                    AA4=A4.subs(v4, vv4)
                    A_numpy1 = np.array(AA1.tolist(), dtype=float)
                    A_inv1 = np.linalg.inv(A_numpy1)
                    A_numpy2 = np.array(AA2.tolist(), dtype=float)
                    A_inv2 = np.linalg.inv(A_numpy2)
                    A_numpy3 = np.array(AA3.tolist(), dtype=float)
                    A_inv3 = np.linalg.inv(A_numpy3)
                    A_numpy4 = np.array(AA4.tolist(), dtype=float)
                    A_inv4 = np.linalg.inv(A_numpy4)
                    f1=-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])
                    f3=-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = -1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])
                    #blocca_stampe()
                    if abs(s1)<0.001 and abs(m1) < 0.01:
                        DT1 = 0
                        DM1 = 0
                        Dv1 = 0
                        s1 = 0
                        m1 = 0
                        ewr1 = 0
                        B1 = np.array([s1, f1, m1, ewr1])
                        DM1 = (np.dop(A_inv1, B1), [1])
                    else:
                        if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT1 = (np.dot(A_inv1, B1))[0]
                            DG1 = (np.dot(A_inv1, B1))[1]
                            DM1 = (np.dot(A_inv1, B1))[2]
                            Dv1 = (np.dot(A_inv1, B1))[3]
                            if abs(DG1) > 0.5:
                                DG1=0.5*(DG1/abs(DG1))
                            if abs(DT1) > 250:
                                DT1 = 250*(DT1/abs(DT1))
                            if abs(DM1) > 0.25:
                                DM1 = 0.25*(DM1/abs(DM1))
                            if abs(Dv1) > 0.3:
                                Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(s2)<0.001 and abs(m2) < 0.01:
                        DT2 = 0
                        DM2 = 0
                        Dv2 = 0
                        s2 = 0
                        m2 = 0
                        ewr2 = 0
                        B2 = np.array([s2, f2, m2, ewr2])
                        DM2 = (np.dop(A_inv2, B2), [1])
                    else:
                        if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT2 = (np.dot(A_inv2, B2))[0]
                            DG2 = (np.dot(A_inv2, B2))[1]
                            DM2 = (np.dot(A_inv2, B2))[2]
                            Dv2 = (np.dot(A_inv2, B2))[3]
                            if abs(DG2) > 0.5:
                                DG2=0.5*(DG2/abs(DG2))
                            if abs(DT2) > 250:
                                DT2 = 250*(DT2/abs(DT2))
                            if abs(DM2) > 0.25:
                                DM2 = 0.25*(DM2/abs(DM2))
                            if abs(Dv2) > 0.3:
                                Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(s3)<0.001 and abs(m3) < 0.01:
                        DT3 = 0
                        DM3 = 0
                        Dv3 = 0
                        s3 = 0
                        m3 = 0
                        ewr3 = 0
                        B3 = np.array([s3, f3, m3, ewr3])
                        DM3 = (np.dop(A_inv3, B3), [1])
                    else:
                        if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT3 = (np.dot(A_inv3, B3))[0]
                            DG3 = (np.dot(A_inv3, B3))[1]
                            DM3 = (np.dot(A_inv3, B3))[2]
                            Dv3 = (np.dot(A_inv3, B3))[3]
                            if abs(DG3) > 0.5:
                                DG3=0.5*(DG3/abs(DG3))
                            if abs(DT2) > 250:
                                DT3 = 250*(DT3/abs(DT3))
                            if abs(DM3) > 0.25:
                                DM3 = 0.25*(DM3/abs(DM3))
                            if abs(Dv3) > 0.3:
                                Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(s4)<0.001 and abs(m4) < 0.01:
                        DT4 = 0
                        DM4 = 0
                        Dv4 = 0
                        s4 = 0
                        m4 = 0
                        ewr4 = 0
                        B4 = np.array([s4, f4, m4, ewr4])
                        DM4 = (np.dop(A_inv4, B4), [1])
                    else:
                        if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT4 = (np.dot(A_inv4, B4))[0]
                            DG4 = (np.dot(A_inv4, B4))[1]
                            DM4 = (np.dot(A_inv4, B4))[2]
                            Dv4 = (np.dot(A_inv4, B4))[3]
                            if abs(DG4) > 0.5:
                                DG4=0.5*(DG4/abs(DG4))
                            if abs(DT4) > 250:
                                DT4 = 250*(DT4/abs(DT4))
                            if abs(DM4) > 0.25:
                                DM4 = 0.25*(DM4/abs(DM4))
                            if abs(Dv2) > 0.3:
                                Dv4 = 0.3*(Dv4/abs(Dv4))
                    FIXD_MAT=0
                else:
                    A_numpy1 = crea_dynMatrix(master_search_data1[j-1][0], master_search_data1[j-1][1], master_search_data1[j][0], master_search_data1[j][1])
                    DDT1 = 2*(master_search_data1[j-1][0][0]-master_search_data1[j][0][0])
                    DDG1 = 2*(master_search_data1[j-1][0][1]-master_search_data1[j][0][1])
                    DDM1 = 2*(master_search_data1[j-1][0][2]-master_search_data1[j][0][2])
                    DDV1 = 2*(master_search_data1[j-1][0][3]-master_search_data1[j][0][3])
                    A_inv1 = np.linalg.pinv(A_numpy1)
                    f1 =-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])

                    A_numpy2 = crea_dynMatrix(master_search_data2[j-1][0], master_search_data2[j-1][1], master_search_data2[j][0], master_search_data2[j][1])
                    DDT2 = 2*(master_search_data2[j-1][0][0]-master_search_data2[j][0][0])
                    DDG2 = 2*(master_search_data2[j-1][0][1]-master_search_data2[j][0][1])
                    DDM2 = 2*(master_search_data2[j-1][0][2]-master_search_data2[j][0][2])
                    DDV2 = 2*(master_search_data2[j-1][0][3]-master_search_data2[j][0][3])
                    A2_inv1 = np.linalg.pinv(A_numpy2)
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])

                    A_numpy3 = crea_dynMatrix(master_search_data3[j-1][0], master_search_data3[j-1][1], master_search_data3[j][0], master_search_data3[j][1])
                    DDT3 = 2*(master_search_data3[j-1][0][0]-master_search_data3[j][0][0])
                    DDG3 = 2*(master_search_data3[j-1][0][1]-master_search_data3[j][0][1])
                    DDM3 = 2*(master_search_data3[j-1][0][2]-master_search_data3[j][0][2])
                    DDV3 = 2*(master_search_data3[j-1][0][3]-master_search_data3[j][0][3])
                    A_inv3 = np.linalg.pinv(A_numpy3)
                    f3 =-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])

                    A_numpy4 = crea_dynMatrix(master_search_data4[j-1][0], master_search_data4[j-1][1], master_search_data4[j][0], master_search_data4[j][1])
                    DDT4 = 2*(master_search_data4[j-1][0][0]-master_search_data4[j][0][0])
                    DDG4 = 2*(master_search_data4[j-1][0][1]-master_search_data4[j][0][1])
                    DDM4 = 2*(master_search_data4[j-1][0][2]-master_search_data4[j][0][2])
                    DDV4 = 2*(master_search_data4[j-1][0][3]-master_search_data4[j][0][3])
                    A4_inv1 = np.linalg.pinv(A_numpy4)
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = -1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])

                    #blocca_stampe()
                    if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT1 = (np.dot(A_inv1, B1))[0]
                        DG1 = (np.dot(A_inv1, B1))[1]
                        DM1 = (np.dot(A_inv1, B1))[2]
                        Dv1 = (np.dot(A_inv1, B1))[3]
                        if abs(DG1) > 0.5:
                            DG1=0.5*(DG1/abs(DG1))
                        if abs(DT1) > 250:
                            DT1 = 250*(DT1/abs(DT1))
                        if abs(DM1) > 0.25:
                            DM1 = 0.25*(DM1/abs(DM1))
                        if abs(Dv1) > 0.3:
                            Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT2 = (np.dot(A_inv2, B2))[0]
                        DG2 = (np.dot(A_inv2, B2))[1]
                        DM2 = (np.dot(A_inv2, B2))[2]
                        Dv2 = (np.dot(A_inv2, B2))[3]
                        if abs(DG2) > 0.5:
                            DG2=0.5*(DG2/abs(DG2))
                        if abs(DT2) > 250:
                            DT2 = 250*(DT2/abs(DT2))
                        if abs(DM2) > 0.25:
                            DM2 = 0.25*(DM2/abs(DM2))
                        if abs(Dv2) > 0.3:
                            Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT3 = (np.dot(A_inv3, B3))[0]
                        DG3 = (np.dot(A_inv3, B3))[1]
                        DM3 = (np.dot(A_inv3, B3))[2]
                        Dv3 = (np.dot(A_inv3, B3))[3]
                        if abs(DG3) > 0.5:
                            DG3=0.5*(DG3/abs(DG3))
                        if abs(DT3) > 250:
                            DT3 = 250*(DT3/abs(DT3))
                        if abs(DM3) > 0.25:
                            DM3 = 0.25*(DM3/abs(DM3))
                        if abs(Dv3) > 0.3:
                            Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT4 = (np.dot(A_inv4, B4))[0]
                        DG4 = (np.dot(A_inv4, B4))[1]
                        DM4 = (np.dot(A_inv4, B4))[2]
                        Dv4 = (np.dot(A_inv4, B4))[3]
                        if abs(DG4) > 0.5:
                            DG4=0.5*(DG4/abs(DG4))
                        if abs(DT4) > 250:
                            DT4 = 250*(DT4/abs(DT4))
                        if abs(DM4) > 0.25:
                            DM4 = 0.25*(DM4/abs(DM4))
                        if abs(Dv4) > 0.3:
                            Dv4 = 0.3*(Dv4/abs(Dv4))
                temperatu1=tt1+DT1
                G1=gg1+DG1
                velturb1=vv1+Dv1
                teff1= temperatu1
                temperatu2=tt2+DT2
                G2=gg2+DG2
                velturb2=vv2+Dv2
                teff2= temperatu2

                temperatu3=tt3+DT3
                G3=gg3+DG3
                velturb3=vv3+Dv3
                teff3= temperatu3
                temperatu4=tt4+DT4
                G4=gg4+DG4
                velturb4=vv4+Dv4
                teff4= temperatu4

            search_data1 = []
            search_data1.append([teff1, G1, mm1+DM1, velturb1])
            search_data2 = []
            search_data2.append([teff2, G2, mm2+DM2, velturb2])
            search_data3 = []
            search_data3.append([teff3, G3, mm3+DM3, velturb3])
            search_data4 = []
            search_data4.append([teff4, G4, mm4+DM4, velturb4])
            old_best1_index=best1_index
            old_best2_index=best2_index
            old_best3_index=best3_index
            old_best4_index=best4_index
            old_best1=best1
            old_best2=best2
            old_best3=best3
            old_best4=best4
        #riattiva_stampe()
        #riattiva_stampe()
        self.nome = "1" + nome_temp
        O1 = self.evaluate_Models(tt1, gg1, mm1, vv1, self.folder+"/TEMP")
        self.nome = "2" + nome_temp
        O2 = self.evaluate_Models(tt2, gg2, mm2, vv2, self.folder+"/TEMP")
        self.nome = "3" + nome_temp
        O3 = self.evaluate_Models(tt3, gg3, mm3, vv3, self.folder+"/TEMP")
        self.nome = "4" + nome_temp
        O4 = self.evaluate_Models(tt4, gg4, mm4, vv4, self.folder+"/TEMP")
        print(O1)
        print(O2)
        print(O3)
        print(O4)
        self.nome = nome_temp
        print(f"{tt1}, {gg1}, {mm1}. {vv1}")
        print(f"{tt2}, {gg2}, {mm2}. {vv2}")
        vettori_selezionati = []
        for quaterna in self.ALL_STARS:
            # Verifica che la quaterna contenga esattamente quattro vettori
            if len(quaterna) == 4:
                # Trova il vettore con il primo elemento più piccolo nella quaterna
                minimo_quaterna = min(quaterna, key=lambda x: x[0])
                vettori_selezionati.append(minimo_quaterna)

        # Trova il vettore con il primo elemento più piccolo tra i selezionati
        vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])

        # for coppia in self.ALL_STARS:
        #     # Verifica che la coppia contenga due vettori
        #     if len(coppia) == 2:
        #         # Confronta best1 e best2
        #         if coppia[0][0] < coppia[1][0]:
        #             vettori_selezionati.append(coppia[0])  # Se best1 < best2, prendi il primo vettore
        #         else:
        #             vettori_selezionati.append(coppia[1])  # Altrimenti, prendi il secondo vettore
        # vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])



        self.t_eff = vettore_minimo[1]
        self.logg = vettore_minimo[2]
        self.metal = vettore_minimo[3]
        self.v_m = vettore_minimo[4]
        self.param_error = vettore_minimo[0]
        #sposta()

    def dopcor(self):
        self.xi= corrDoppler(self.xi, self.v_r)
        self.dxi= corrDoppler(self.dxi, self.v_r)
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)
        self.v_r=0
        print("Correzione doppler!")

    def spectrum(self):
        Xi=self.xi
        Xf=self.xf
        os.chdir(self.fits_folder)
        file_fits = self.fits_name
        hdul = fits.open(file_fits)

        # Stampa informazioni sul file FITS
        hdul.info()

        print(f"{len(hdul[0].data)} {type(hdul[0].data)}")

        Y = hdul[0].data
        X = np.linspace(Xi, Xf, len(Y))

        plt.figure(figsize=(10, 6))
        plt.plot(X, Y, label="Spettro HARPS-N")
        plt.xlabel("Lunghezza d'onda (Å)")
        plt.ylabel("Flusso")
        plt.title("Spettro Stellare - HARPS-N")
        plt.legend()
        plt.grid()
        plt.show()

    def crea_log(self, FOLDER='', NLTE = 0):
        '''Crea una file di testo log.txt contenente i principali risultati dell'analisi atmosferica.
        folderLog2 stampa anche le quantità necessarie alla determinazione della microturbolenza. 
        Restituisce una lista con i parametri delle atmosfere vincenti.\n
        mode (1, 2 o 3) varia a seconda del tipo di parsing del nome dei file che vogliamo analizzare.
        1 per i file che iniziano con 'STELLA', 2 per quelli che iniziano con 'AH_STELLA'. \n mode=3 è per trattare un nome qualsiasi, in questo caso si DEVE specificare anche il nome tramite la variabile 'baseNome', eg. "HARPN1.fits_INT <--- per ora non implementato, solo per uso manuale (cioè modifica la funzione sparseFile3 se vuoi cambiare)'''
        if FOLDER=='':
            path = self.folder + "/TEMP"
        else:
            path = FOLDER
        old_dir = os.getcwd()
        os.chdir(path)
        a = os.listdir()
        b = []
        returner = np.zeros(9)
        baseNome = str(self.nome)+"_"
        for i in range(len(a)):
            if a[i] != "log.txt" and a[i] != "exit" and a[i] != 'stellaFes':
                b.append(a[i]) #in b i nomi delle stelle
        H = np.zeros((len(b), 9)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std(FeI), H[7]=std(FeII), H[8]=slope(EWR-A(Fe))
        min = 10
        j=0
        with open("log.txt", 'w') as f:
            f.write("    T      G      M    vturb    FeI        FeII           FeI-M/H        FeI-FeII       s(FeI)    EWR\n")
            for i in range(len(b)):
                print(os.getcwd())
                H[i][3] = (sparseFileName4(path, b[i], name_suffix=baseNome))[1]
                H[i][4] = (sparseFileName4(path, b[i], name_suffix=baseNome))[2]
                H[i][5] = (sparseFileName4(path, b[i], name_suffix=baseNome))[0]
                vturbos = (sparseFileName4(path, b[i], name_suffix=baseNome))[3]
                if NLTE == 0:
                    tempD = simpleAnalysisSingle(path, b[i], 26.1)
                    tempS = simpleAnalysisSingle(path, b[i], 26.0)
                elif NLTE == 1:
                    tempD = NLTE_AnalysisSingle(path, b[i], H[i][3],  H[i][4],  H[i][5], vturbos, 26.1)
                    tempS = NLTE_AnalysisSingle(path, b[i], H[i][3],  H[i][4],  H[i][5], vturbos, 26.0)
                H[i][0] = tempS[1]
                H[i][1] = float(tempS[0])
                H[i][2] = float(tempD[0])
                H[i][6] = tempS[2]
                H[i][7] = tempD[2]
                f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {vturbos:.2f}    {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}        {tempS[3]:.5f}\n")
        winners = []
        for i in range(len(b)): #erano 0.6                            0.21                    0.11
            if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
                winners.append([H[i][3], H[i][4], H[i][6], 100])
        ds=0.002
        dg=0.04
        dm=0.08
        q=1
        for u in range(100):
            for i in range(len(b)):
                if len(winners) < 4:
                    if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                        if 100-5*q <= -700:
                            winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                        else:
                            winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
                q=q+1
                ds=ds+0.001
                dg=dg+0.005
                dm=dm+0.01
        os.chdir(old_dir)
        return winners

    def evaluate_Models(self, t, g, M, V, folder=' ', eigen_weights=None):
        '''Restituisce il nome della migliore atmosfera'''
        if folder == ' ':
            folder = self.folder + "/TEMP"
        A = s_Matrix
        v=sp.symbols('v')
        AA=A.subs(v, V)
        A_numpy = np.array(AA.tolist(), dtype=float)
        A_inv = np.linalg.inv(A_numpy)
        DATI = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}")
        DATI2 = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", 26.1)
        s = -1*DATI[1]
        f=-0.0
        m = -0.0
        ewr = -0.0
        A1 = np.array([s, f, m, ewr])
        s = 0.00
        f=-1*(DATI[0]-DATI2[0])
        m = -0.0
        ewr = -0.0
        A2 = np.array([s, f, m, ewr])
        s = 0.00
        f=-0.0
        m = -1*(DATI[0]-M)
        ewr = -0.0
        A3 = np.array([s, f, m, ewr])
        s = 0.0
        f=-0.0
        m = -0.0
        ewr = -1*DATI[3]
        A4 = np.array([s, f, m, ewr])
        if eigen_weights==None:
            eigen_weights = [20, 0.05, 0.082, 0.7]
        if True:
            DT1 = abs((np.dot(A_inv, A1))[0])
            DG1 = abs((np.dot(A_inv, A1))[1])
            DM1 = abs((np.dot(A_inv, A1))[2])
            Dv1 = abs((np.dot(A_inv, A1))[3])
            DT2 = abs((np.dot(A_inv, A2))[0])
            DG2 = abs((np.dot(A_inv, A2))[1])
            DM2 = abs((np.dot(A_inv, A2))[2])
            Dv2 = abs((np.dot(A_inv, A2))[3])
            DT3 = abs((np.dot(A_inv, A3))[0])
            DG3 = abs((np.dot(A_inv, A3))[1])
            DM3 = abs((np.dot(A_inv, A3))[2])
            Dv3 = abs((np.dot(A_inv, A3))[3])
            DT4 = abs((np.dot(A_inv, A4))[0])
            DG4 = abs((np.dot(A_inv, A4))[1])
            DM4 = abs((np.dot(A_inv, A4))[2])
            Dv4 = abs((np.dot(A_inv, A4))[3])
            DT=(DT1**2+DT2**2+DT3**2+DT4**2)/(eigen_weights[0]**2)
            DG=(DG1**2+DG2**2+DG3**2+DG4**2)/(eigen_weights[1]**2)
            DM=(DM1**2+DM2**2+DM3**2+DM4**2)/(eigen_weights[2]**2)
            Dv=(Dv1**2+Dv2**2+Dv3**2+Dv4**2)/(eigen_weights[3]**2)
        # print(f"DT: {DT}\nDG: {DG}\nDM: {DM}\nDV: {Dv}")
        # print(DT+DG+DM+Dv)
        return DT+DG+DM+Dv

    def stampaStella(self):
        with open(self.folder + ".star", 'w') as f:
            f.write(f"Nome: {self.nome}\n")
            f.write(f"T_eff: {self.t_eff}K\n")
            f.write(f"log(g): {self.logg}dex\n")
            f.write(f"[Fe/H]: {self.metal}dex\n")
            f.write(f"v_microturbulence: {self.v_m} km/s\n")
            f.write(f"Error: {self.param_error}\n")
            f.write("\n \n")

            f.write("ALL STARS: \n")
            for i in range(len(self.ALL_STARS)):
                f.write(f"{self.ALL_STARS[i]}\n")
star1 = Stella(nomi_fileFits[0])



Nome della stella: HD87740
La cartella '/home/starcat8/Scrivania/ANALISIATM/ABBONDANZE_OUT/HD87740' esiste già.


In [ ]:
star1.ALL_STARS

In [120]:
a=-1
if os.path.exists(star1.folder+"/TEMP/2HD87740_-0.23_4704_2.03_1.47"):
    print("Il file esiste.")
    a=1
else:
    print("Il file non esiste.")
    a=0

In [121]:
a

0

In [113]:
os.chdir(star1.folder+"/TEMP/")

In [71]:
stars[1].folder+"/TEMP"

'/home/starcat8/Scrivania/ANALISIATM/ABBONDANZE_OUT/HD91276/TEMP'

In [66]:
with open(path_to_output+ f'exit{i}.star', 'w') as f:
    for i in range(len(stars)):
        if i != 2:    
            try:
                a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m)
                b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
            except FileNotFoundError:
                a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, )
                b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
            F = a[1]-b[1]
            Y = stars[i].metal - a[0]
            S = a[3]
            #f.write(f"M: {a[0]}\ns(FeI): \nF: \nY: \nS: ")
            f.write(f"Nome: {stars[i].nome}\n")
            f.write(f"T: {stars[i].t_eff} \n")
            f.write(f"G: {stars[i].logg} \n")
            f.write(f"M: {stars[i].metal}\n")
            f.write(f"v: {stars[i].v_m}\n")
            f.write(f"Uncertainty: {stars[i].param_error}\n")
            f.write(f"s: {a[1]}\n")
            f.write(f"F: {F}\n")
            f.write(f"Y: {Y}\n")
            f.write(f"S: {S}\n")
            f.write("____________________________________________\n\n\n\n")
#stars[0].t_eff

In [106]:
#star1.evaluate_Models(stars[0].t_eff, stars[0].logg, stars[0].metal, stars[0].v_m)
star1.evaluate_Models(4684, 1.88, -0.59, 1.57)

np.float64(16.00967727230212)

In [ ]:
#riattiva_stampe()
i=2
try:
    a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m)
    b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
except FileNotFoundError:
    a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, )
    b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
F = a[1]-b[1]
Y = stars[i].metal - a[0]
S = a[3]
with open(stars[i].folder + '/UGS', 'w') as f:
    f.write(f"M: {a[0]}\ns(FeI): {a[1]}\nF: {F}\nY: {Y}\nS: {S}")

# for i in range(17):
#     if i != 3 and i != 7:
#         try:
#             a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m)
#             b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "2"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
#         except FileNotFoundError:
#             a=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, )
#             b=NLTE_AnalysisSingle(stars[i].folder + "/TEMP", "1"+stars[i].nome+f"_{stars[i].metal:.2f}_{stars[i].t_eff:.0f}_{stars[i].logg:.2f}_{stars[i].v_m:.2f}", stars[i].t_eff, stars[i].logg, stars[i].metal, stars[i].v_m, 26.1)
#         F = a[1]-b[1]
#         Y = stars[i].metal - a[0]
#         S = a[3]
#         with open(stars[i].folder + '/UGS', 'w') as f:
#             f.write(f"M: {a[0]}\ns(FeI): {a[1]}\nF: {F}\nY: {Y}\nS: {S}")
#(simpleAnalysisSingle(stars[0].folder + "/TEMP", "2"+stars[0].nome+f"_{stars[0].metal:.2f}_{stars[0].t_eff:.0f}_{stars[0].logg:.2f}_{stars[0].v_m:.2f}"))

In [25]:
stars = []
for i in range(1,3):
    if i != 4 and i != 7:
        star1 = Stella(nomi_fileFits[i])
        star1.searchParam2(12, TRIALS=100, t0=4001)#, t0=int(4001), g0=float(1.0), m0=float(-2.0))
        stars.append(star1)
        star1.stampaStella()

# star1 = Stella(nomi_fileFits[8])
# star1.searchParam1(10, TRIALS=1000, t0=4001)#, t0=int(4001), g0=float(1.0), m0=float(-2.0))
# stars.append(star1)
# star1.stampaStella()


Nome della stella: HD91276
La cartella '/home/starcat8/Scrivania/ANALISIATM/ABBONDANZE_OUT/HD91276' esiste già.
Correzione doppler!
File FITS aggiornato salvato come spettro_corretto.fits
mine.opt
number of options: 11
Input Parameters:

specfits: spettro_corretto.fits
readlinedat: Iron_lines
fileout: stella.ares
lambdai: 4600.0
lambdaf: 6700.0
smoothder: 4.0
space: 3.00 
rejt: 1;5764,5766,6047,6052,6068,6076

lineresol: 0.100
miniline: 2.0
plots_flag: 0
File: spettro_corretto.fits
FILE: spettro_corretto.fits
Extension: .fits - 0
READING FITS FILE: spettro_corretto.fits 
Ficheiro Aberto...Lendo ficheiro...
Cleaning zero gaps
Velocidade radial: 0.000000
ponto 1000, lambda: 3883.426650 
ponto 1000, lambda: 3883.426650 
Tree: 1;5764,5766,6047,6052,6068,6076

Computing rejt for spectrum...Nranges: 1

Refining search for central wavelenght: Inicial wavelenght: 5764.004907, index: 189073

Refining search for central wavelenght: Inicial wavelenght: 5766.004746, index: 189273
Range 0: [5764.00

mv: '/home/starcat8/Scrivania/ANALISIATM/spectrum277c/param.txt' e '/home/starcat8/Scrivania/ANALISIATM/spectrum277c/param.txt' sono lo stesso file


stellaFesiodPiodPAOOO!stellaFes 
Nome della stella: HD91276
mine.opt
number of options: 11
Input Parameters:

specfits: spettro_corretto.fits
readlinedat: Iron_lines
fileout: stella.ares
lambdai: 4600.0
lambdaf: 6700.0
smoothder: 4.0
space: 3.00 
rejt: 1;5764,5766,6047,6052,6068,6076

lineresol: 0.100
miniline: 2.0
plots_flag: 0
File: spettro_corretto.fits
FILE: spettro_corretto.fits
Extension: .fits - 0
READING FITS FILE: spettro_corretto.fits 
Ficheiro Aberto...Lendo ficheiro...
Cleaning zero gaps
Velocidade radial: 0.000000
ponto 1000, lambda: 3883.426650 
ponto 1000, lambda: 3883.426650 
Tree: 1;5764,5766,6047,6052,6068,6076

Computing rejt for spectrum...Nranges: 1

Refining search for central wavelenght: Inicial wavelenght: 5764.004907, index: 189073

Refining search for central wavelenght: Inicial wavelenght: 5766.004746, index: 189273
Range 0: [5764.000000 - 5766.000000] : SN: 145.219510
Median SN: 145.219510
Rejt used for computations: 0.993114
S/N  used for rejt: 145.219510
l

mv: '/home/starcat8/Scrivania/ANALISIATM/spectrum277c/param.txt' e '/home/starcat8/Scrivania/ANALISIATM/spectrum277c/param.txt' sono lo stesso file


stellaFesiodPiodPAOOO!stellaFes 
Nome della stella: HD91276
mine.opt
number of options: 11
Input Parameters:

specfits: spettro_corretto.fits
readlinedat: Iron_lines
fileout: stella.ares
lambdai: 4600.0
lambdaf: 6700.0
smoothder: 4.0
space: 3.00 
rejt: 1;5764,5766,6047,6052,6068,6076

lineresol: 0.100
miniline: 2.0
plots_flag: 0
File: spettro_corretto.fits
FILE: spettro_corretto.fits
Extension: .fits - 0
READING FITS FILE: spettro_corretto.fits 
Ficheiro Aberto...Lendo ficheiro...
Cleaning zero gaps
Velocidade radial: 0.000000
ponto 1000, lambda: 3883.426650 
ponto 1000, lambda: 3883.426650 
Tree: 1;5764,5766,6047,6052,6068,6076

Computing rejt for spectrum...Nranges: 1

Refining search for central wavelenght: Inicial wavelenght: 5764.004907, index: 189073

Refining search for central wavelenght: Inicial wavelenght: 5766.004746, index: 189273
Range 0: [5764.000000 - 5766.000000] : SN: 145.219510
Median SN: 145.219510
Rejt used for computations: 0.993114
S/N  used for rejt: 145.219510
l

...

stellaFesiodPiodPAOOO!stellaFes 
stellaFesiodPiodPAOOO!stellaFes 


INFO:root:Processo 8 completato con successo.
INFO:root:Processo 11 completato con successo.
INFO:root:Processo 9 completato con successo.


stellaFesiodPiodPAOOO!stellaFes 
stellaFesiodPiodPAOOO!stellaFes 
stellaFesiodPiodPAOOO!stellaFes 


INFO:root:Processo 6 completato con successo.
INFO:root:Tutti i processi sono stati completati.


stellaFesiodPiodPAOOO!stellaFes 
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
Valori univoci nella sesta colonna:
AIEEEEEEEIROA!
AIEEEEEEEIROA!
AIEEEEEEEIROA!
AIEEEEEEEIROA!
19.681672669106383
0.8659070057753089
24.204622359143677
12.897129945965101
4353.072185439288, 1.4152552933260831, -0.9938372493751876. 1.7435601051340182
4342.9585969766995, 1.436044514114313, -0.7481693400680226. 1.7112027467495552


In [4]:
star1.nome

'HD87740'

In [23]:
a=checkExist_newParam("1" + star1.nome, 4251, 0.87, -1.43, 1.84, star1.folder + "/TEMP")
print(a)
#-1.43_4251_0.87_1.85

[4251, 0.87, -1.43, 1.84]


In [ ]:
for W in range(15,16):
    star1 = Stella(nomi_fileFits[W])

In [ ]:
WONG=[]
for W in range(15, 16):
    star1 = Stella(nomi_fileFits[W])
    risultette1 = []
    file_path = star1.folder + "/TEMP"
    for i in range(80):
        for j in range(40):
            for k in range(20):
                for iu in range(40):
                    #print(file_path + f"/{star1.nome}_{(-1.88-0.01*k):.2f}_{(4380+i):.0f}_{(0.70+0.01*j):.2f}_{(1.90 + 0.01*iu):.2f}")
                    # print(f"{star1.nome}_{(-1.88-0.01*k):.2f}_{(4300+i):.0f}_{(0.58+0.01*j):.2f}_{(1.90 + 0.01*iu):.2f}")
                    if os.path.exists(file_path + f"/{star1.nome}_{(-1.88-0.01*k):.2f}_{(4380+i):.0f}_{(0.70+0.01*j):.2f}_{(1.90 + 0.01*iu):.2f}"):
                        #print("Il file esiste!")                                                                                                                                                                                                                                                                                       
                        try:
                            risultette1.append([star1.evaluate_Models(4380+i, 0.70+0.01*j, -1.88-0.01*k, 1.90 + 0.01*iu), 4380+i, 0.70+0.01*j, -1.88-0.01*k, 1.90 + 0.01*iu])
                        except FileNotFoundError:                                                                                                                                                                                                                               
                            #print(f"ma allora dé {i}")
                            aaaaaa=0
                        if j == 20 and k == 14 and iu == 14:
                            print(f"{i}")
                    else:
                        aaaA=231
                        #print("UN CE N'E'!")
    WONG.append(risultette1)

WONG=[]
for W in range(15, 16):
    star1 = Stella(nomi_fileFits[W])
    risultette = []
    file_path = star1.folder + "/TEMP"
    for i in range(80):
        for j in range(40):
            for k in range(40):
                for iu in range(40):
                    #print(file_path + f"/{star1.nome}_{(-1.88-0.01*k):.2f}_{(4380+i):.0f}_{(0.70+0.01*j):.2f}_{(1.90 + 0.01*iu):.2f}")
                    # print(f"{star1.nome}_{(-1.88-0.01*k):.2f}_{(4300+i):.0f}_{(0.58+0.01*j):.2f}_{(1.90 + 0.01*iu):.2f}")
                    if os.path.exists(file_path + f"/{star1.nome}_{(-1.98-0.01*k):.2f}_{(4180+i):.0f}_{(0.00+0.01*j):.2f}_{(2.00 + 0.01*iu):.2f}"):
                        #print("Il file esiste!")                                                                                                                                                                                                                                                                                       
                        try:
                            risultette.append([star1.evaluate_Models(4180+i, 0.00+0.01*j, -1.98-0.01*k, 2.00 + 0.01*iu), 4180+i, 0.00+0.01*j, -1.98-0.01*k, 2.00 + 0.01*iu])
                        except FileNotFoundError:                                                                                                                                                                                                                               
                            #print(f"ma allora dé {i}")
                            aaaaaa=0
                        if j == 20 and k == 14 and iu == 14:
                            print(f"{i}")
                    else:
                        aaaA=231
                        #print("UN CE N'E'!")
    #WONG.append(risultette)

In [ ]:
#os.path.exists(file_path + "/HD115575_-1.97_4332_0.75_1.99")
#print(file_path + "/HD115575_-1.97_4332_0.75_1.99")
risultette
ressi=[]

for i in range(len(risultette)):
    ressi.append(risultette[i][0])

print(risultette)

In [ ]:
lista_ordinata = sorted(risultette, key=lambda x: x[0])
lista_ordinata1 = sorted(risultette1, key=lambda x: x[0])

print("Lista ordinata1:", lista_ordinata)
print("Lista ordinata2:", lista_ordinata1)

In [ ]:
np.argmin(ressi)
risultette[458]

In [ ]:
def NLTE_wlCorrection(wl, el, T, G, M, V):
    DIR_INI = os.getcwd()
    M=M+7.5
    os.chdir(path_to_output)
    file_path = 'stellaFes'
    #lista_primi_elementi = crea_lista_primo_elemento(file_path)
    #print(lista_primi_elementi)
    correctionList = []
    PATH= path_to_correctionGrids1
    PATH=path_to_correctionGrids1
    if wl < 5000:
        PATH=PATH+"4500-5000/correctionGrid"
    elif wl > 5000 and wl < 5250:
        PATH=PATH+"5000-5250/correctionGrid"
    elif wl > 5250 and wl < 5500:
        PATH=PATH+"5250-5500/correctionGrid"
    elif wl > 5500 and wl < 5750:
        PATH=PATH+"5500-5750/correctionGrid"
    elif wl > 5750 and wl < 6000:
        PATH=PATH+"5750-6000/correctionGrid"
    elif wl > 6000 and wl < 6250:
        PATH=PATH+"6000-6250/correctionGrid"
    elif wl > 6250 and wl < 6500:
        PATH=PATH+"6250-6500/correctionGrid"
    elif wl > 6500 and wl < 6750:
        PATH=PATH+"6500-6750/correctionGrid"
    else:
        print("Nooo!")
    correctionList.append(findLine(f"{PATH}", T, G, M, wl, V))
        #print(correctionList[i])
    correctionList=[el for el in correctionList if el is not None]
    # print("AIUO")
    #print(correctionList)

    for i in range(len(correctionList)):
        if abs(wl - correctionList[i][6])<0.001:
            return correctionList[i][0]

#def NLTE_interpola()

def NLTE_intCorrection(wl, el, T, G, M, V):
    #maiuscole inserite, minuscole sulla griglia
    Tcheck = False
    Gcheck = False
    Mcheck = False
    Vcheck = False
    overT = []
    overG = []
    overM = []
    overV = []
    overT.append(NLTE_wlCorrection(wl, el, T, G, M, V)) #dà il valore della correzione nel punto più vicino sulla griglia
    overG.append(overT[0])
    overM.append(overT[0])
    overV.append(overT[0])
    t=find_closest(T, temperatura)
    g=find_closest(G, LOGG_arr)
    m=find_closest(M+7.5, METAL)-7.5
    v=find_closest(V, MICROTURB)
    if True:
        if (g==2.0) or (g==2.5):
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M
                m1=m


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print('ayyy lmao')
            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print("oyyy vey!")

            if G>g:
                g1=g+0.5
            elif G<g:
                g1=g-0.5
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==1.5:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t1=t


            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M+0.01
                m1=find_closest(M-0.25+7.5, METAL)-7.5


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print('ayyy lmao')
            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print("oyyy vey!")

            if G>g:
                g1=g+0.5
            elif G<g:
                print("Non c'è logg sotto 1.5dex e sopra 3dex!")
                g1=g
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==3.0:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M
                m1=m
            if m1 == 4.25 or m1 == 7.75:
                print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
                return 0

            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print('ayyy lmao')
            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati SOTTO i 0.75km/s e sopra i 3km/s!")
                    return 0
                else:
                    v1=v
                    print("oyyy vey!")

            if G<g:
                g1=g-0.5
            elif G>=g:
                print(f"Non c'è logg sotto 1.5dex e SOPRA 3dex! {G}, {g}")
                g1=g


        if t1 == 3750 or t1 == 5750:
            print('Non ci sono dati sotto i 4000K o sopra i 5500K!')
            #return 0
        if m1 == 4.25 or m1 == 7.75:
            print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
            return 0
        if g1==1.5 or g == 1.5:
            if t1 == 3750 or t1 == 4750:
                print('Non ci sono dati SOTTO i 4000K o sopra i 4500K! (per questo logg)')
        if g1 == 3 or g == 3:
            if t1 == 4250 or t1 == 5750:
                print('Non ci sono dati sotto i 4500K o SOPRA i 5500K! (per questo logg)')
                #return 0
            if g==3:
                if g1 > g:
                    print(f'Non ci sono dati sopra ai 3dex di logg {G}, {g}, {g1}')
        
    #print(v1)
    print(f"true  close next\n{T}, {t}, {t1}\n{G},  {g},  {g1}\n{M},  {m},  {m1}\n{V},  {v},  {v1}")
    if g==1.5 and t1>4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    if g==3.0 and t1<4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    # if G < 1.5:
    #     if t1>4500:    
    #         overG.append(NLTE_wlCorrection(wl, el, 4500, 2, M, V)) #
    overG.append(NLTE_wlCorrection(wl, el, T, g1, M, V))
    overM.append(NLTE_wlCorrection(wl, el, T, G, m1, V))
    overV.append(NLTE_wlCorrection(wl, el, T, G, M, v1))

    #print(f"{overT} --- {overG} --- {overM} --- {overV}")
    if t != t1:
        corrt=(overT[1]-overT[0])/(max(t1, t) - min(t1, t))*(T-t)
    else:
        corrt=0
    if g != g1:
        corrg=(overG[1]-overG[0])/(max(g1, g) - min(g1, g))*(G-g)
    else:
        corrg=0
    if m != m1:
        corrm=(overM[1]-overM[0])/(max(m1, m) - min(m1, m))*(M-m)
    else:
        corrm=0
    if v != v1:
        corrv=(overV[1]-overV[0])/(max(v1, v) - min(v1, v))*(V-v)
    else:
        corrv=0
    corrT=overT[0]+corrt
    corrG=overG[0]+corrg
    corrM=overM[0]+corrm
    corrV=overV[0]+corrv
    totalCorr = corrt + corrg + corrm + corrv
    #print(f"{t}, {t1}, eheh {overT}, {corrT}\n{g}, {g1}, eheh {overG}, {corrG}\n{m}, {m1}, eheh {overM}, {corrM}\n{v}, {v1}, eheh {overV}, {corrV}\n")
    #print(f"originale: {overT[0]}, \ninterpol: {overT[0]+totalCorr}\nDelta:{totalCorr}")
    return [overT[0], overT[0]+totalCorr, totalCorr]
#print(NLTE_intCorrection(5044.211, 26.0, 5000, 1.51, -1.01, 1.51))
#print(NLTE_wlCorrection(5044.211, 26.0, 4249, 2.01, -1.01, 1.13))



import matplotlib.cm as cm
X = []
Y=[]
S=[]
y = []
colormap = cm.get_cmap('inferno', 10)
norm = plt.Normalize(vmin=0, vmax=10)
for j in range(11):
    Y=[]
    X = []
    for i in range(200):
        X.append(4000.0+i*5)
        #X.append(1.51+0.1*i)
        print(f"{i+1}): {X[i]}")
        if NLTE_intCorrection(4903.309, 26.0, X[i], 0.0+j*0.25, -1.01, 2.51)!=0:
            Y.append(NLTE_intCorrection(4903.309, 26.0, X[i], 1.8+j*0.1, -1.01, 2.51)[1])
        else:
            print("UUGAS")
    plt.plot(X, Y, color=colormap(norm(j)))
# sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)  # Collegamento tra valori e colormap
# sm.set_array([])  # Necessario per generare la colorbar
# cbar = plt.colorbar(sm)  # Aggiunge la barra dei colori
# cbar.set_label('Valore parametro') 
plt.show()


In [ ]:
for i in range(220):
    NLTE_intCorrection(4630.119, 26.0, 4500, 0.70, -0.50, 1.5)[1]

In [ ]:
print(NLTE_wlCorrection(4630.119, 26.0, 5600, 1.51, -1.56, vturbo(4458, 1.54, -0.56)))

In [ ]:
def NLTE_caricaWLdisponibili():

In [ ]:
NLTE_intCorrection(5044.211, 26.0, 4249, 1.51, -2.81, 0.75)

In [ ]:
X = []
Y=[]
for i in range(12):
    #X.append(4249+i*50)
    X.append(1.51+0.1*i)
    print(f"{i+1}): {X[i]}")
    Y.append(NLTE_intCorrection(5044.211, 26.0, 4249, X[i], -1.01, 1.51)[1])
print(Y)
plt.plot(X, Y, color='cyan')
plt.show()

In [ ]:
# Example Inputs
y1 = [0.08, 0.1, -0.1, 0.6]
x1 = [4350, 1.5, -0.96, 1.6]
y2 = [0.09, 0.05, -0.13, 0.7]
x2 = [4379, 1.76, -1.12, 1.45]

# y1 = [np.float64(-0.18490570193990186), np.float64(-0.4551644007155634), np.float64(-0.5408924934581821), np.float64(8.788626719352498)]
# x1= [4422.659800808306, 2.046827709084474, -0.7643795995650736, 1.3154839604266206]
# y2 = [np.float64(-0.12292901130642779), np.float64(-0.4992631299734749), np.float64(-0.5908942574544997), np.float64(7.535652830620205)]
# x2= [4230.459530840902, 1.4012595951349482, -1.0804919494420522, 1.5793514417194723]
A_numpy = 0
# Create and display the matrix
A_numpy = crea_dynMatrix1(x1, y1, x2, y2)
if A_numpy is not None:
    print("Dynamic Matrix (A_numpy):")
    print(A_numpy)
    try:
        AI = np.linalg.pinv(A_numpy)
        print("Inverse of A_numpy (AI):")
        print(AI)
    except np.linalg.LinAlgError as e:
        print(f"Matrix inversion error: {e}")




# [[ 3.44827586e-04  3.84615385e-02 -6.25000000e-02 -6.66666667e-02]
#  [-1.72413793e-03 -1.92307692e-01  3.12500000e-01  3.33333333e-01]
#  [-1.03448276e-03 -1.15384615e-01  1.87500000e-01  2.00000000e-01]
#  [ 3.44827586e-03  3.84615385e-01 -6.25000000e-01 -6.66666667e-01]]

# [[ 3.44827586e-04  3.84615385e-02 -6.25000000e-02 -6.66666667e-02]
#  [-1.72413793e-03 -1.92307692e-01  3.12500000e-01  3.33333333e-01]
#  [-1.03448276e-03 -1.15384615e-01  1.87500000e-01  2.00000000e-01]
#  [ 3.44827586e-03  3.84615385e-01 -6.25000000e-01 -6.66666667e-01]]

In [ ]:
y1=[0.08, 0.1, -0.1, 0.6]
x1=[4350, 1.5, -0.96, 1.6]
y2=[0.09, 0.05, -0.13, 0.7]
x2=[4379, 1.76, -1.12, 1.45]
A_numpy=crea_dynMatrix(x1, y1, x2, y2)
print(A_numpy)
AI=np.linalg.inv(A_numpy)

In [ ]:
# [np.float64(-0.18490570193990186), np.float64(-0.4551644007155634), np.float64(-0.5408924934581821), np.float64(8.788626719352498)]
# [4422.659800808306, 2.046827709084474, -0.7643795995650736, 1.3154839604266206]
# [np.float64(-0.12292901130642779), np.float64(-0.4992631299734749), np.float64(-0.5908942574544997), np.float64(7.535652830620205)]
# [4230.459530840902, 1.4012595951349482, -1.0804919494420522, 1.5793514417194723]

for i in range(1, 2):
    star = Stella(nomi_fileFits[i])
    star.searchParam1(processori=10, TRIALS=50, NLTE=0)

In [ ]:
# star1.crea_log(FOLDER=star1.folder+"/TEMP", NLTE=1)
star1.evaluate_Models()

In [ ]:
os.chdir(path_to_output)
for i in range(10):
    t = Table.read(f"s{i}", format='ascii')
    y = t['col2']
    x = t['col1']

    plt.figure(figsize=(10, 6))
    plt.plot(x, y, label="Spettro synth-N")
    plt.xlabel("Lunghezza d'onda (Å)")
    plt.ylabel("Flusso")
    plt.title("Spettro Stellare - HARPS-N")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                star1.creaModel(minceData[9][1]-400+400*i, minceData[9][2]-0.5+0.5*j, minceData[9][3]+0.5-0.5*k, vturbo(minceData[9][1]-400+400*i, minceData[9][2]-0.5+0.5*j, minceData[9][3]+0.5-0.5*k), processore=(l+10*k+100*j+1000*i))

In [ ]:
print(star1.evaluate_Models(4239, 0.84, -0.90, 1.81, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4171, 0.84, -0.86, 1.85, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4188, 0.94, -0.83, 1.82, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4172, 0.84, -0.82, 1.72, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4145, 0.84, -0.82, 1.77, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4263, 0.84, -0.79, 1.91, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4208, 0.84, -0.79, 1.89, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4259, 0.84, -0.75, 1.77, star1.folder+"/TEMP"))
print(star1.evaluate_Models(4248, 0.84, -0.75, 1.83, star1.folder+"/TEMP"))

In [ ]:
star1.t_eff = 4342
star1.logg = 0.7
star1.metal = -1.98
star1.v_m = 1.94
star1.next_param()
#star1.fits_name

In [55]:
def evaluate_Model(self):
    '''Restituisce il nome della migliore atmosfera'''
    s = simpleAnalysisSingle()
    f=-0.0
    m = -0.0
    ewr = -0.0
    A1 = np.array([s, f, m, ewr])
    s = 0.00
    f=-0.01
    m = -0.0
    ewr = -0.0
    A2 = np.array([s, f, m, ewr])
    s = 0.00
    f=-0.0
    m = -0.01
    ewr = -0.0
    A3 = np.array([s, f, m, ewr])
    s = 0.001
    f=-0.0
    m = -0.0
    ewr = -0.1
    A4 = np.array([s, f, m, ewr])

In [ ]:

A = s_Matrix
V=1
AA=A.subs(v, V)
A_numpy = np.array(AA.tolist(), dtype=float)
A_inv = np.linalg.inv(A_numpy)
nlte_corrF=0#.2
nlte_corrS=0#-0.027
nlte_corrM=0#.1
s = 0.001
f=-0.0
m = -0.0
ewr = -0.0
A1 = np.array([s, f, m, ewr])
s = 0.00
f=-0.01
m = -0.0
ewr = -0.0
A2 = np.array([s, f, m, ewr])
s = 0.00
f=-0.0
m = -0.01
ewr = -0.0
A3 = np.array([s, f, m, ewr])
s = 0.001
f=-0.0
m = -0.0
ewr = -10.1
A4 = np.array([s, f, m, ewr])

eigen =[-2.64106774e+01, -5.11191519e-01, -3.76792106e-01, -9.78183094e-01]

eigen_weights = [25, 0.175, 0.082, 1]
#blocca_stampe()
#if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
    # DG = -1*temp0[1] / (dsdg + k * dfdt) 
    # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
if True:
    DT1 = (np.dot(A_inv, A1))[0]
    DG1 = (np.dot(A_inv, A1))[1]
    DM1 = (np.dot(A_inv, A1))[2]
    Dv1 = (np.dot(A_inv, A1))[3]
    DT2 = (np.dot(A_inv, A2))[0]
    DG2 = (np.dot(A_inv, A2))[1]
    DM2 = (np.dot(A_inv, A2))[2]
    Dv2 = (np.dot(A_inv, A2))[3]
    DT3 = (np.dot(A_inv, A3))[0]
    DG3 = (np.dot(A_inv, A3))[1]
    DM3 = (np.dot(A_inv, A3))[2]
    Dv3 = (np.dot(A_inv, A3))[3]
    DT4 = (np.dot(A_inv, A4))[0]
    DG4 = (np.dot(A_inv, A4))[1]
    DM4 = (np.dot(A_inv, A4))[2]
    Dv4 = (np.dot(A_inv, A4))[3]
    DT=DT1+DT2+DT3+DT4
    DG=DG1+DG2+DG3+DG4
    DM=DM1+DM2+DM3+DM4
    Dv=Dv1+Dv2+Dv3+Dv4
print(f"DT: {DT}\nDG: {DG}\nDM: {DM}\nDV: {Dv}")
if True:
    DT1 = abs((np.dot(A_inv, A1))[0])
    DG1 = abs((np.dot(A_inv, A1))[1])
    DM1 = abs((np.dot(A_inv, A1))[2])
    Dv1 = abs((np.dot(A_inv, A1))[3])
    DT2 = abs((np.dot(A_inv, A2))[0])
    DG2 = abs((np.dot(A_inv, A2))[1])
    DM2 = abs((np.dot(A_inv, A2))[2])
    Dv2 = abs((np.dot(A_inv, A2))[3])
    DT3 = abs((np.dot(A_inv, A3))[0])
    DG3 = abs((np.dot(A_inv, A3))[1])
    DM3 = abs((np.dot(A_inv, A3))[2])
    Dv3 = abs((np.dot(A_inv, A3))[3])
    DT4 = abs((np.dot(A_inv, A4))[0])
    DG4 = abs((np.dot(A_inv, A4))[1])
    DM4 = abs((np.dot(A_inv, A4))[2])
    Dv4 = abs((np.dot(A_inv, A4))[3])
    # if abs(DG) > 0.5:
        #     DG=0.5*(DG/abs(DG))
        # if abs(DT) > 250:
        #     DT = 250*(DT/abs(DT))
        # if abs(DM) > 0.25:
        #     DM = 0.25*(DM/abs(DM))
        # if abs(Dv) > 0.3:
        #     Dv = 0.3*(Dv/abs(Dv))
    DT=(DT1+DT2+DT3+DT4)/eigen_weights[0]
    DG=(DG1+DG2+DG3+DG4)/eigen_weights[1]
    DM=(DM1+DM2+DM3+DM4)/eigen_weights[2]
    Dv=(Dv1+Dv2+Dv3+Dv4)/eigen_weights[3]
print(f"DT: {DT}\nDG: {DG}\nDM: {DM}\nDV: {Dv}")
print(DT+DG+DM+Dv)


In [ ]:
ddt = -1
ddg = 0.0
ddm= 0.0
ddv = 0.0
ddA = np.array([ddt, ddg, ddm, ddv])
v=sp.symbols('v')
SMAT = s_Matrix.subs(v, ddv)
NP = np.array(SMAT.tolist(), dtype=float)
U = np.dot(NP, ddA)
# ddt = 1
# ddg = 0.015
# ddm = 0.01
# ddv = 0.001
# ddA = np.array([ddt, ddg, ddm, ddv])
# v=sp.symbols('v')
# SMAT = s_Matrix.subs(v, ddv)
# B = np.dot(SMAT, ddA)
print(U)

In [ ]:
star1.searchParam1()

In [ ]:
from sympy import symbols, Eq, solve

# Definire le variabili simboliche
x, y, z, w = symbols('x y z w')

# Scrivere le equazioni
eq1 = Eq(x + y + z + w, 10)
eq2 = Eq(2*x - y + z + 3*w, 5)
eq3 = Eq(x + 2*y - z + w, 7)
eq4 = Eq(3*x - y + 2*z - w, 3)

# Risolvere il sistema
soluzione = solve([eq1, eq2, eq3, eq4], (x, y, z, w))

print("Soluzione:")
print(soluzione)

In [ ]:
A=NP
print(type(A))
autovalori, autovettori = np.linalg.eig(A)

# Normalizzazione degli autovettori (NumPy li restituisce già normalizzati)
autovettori_normalizzati = autovettori / np.linalg.norm(autovettori, axis=0)

# Visualizzazione dei risultati
print("Autovalori:")
print(autovalori)

print("\nAutovettori normalizzati:")
print(autovettori_normalizzati)

# Ricostruzione della matrice diagonalizzata
D = np.diag(autovalori)  # Matrice diagonale con gli autovalori
P = autovettori_normalizzati  # Matrice degli autovettori come colonne
P_inv = np.linalg.inv(P)  # Matrice inversa di P

# Verifica della diagonalizzazione: P^(-1) * A * P = D
A_diagonalizzata = P @ D @ np.linalg.inv(P)
print("\nMatrice ricostruita (deve essere uguale ad A):")
print(A_diagonalizzata)


# Scrivere le equazioni0.00027375 -0.002228   -0.001255   -0.003307
eq1 = Eq(-2.73752000e-04*x  -3.53620000e-02*y  -1.50860000e-02*z + 2.40701100e-02*w, 0.00027375)
eq2 = Eq(2.22800000e-03*x -4.34498000e-01*y -4.02286000e-01*z  -4.44147195e-01*w, -0.002228)
eq3 = Eq(1.25500000e-03*x  -1.34194000e-01*y -1.15394200e+00*z -9.37120394e-01*w, -0.001255)
eq4 = Eq(3.30700000e-03*x  -6.97847200e+00*y  -2.93710700e+00*z -2.61599857e+01* w, -0.003307)
x, y, z, w = symbols('x y z w')
e=sp.Matrix([[-2.73752000e-04*x, -3.53620000e-02*y, -1.50860000e-02*z,  2.40701100e-02*w],
                [1.25500000e-03*x, -1.34194000e-01*y, -1.15394200e+00*z, -9.37120394e-01*w],
                [1.25500000e-03*x, -1.34194000e-01*y, -1.15394200e+00*z, -9.37120394e-01*w],
                [3.30700000e-03*x, -6.97847200e+00*y, -2.93710700e+00*z, -2.61599857e+01*w]])
# Risolvere il sistema
J=e.diagonalize()
soluzione = solve([eq1, eq2, eq3, eq4], (x, y, z, w))

print("Soluzione:")
print(soluzione)

In [ ]:
for i in range(10):
    star1.creaModel(4393, 1.08, -1.99, 1.1+0.1*i, processore=i)

In [ ]:
I = 14
star = Stella(minceData[I][0], -20, minceData[I][1], minceData[I][2], minceData[I][3], vturbo(minceData[I][1], minceData[I][2], minceData[I][3]))

In [15]:
from scipy.special import wofz

def voigt(x, sigma, gamma):
    """
    Calcola il profilo Voigtiano per un array x.
    
    Parametri:
    - x: array di valori per il quale calcolare il profilo Voigtiano
    - sigma: larghezza della componente Gaussiana
    - gamma: larghezza della componente Lorentziana
    
    Ritorna:
    - Il profilo Voigtiano calcolato per x
    """
    z = (x + 1j * gamma) / (sigma * np.sqrt(2))
    return np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))

def plot_voigt(x_range=(-10, 10), sigma=1.0, gamma=1.0):
    """
    Calcola e traccia il profilo Voigtiano in un intervallo specifico.
    
    Parametri:
    - x_range: tuple che definisce il range di x (xmin, xmax)
    - sigma: larghezza della componente Gaussiana
    - gamma: larghezza della componente Lorentziana
    """
    x = np.linspace(x_range[0], x_range[1], 500)
    voigt_profile = voigt(x, sigma, gamma)
    
    plt.figure(figsize=(8, 5))
    plt.plot(x, voigt_profile, label=f"Voigt ($\sigma={sigma}, \gamma={gamma}$)")
    plt.title("Profilo Voigtiano")
    plt.xlabel("x")
    plt.ylabel("Intensità")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.show()

# Esempio di utilizzo
plot_voigt(sigma=2.0, gamma=1.0)

In [ ]:
def fit_voigt(x_data, y_data):
    """
    Fitta i dati con una funzione Voigtiana.
    
    Parametri:
    - x_data: array di valori x dei dati
    - y_data: array di valori y dei dati
    
    Ritorna:
    - popt: array dei parametri ottimali [sigma, gamma, amplitude, center, offset]
    - pcov: matrice di covarianza dei parametri
    """
    # Stime iniziali per i parametri
    sigma_init = np.std(x_data) / 2  # Larghezza iniziale della Gaussiana
    gamma_init = sigma_init / 2      # Larghezza iniziale della Lorentziana
    amplitude_init = max(y_data)     # Altezza iniziale
    center_init = x_data[np.argmax(y_data)]  # Posizione centrale iniziale
    offset_init = min(y_data)        # Livello di offset iniziale

    # Fit dei dati
    initial_guess = [sigma_init, gamma_init, amplitude_init, center_init, offset_init]
    popt, pcov = curve_fit(voigt, x_data, y_data, p0=initial_guess)

    return popt, pcov

# Esempio di dati fittizi
x_data = np.linspace(-10, 10, 500)
true_params = [2.0, 1.0, 10.0, 0.0, 1.0]  # sigma, gamma, amplitude, center, offset
y_data = voigt(x_data, *true_params) + np.random.normal(0, 0.5, size=x_data.shape)

# Fit dei dati
popt, pcov = fit_voigt(x_data, y_data)

# Stampa dei risultati
print("Parametri trovati:")
print(f"Sigma: {popt[0]:.2f}, Gamma: {popt[1]:.2f}, Amplitude: {popt[2]:.2f}, Center: {popt[3]:.2f}, Offset: {popt[4]:.2f}")

# Grafico dei dati e del fit
plt.figure(figsize=(8, 5))
plt.scatter(x_data, y_data, s=5, color="blue", label="Dati (con rumore)")
plt.plot(x_data, voigt(x_data, *popt), color="red", label="Fit Voigtiano")
plt.title("Fit Voigtiano")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
Nome=nome_da_Fits()
print(type(Nome))
#star = Stella(minceData[0][0], Nome, 42, minceData[0][1], minceData[0][2], minceData[0][3], vturbo(minceData[0][1], minceData[0][2], minceData[0][3]))
star.creaModel()
star.searchParam()

In [ ]:
# Apri il file FITS
os.chdir(path_to_ARES)
file_fits = "HARPN14.fits"
hdul = fits.open(file_fits)

# Stampa informazioni sul file FITS
hdul.info()

# Accedi ai dati
data = hdul[0].data  # Tipicamente lo spettro si trova nella seconda estensione
header = hdul[0].header

# Esplora i dati
print(header) 

# Chiudi il file FITS
hdul.close()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Funzione per convertire l'equazione generica nella forma canonica
def parabola_to_canonical(a, b, c):
    h = -b / (2 * a)  # Ascissa del vertice
    k = a * h**2 + b * h + c  # Ordinata del vertice
    return h, k

# Funzione per generare la nuova parabola con il vertice specificato
def translate_parabola(a, new_h, new_k):
    returner = []
    # Equazione traslata: y = a(x - new_h)^2 + new_k
    def parabola(x):
        return a * (x - new_h)**2 + new_k
    returner.append()
    return parabola
def canonical_to_standard(a, h, k):
    """
    Converti l'equazione di una parabola dalla forma canonica
    y = a(x - h)^2 + k alla forma standard y = ax^2 + bx + c.
    
    Args:
        a (float): Curvatura della parabola (coefficiente di x^2).
        h (float): Ascissa del vertice.
        k (float): Ordinata del vertice.

    Returns:
        tuple: Coefficienti (a, b, c) nella forma standard.
    """
    b = -2 * a * h
    c = a * h**2 + k
    return a, b, c
# Parametri iniziali
a, b, c = 1, -4, 3  # Coefficienti iniziali: y = x^2 - 4x + 3

# Trova il vertice originale
original_h, original_k = parabola_to_canonical(a, b, c)

# Specifica il nuovo vertice
new_h, new_k = 2, 5

# Genera la parabola traslata
parabola = translate_parabola(a, new_h, new_k)

# Plot per confrontare
x = np.linspace(-2, 6, 500)  # Intervallo di x
original_parabola = a * x**2 + b * x + c
translated_parabola = parabola(x)

plt.figure(figsize=(8, 6))
plt.plot(x, original_parabola, label="Originale: y = x^2 - 4x + 3", linestyle="--")
plt.plot(x, translated_parabola, label=f"Traslata: Vertice ({new_h}, {new_k})", color="orange")
plt.scatter([original_h], [original_k], color="blue", label="Vertice originale", zorder=5)
plt.scatter([new_h], [new_k], color="red", label="Nuovo vertice", zorder=5)
plt.axhline(0, color="black", linewidth=0.5, linestyle="--")
plt.axvline(0, color="black", linewidth=0.5, linestyle="--")
plt.legend()
plt.grid()
plt.title("Traslazione di una parabola mantenendo la forma")
plt.xlabel("x")
plt.ylabel("y")
plt.show()


In [ ]:
class Stella:
    def __init__(self, fits_name='', nome='', rad_vel=0, t_eff=0, logg=0, metal=0):
        """
        Inizializza un oggetto Stella.
        
        :param nome: Nome della stella
        :param temperatura: Temperatura della stella (in Kelvin)
        :param spettro: Lista o array con i dati dello spettro
        """
        self.nome = nome
        self.t_eff = t_eff
        self.fits_name = fits_name
    
    def creaModel(self, T, logg, MH_g, microTurb, nome_Stella, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
        '''T: temperatura (kelvin)
        logg: gravità superficiale (dex)
        MH_g: [M/H] (dex)'''
        os.chdir(path_to_interpol)
        if processore==0:
            if n_istanze == 1:
                #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
                vturb=microTurb
                commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
                os.system(commandInterpol)
                #### FORMATTA KURUCZ ####
                header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
                header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
                tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
                def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                    with open(input_file, 'r') as f_in:
                        lines = f_in.readlines()
                    # Remove the first 3 rows and the last 2 rows
                    lines = lines[3:-2]
                    with open(output_file, 'w') as f_out:
                        f_out.write(header1)
                        f_out.write(header2)
                        f_out.writelines(lines)
                        f_out.write(tail)
                read_and_copy_file('modelatmosphere.txt', 'out.txt')
                comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
                os.system(comandoATMOS)
                #preparaAbundance(nome_Stella)
                os.chdir(path_to_abundance)
                creaParamfileAbnd(vturb)
                SlanciaAbundance()
                nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
                #comando_bsh2 = f"mv PINOLOS {path_to_calibrationFolder}/{nome_output}"# calibrazione
                comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}" # ricerca
                os.system(comando_bsh2)
                return nome_output
        else:
            #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
            for i in range(processore):
                vturb=microTurb[i]
                #print(f"{type(T)}   {type(logg)}   {type(MH_g)}   {type(vturb)}")
                os.chdir(path_to_abundance)
                password = "Birdos"
                commandInterpol = f"sudo {path_to_pythonEnv} {path_to_interpol}pykmod.py {T} {logg} {vturb} {MH_g}"
                # Usa subprocess per passare la password a sudo
                process = subprocess.run(
                    f"echo {password} | sudo -S {commandInterpol}",
                    shell=True,
                    text=True,  # Necessario per trattare input/output come stringhe
                    capture_output=True  # Per catturare stdout e stderr
                )
                #### FORMATTA KURUCZ ####
                header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
                MH_g=float(MH_g)
                header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
                tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
                def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                    with open(input_file, 'r') as f_in:
                        lines = f_in.readlines()
                    # Remove the first 3 rows and the last 2 rows
                    lines = lines[3:-2]
                    with open(output_file, 'w') as f_out:
                        f_out.write(header1)
                        f_out.write(header2)
                        f_out.writelines(lines)
                        f_out.write(tail)
                #print(f"aa {os.getcwd()}  ehiehi")
                os.chdir(path_to_interpol)
                read_and_copy_file('modelatmosphere.txt', 'out.txt')
                comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/Model{i+1}"
                os.system(comandoATMOS)
                creaParamfileAbnd(vturb, i+1)
            os.chdir(path_to_abundance)
            if __name__ == "__main__":
                main_integration()
            crea_cartella(path_to_calibrationFolder + "/" + nome_Stella)
            for i in range(processore):
                vturb=microTurb[i]
                nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
                copia("stellaFes", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, "stellaFes")
                sposta(f"PINOLOS{i+1}", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, nome_output)
            return nome_output
    
    def avvicina_interpol4(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
        #blocca_stampe()
        path_to_stella = self.folder
        if NLTE==0:
            temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
            temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
        elif NLTE == 1:
            temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
            temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
        sgn=1
        g=-1.0
        teff=-1.0
        met=1.0
        DG = 0
        DT = 0
        returner = []
        v=sp.symbols('v')
        # A = np.array([[dsdg, dsdt, dsdm],
        #      [dfdg, dfdt, dfdm],
        #      [dydg, dydt, dydm]])
        #riattiva_stampe()
        #sp.pprint(s_Matrix)
        A = s_Matrix
        AA=A.subs(v, V)
        A_numpy = np.array(AA.tolist(), dtype=float)
        A_inv = np.linalg.inv(A_numpy)
        f=-1*((temp0[0])-(temp1[0]))
        s = -1*(temp0[1])
        m = -1*(temp0[0]-(M))
        ewr = -1*(temp0[3])
        B = np.array([s, f, m, ewr])
        blocca_stampe()
        if abs(temp0[1]) > 0.0001 or abs((temp0[0])-(temp1[0])) > 0.0003 or abs(M - (temp0[0]))>0.001:
            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
            DT = (np.dot(A_inv, B))[0]
            DG = (np.dot(A_inv, B))[1]
            DM = (np.dot(A_inv, B))[2]
            Dv = (np.dot(A_inv, B))[3]
            if abs(DG) > 0.5:
                DG=0.5*(DG/abs(DG))
            if abs(DT) > 250:
                DT = 250*(DT/abs(DT))
            if abs(DM) > 0.5:
                DM = 0.5*(DM/abs(DM))
            if abs(Dv) > 0.3:
                Dv = 0.3*(Dv/abs(Dv))
            temperatu=temperatu+DT
            G=G+DG
            velturb=V+Dv
            teff= temperatu
            #temp0[0] = temp0[0]+DM
            crea_ATMinterpol2(teff, G, temp0[0]+nlte_corrM, velturb, nome_stella)
            temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", 26.0)[0]
            nome_file1 = f"AH_STELLA{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}"
            sposta(f"{nome_stella}_INT{(temp0[0]+nlte_corrM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
            g=G
            met = temp0[0]
        else:
            nome_file1 = nome_file
            teff = temperatu
        stampaAnalisi(path_to_output, nome_file1)
        returner.append(teff)
        returner.append(g)
        returner.append(met)
        returner.append(nome_file1)
        returner.append(velturb)
        return returner

    def abbondanza(self, T=-1, logg=-10, MH_g=10, microTurb=-1, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
        '''T: temperatura (kelvin)
        logg: gravità superficiale (dex)
        MH_g: [M/H] (dex)'''
        nome_Stella = self.fits_name
        if T==-1:
            T=self.t_eff
        if logg==-10:
            logg=self.logg
        os.chdir(path_to_interpol)
        if processore==0:
            if n_istanze == 1:
                #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
                vturb=microTurb
                commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
                os.system(commandInterpol)

In [ ]:
print(trova_dodicesimo_elemento(path_to_correctionGrids1+"5250-5500/correctionGrid", 4500, 1.5, 4.5, 5283.619 ))

In [ ]:
from astropy.io import fits

# Percorso del file FITS
file_fits = "HARPN1.fits"
os.chdir(path_to_ARES)

nome_da_Fits()

In [ ]:
os.chdir(path_to_ARES)
file_fits = "HARPN14.fits"
hdul = fits.open(file_fits)

# Stampa informazioni sul file FITS
hdul.info()

print(f"{len(hdul[0].data)} {type(hdul[0].data)}")

Y = hdul[0].data 
X = np.linspace(3875.321, 6911.574, len(Y))

plt.figure(figsize=(10, 6))
plt.plot(X, Y, label="Spettro HARPS-N")
plt.xlabel("Lunghezza d'onda (Å)")
plt.ylabel("Flusso")
plt.title("Spettro Stellare - HARPS-N")
plt.legend()
plt.grid()
plt.show()


In [112]:
def NLTE_caricaWLdisponibili():
    filename = "/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/4500-5000/correctionGrid"

    # Lista per i valori univoci
    unique_values = set()
    with open(filename, "r") as file:
        for line in file:
            # Ignora righe vuote o non formattate correttamente
            if line.strip():
                # Divide la riga in colonne
                columns = line.split()
                try:
                    # Estrae il valore della sesta colonna e lo aggiunge al set
                    unique_values.add(round(float(columns[5])*10, 3))
                except IndexError:
                    # Gestisce righe che non hanno abbastanza colonne
                    print(f"Riga non valida (meno di 6 colonne): {line.strip()}")
    # Lettura del file
    for i in range(7):
        filename = f"/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/{5000+250*i}-{5250+250*i}/correctionGrid"
        with open(filename, "r") as file:
            for line in file:
                # Ignora righe vuote o non formattate correttamente
                if line.strip():
                    # Divide la riga in colonne
                    columns = line.split()
                    try:
                        # Estrae il valore della sesta colonna e lo aggiunge al set
                        unique_values.add(round(float(columns[5])*10, 3))
                    except IndexError:
                        # Gestisce righe che non hanno abbastanza colonne
                        print(f"Riga non valida (meno di 6 colonne): {line.strip()}")

    # Converte il set in una lista ordinata
    unique_values_list = sorted(unique_values)

    # Stampa dei risultati
    print("Valori univoci nella sesta colonna:")
    return unique_values_list